In [1]:
import json
import torch
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from transformers import *
import torch.utils.data as Data
import collections
import os
import random
import tarfile
from torch import nn
import torchtext.vocab as Vocab
import pickle as pk
import torch.nn.functional as F
from IPython.display import display,HTML
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pack_sequence
from torch.nn import CrossEntropyLoss, MSELoss
from torchcrf import CRF
from sklearn import metrics
import joblib
import math
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import argparse
import glob
import logging
import unicodedata
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook as tqdm
import torch.utils.data as Data
import jieba
import jieba.posseg as pseg
import copy
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import KFold
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO 
)
ARG=collections.namedtuple('ARG',['train_batch_size',
 'eval_batch_size',
 'weight_decay',
 'learning_rate',
 'adam_epsilon',
 'num_train_epochs',
 'warmup_steps',
 'gradient_accumulation_steps',
 'save_steps',
 'max_grad_norm',
 'model_name_or_path',
 'output_dir',
 'seed',
 'device',
 'n_gpu',
 'max_steps',
 'output_mode',
'fp16_opt_level',
'fp16',
'card_list'])

unable to import 'smart_open.gcs', disabling that module
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

# 数据预处理

## 工具函数

In [8]:
#后处理的时候考虑：
#复杂类的value一定不能缺
#复杂类的其他ob跟value公用一个sub
#同一复杂类出现多次的时候怎么组织value和其他ob还是一个问题
#后续可以考虑再训练一个model，给定复杂类的sub ob的情况下，预测出其他附属属性

In [3]:
def convert_text_to_ids(tokenizer, text, max_len=100):
    if isinstance(text,str):
        output=tokenizer.encode_plus(text,max_length=max_len,pad_to_max_length=True,return_tensors="pt")
        input_ids=output["input_ids"].squeeze(0)
        token_type_ids=output["token_type_ids"].squeeze(0)
        attention_mask=output["attention_mask"].squeeze(0)
    elif isinstance(text,list):
        input_ids,token_type_ids,attention_mask=[],[],[]
        for e in text:
            output=tokenizer.encode_plus(e,max_length=max_len,pad_to_max_length=True,return_tensors="pt")
            input_ids.append(output["input_ids"].squeeze(0))
            token_type_ids.append(output["token_type_ids"].squeeze(0))
            attention_mask.append(output["attention_mask"].squeeze(0))
    else:
        raise Exception('type error')
    return torch.stack(input_ids).long(),torch.stack(token_type_ids).long(),torch.stack(attention_mask).long()        
class RelDataset(Data.Dataset):
    def __init__(self,examples):
        self.input_ids=torch.stack([e['input_ids'] for e in examples]).long()
        self.token_type_ids=torch.stack([e['token_type_ids'] for e in examples]).long()
        self.attention_mask=torch.stack([e['attention_mask'] for e in examples]).long()
        self.rel_label=torch.stack([e['rel_label'] for e in examples]).long()
        self.postag=torch.stack([e['postag'] for e in examples]).long()
        self.feature=torch.stack([e['feature'] for e in examples]).float()
        if examples[0]['token_vec'] is None:
            self.token_vec=np.zeros(len(examples))
            self.word_vec=np.zeros(len(examples))
            self.word_mask=np.zeros(len(examples))
        else:
            self.token_vec=np.stack([np.array(e['token_vec'])  for e in examples])
            self.word_vec=np.stack([np.array(e['word_vec']) for e in examples])
            self.word_mask=np.stack([np.array(e['word_mask']) for e in examples])
        self.plan_label=np.stack([np.array(e['plan_label']) for e in examples])
        self.token2docs=[e["token2doc"] for e in examples]
    def __len__(self):
        return self.input_ids.shape[0]
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx],self.token_type_ids[idx],\
    self.rel_label[idx],self.postag[idx],self.feature[idx],self.token_vec[idx],self.word_vec[idx],\
    self.word_mask[idx],self.plan_label[idx],self.token2docs[idx]  
class NerDataset(Data.Dataset):
    def __init__(self,examples):
        self.input_ids=torch.stack([e['input_ids'] for e in examples]).long()
        self.token_type_ids=torch.stack([e['token_type_ids'] for e in examples]).long()
        self.attention_mask=torch.stack([e['attention_mask'] for e in examples]).long()
        self.rel_label=torch.stack([e['rel_label'] for e in examples]).long()
        self.labels=torch.stack([e['labels'] for e in examples]).long()
        self.postag=torch.stack([e['postag'] for e in examples]).long()
        self.feature=torch.stack([e['feature'] for e in examples]).float()
        self.token2docs=[e["token2doc"] for e in examples]
    def __len__(self):
        return self.input_ids.shape[0]
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx],self.token_type_ids[idx],\
               self.rel_label[idx],self.labels[idx],self.postag[idx],self.feature[idx],self.token2docs[idx]  
class NerDataset_pointer(Data.Dataset):
    def __init__(self,examples):
        self.input_ids=torch.stack([e['input_ids'] for e in examples]).long()
        self.token_type_ids=torch.stack([e['token_type_ids'] for e in examples]).long()
        self.attention_mask=torch.stack([e['attention_mask'] for e in examples]).long()
        self.rel_label=torch.stack([e['rel_label'] for e in examples]).long()
        self.labels=torch.stack([e['labels'] for e in examples]).long()
        self.postag=torch.stack([e['postag'] for e in examples]).long()
        self.feature=torch.stack([e['feature'] for e in examples]).float()
        self.plan_labels=torch.stack([e['plan_labels'] for e in examples]).long()
        self.token2docs=[e["token2doc"] for e in examples]
    def __len__(self):
        return self.input_ids.shape[0]
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx],self.token_type_ids[idx],\
               self.rel_label[idx],self.labels[idx],self.postag[idx],self.feature[idx],\
               self.plan_labels[idx],self.token2docs[idx]  
import unicodedata
def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False
def is_chinese_char(cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if (
        (cp >= 0x4E00 and cp <= 0x9FFF)
        or (cp >= 0x3400 and cp <= 0x4DBF)  #
        or (cp >= 0x20000 and cp <= 0x2A6DF)  #
        or (cp >= 0x2A700 and cp <= 0x2B73F)  #
        or (cp >= 0x2B740 and cp <= 0x2B81F)  #
        or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
        or (cp >= 0xF900 and cp <= 0xFAFF)
        or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
    ):  #
        return True

    return False
def is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
def cut_buff(info):
    buff=""
    tokens=[]
    for e in info:
        if (e.isdigit() and (buff.isdigit() or buff=='')):
            buff+=e
        elif (e.isalpha() and (buff.isalpha() or buff=='')):
            buff+=e
        else:
            if buff!="":
                tokens.append(buff)
            buff=""
            if not (e.isdigit() or e.isalpha()):
                tokens.append(e)
            else:
                buff+=e

    if buff!="":
        tokens.append(buff)
    return tokens
def build_tfidf_svd_matrix(texts,n_output,tfidf_vec=None,svd=None):
    corpus=[]
    for text in tqdm(texts):
#         print(text)
        words=word_segment(str(text['text']))
#         print(words)
        use_words=[]
        for word in words:
            if word in stop_words:
                continue
            use_words.append(word)
#         print(use_words)
        corpus.append(" ".join(use_words))
    print(len(corpus))
    print(corpus[0])
    if tfidf_vec is None:
        tfidf_vec=TfidfVectorizer()
        tfidf_matrix=tfidf_vec.fit_transform(corpus)
    else:
        tfidf_matrix=tfidf_vec.transform(corpus)
    if svd is None:
        svd=TruncatedSVD(n_components=n_output,n_iter=7,random_state=42)
        tf_idf_svd=svd.fit_transform(tfidf_matrix)
    else:
        tf_idf_svd=svd.transform(tfidf_matrix)
    return tf_idf_svd,tfidf_vec,svd
def word_segment(sentence):
    words=jieba.cut(sentence)
    return ",".join(words).split(",")
stop_words=set()
def load_stopwords():
    with open("./middle_data/stopwords.txt","r",encoding="UTF-8") as r:
        for line in r.readlines():
            stop_words.add(line.strip())
load_stopwords()
def remove_stopwords(word_list):
    res=[]
    for word in word_lists:
        if word not in stop_words:
            res.append(word)
    return ' '.join(res)
def clean_text(string):
    return string.replace(' ', '').replace('\n', '').replace('\u3000', '')
def special_tokenize(text_raw,tokenizer):
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.extend(cut_buff(buff))
                buff = ""
            sub_text.append(char)

        else:
            buff += char
    if buff != "":
        sub_text.extend(cut_buff(buff))
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        sub_tokens = tokenizer.tokenize(token)
        for sub_token in sub_tokens:
            tokens.append(sub_token)
        else:
            continue
        break 
    return tokens
def _convert_example_to_cls_record(example,
                               tokenizer):
    if example.__contains__('spo_list'):
        spo_list = example['spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.append(buff)
                buff = ""
            sub_text.append(char)
        else:
            buff += char
    if buff != "":
        sub_text.append(buff)
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels_list=[]
    rel_labels=[]
    for spo in spo_list:
        for spo_object in spo['object'].keys():
            labels = [0
                  for i in range(len(tokens))]  # initialize tag
            if not spo['predicate'] in special_rels:
                rel_label=rels2id[spo["predicate"]]
            else:
                rel_label=rels2id[spo["predicate"]+"_"+spo_object] 
            rel_labels.append(rel_label)

    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,rel_labels
def _convert_example_to_record(example,
                               tokenizer):
    if example.__contains__('spo_list'):
        spo_list = example['spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.extend(cut_buff(buff))
                buff = ""
            sub_text.append(char)

        else:
            buff += char
    if buff != "":
        sub_text.extend(cut_buff(buff))
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels_list=[[0
              for i in range(len(tokens))] for i in range(len(id2rels))]
    rel_labels=[]
    for spo in spo_list:
        for spo_object in spo['object'].keys():
            if not spo['predicate'] in special_rels:
                rel_label=rels2id[spo["predicate"]]
            else:
                rel_label=rels2id[spo["predicate"]+"_"+spo_object] 
            labels = labels_list[rel_label] #复杂类的不同part还是不会被归到一个，以后再讲
            label_subject = label2ids['B-SUB']
            label_object = label2ids['B-OBJ']
            subject_sub_tokens = special_tokenize(spo['subject'],tokenizer)
#             print("sub",subject_sub_tokens)
            object_sub_tokens = special_tokenize(spo['object'][
                spo_object],tokenizer)
            forbidden_index = None
            if len(subject_sub_tokens) > len(object_sub_tokens):
                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        labels[index]=label_subject
                        for i in range(len(subject_sub_tokens) - 1):
                            labels[index + i + 1]=label_subject+1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        if forbidden_index is None:
                            labels[index]=label_object
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1]=label_object+1
                            break
                        # check if labeled already
                        elif index < forbidden_index or index >= forbidden_index + len(
                                subject_sub_tokens):
                            labels[index]=label_object
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1]=label_object+1
                            break

            else:
                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        labels[index]=label_object
                        for i in range(len(object_sub_tokens) - 1):
                            labels[index + i + 1]=label_object+1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        if forbidden_index is None:
                            labels[index]=label_subject
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1]=label_subject+1
                            break
                        elif index < forbidden_index or index >= forbidden_index + len(
                                object_sub_tokens):
                            labels[index]=label_subject
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1]=label_subject+1
                            break
            labels_list[rel_label]=labels
            if rel_label not in rel_labels:
                rel_labels.append(rel_label)

    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels
def _convert_example_to_record_new(example,
                               tokenizer):
    if example.__contains__('spo_list'):
        spo_list = example['spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.extend(cut_buff(buff))
                buff = ""
            sub_text.append(char)

        else:
            buff += char
    if buff != "":
        sub_text.extend(cut_buff(buff))
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels_list=[[[0] * len(id2labels)
              for i in range(len(tokens))] for i in range(len(id2rels))]
    rel_labels=[]
    for spo in spo_list:
        for spo_object in spo['object'].keys():
            if not spo['predicate'] in special_rels:
                rel_label=rels2id[spo["predicate"]]
            else:
                rel_label=rels2id[spo["predicate"]+"_"+spo_object] 
            labels = labels_list[rel_label] #复杂类的不同part还是不会被归到一个，以后再讲
            label_subject = new_label2ids['B-SUB']
            label_object = new_label2ids['B-OBJ']
            subject_sub_tokens = special_tokenize(spo['subject'],tokenizer)
#             print("sub",subject_sub_tokens)
            object_sub_tokens = special_tokenize(spo['object'][
                spo_object],tokenizer)
            forbidden_index = None
            if len(subject_sub_tokens) > len(object_sub_tokens):
                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        labels[index][label_subject]+=1
                        labels[index + len(subject_sub_tokens)-1][label_subject+1]+=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_object]+=1
                            labels[index + len(object_sub_tokens)-1][label_object+1]+=1
                            break
                        # check if labeled already
                        elif index < forbidden_index or index >= forbidden_index + len(
                                subject_sub_tokens):
                            labels[index][label_object]+=1
                            labels[index + len(object_sub_tokens)-1][label_object+1]+=1
                            break

            else:
                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        labels[index][label_object]+=1
                        labels[index + len(object_sub_tokens)-1][label_object+1]+=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_subject]+=1
                            labels[index + len(subject_sub_tokens)-1][label_subject+1]+=1
                            break
                        elif index < forbidden_index or index >= forbidden_index + len(
                                object_sub_tokens):
                            labels[index][label_subject]+=1
                            labels[index + len(subject_sub_tokens)-1][label_subject+1]+=1
                            break
            labels_list[rel_label]=labels
            if rel_label not in rel_labels:
                rel_labels.append(rel_label)

    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels
def create_cls_example(tokens,rel,token2doc,postag,tfidf_svd,word_vec,token_vec,plan_label,tokenizer):
#         word_vec=copy.deepcopy(word_vec)
        word_vec=None
        word_mask=None
        token_vec=copy.deepcopy(token_vec)
        if len(tokens)>max_len-2:
                tokens=tokens[:(max_len-2)]
                plan_label=plan_label[:(max_len-2)]
                token2doc=[e[:(max_len-2)] if ind<2 else e for ind,e in enumerate(token2doc)]
        
        tag=[postag[token2doc[0][idx]] for idx in range(len(tokens))]
        tokenvec=None
#         tokenvec=[token_vec[token2doc[0][idx]] for idx in range(len(tokens))]
        tag=[pos2id_BIO[tokenizer.cls_token]]+tag+[pos2id_BIO[tokenizer.sep_token]]
#         tokenvec=[word2id[tokenizer.cls_token]]+tokenvec+[word2id[tokenizer.sep_token]]
        full_tokens=[tokenizer.cls_token]+tokens+[tokenizer.sep_token]
        padding_O=[0]*len(id2kglabels)
        padding_O[0]=1
        full_plan_labels=[padding_O]+plan_label+[padding_O]
        token_type_ids=[0]*len(full_tokens)
        attention_mask=[1]*len(token_type_ids)
        cur_len=len(full_tokens)
        if cur_len<max_len:
            full_tokens+=[tokenizer.pad_token]*(max_len-cur_len)
            full_plan_labels+=[padding_O]*(max_len-cur_len)
            attention_mask+=[0]*(max_len-cur_len)
            token_type_ids+=[0]*(max_len-cur_len)
            tag+=[pos2id_BIO[tokenizer.pad_token]]*(max_len-cur_len)
#             tokenvec+=[word2id[tokenizer.pad_token]]*(max_len-cur_len)

#         if len(word_vec)>max_len//2:
#             word_vec=word_vec[:max_len//2]
#         word_mask=[1]*len(word_vec)
#         if len(word_vec)<max_len//2:
#             word_mask+=[0]*(max_len//2-len(word_vec))
#             word_vec+=[word2id[tokenizer.pad_token]]*(max_len//2-len(word_vec))

        full_ids=tokenizer.convert_tokens_to_ids(full_tokens)
        if len(rel)>0:
            example={"input_ids":torch.tensor(full_ids,dtype=torch.long),"token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),\
                    "attention_mask":torch.tensor(attention_mask,dtype=torch.long),
                    "rel_label":(F.one_hot(torch.tensor(rel),len(id2rels)).sum(dim=0)!=0).long(),
                     "postag":torch.tensor(tag).long(),"feature":torch.tensor(tfidf_svd).float(),\
                     "token_vec":tokenvec,"word_vec":word_vec,"word_mask":word_mask,"plan_label":full_plan_labels,"token2doc":token2doc}
        else:
            example={"input_ids":torch.tensor(full_ids,dtype=torch.long),"token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),\
                    "attention_mask":torch.tensor(attention_mask,dtype=torch.long),
                     "rel_label":torch.zeros(len(id2rels)).long(),
                     "postag":torch.tensor(tag).long(),"feature":torch.tensor(tfidf_svd).float(),\
                     "token_vec":tokenvec,"word_vec":word_vec,"word_mask":word_mask,"plan_label":full_plan_labels,"token2doc":token2doc}  
        return example
def create_example(tokens,rel,labels,token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc):
        tag=[postag[token2doc[0][idx]] for idx in range(len(tokens))]
        tag=[pos2id_BIO[tokenizer.cls_token]]+tag+[pos2id_BIO[tokenizer.sep_token]]
        rel_tag=[rel_postag[rel_token2doc[idx]] for idx in range(len(rel_text))]
        rel_tag=rel_tag+[pos2id_BIO[tokenizer.sep_token]]
        tag=tag+rel_tag
        second_token=rel_text
        full_tokens=[[tokenizer.cls_token]+tokens+[tokenizer.sep_token],second_token+[tokenizer.sep_token]]
        full_labels=[[label2ids[tokenizer.cls_token]]+labels+[label2ids[tokenizer.sep_token]],[label2ids["[category]"]]*len(second_token)+[label2ids[tokenizer.sep_token]]]
        token_type_ids=[0]*len(full_tokens[0])+[1]*len(full_tokens[1])
        attention_mask=[1]*len(token_type_ids)
        full_tokens=full_tokens[0]+full_tokens[1]
        full_labels=full_labels[0]+full_labels[1]
        cur_len=len(full_labels)
        if cur_len<max_len:
            full_tokens+=[tokenizer.pad_token]*(max_len-cur_len)
            full_labels+=[label2ids[tokenizer.pad_token]]*(max_len-cur_len)
            attention_mask+=[0]*(max_len-cur_len)
            token_type_ids+=[0]*(max_len-cur_len)
            tag+=[pos2id_BIO[tokenizer.pad_token]]*(max_len-cur_len)
        full_ids=tokenizer.convert_tokens_to_ids(full_tokens)
        example={"input_ids":torch.tensor(full_ids,dtype=torch.long),"token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),\
                "attention_mask":torch.tensor(attention_mask,dtype=torch.long),"labels":torch.tensor(full_labels,dtype=torch.long),
                "rel_label":F.one_hot(torch.tensor(rel),num_classes=len(id2rels)),"postag":torch.tensor(tag).long(),"feature":torch.tensor(tfidf_svd).float(),\
                 "token2doc":token2doc}
        return example
def create_example_new(tokens,rel,labels,token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc,plan_labels):
        tag=[postag[token2doc[0][idx]] for idx in range(len(tokens))]
        tag=[pos2id_BIO[tokenizer.cls_token]]+tag+[pos2id_BIO[tokenizer.sep_token]]
        rel_tag=[rel_postag[rel_token2doc[idx]] for idx in range(len(rel_text))]
        rel_tag=rel_tag+[pos2id_BIO[tokenizer.sep_token]]
        tag=tag+rel_tag
        second_token=rel_text
        full_tokens=[[tokenizer.cls_token]+tokens+[tokenizer.sep_token],second_token+[tokenizer.sep_token]]
        full_labels=[[padding_CLS]+labels+[padding_SEP],[padding_category]*len(second_token)+[padding_SEP]]
        full_plan_labels=[[padding_CLS]+plan_labels+[padding_SEP],[padding_category]*len(second_token)+[padding_SEP]]
        token_type_ids=[0]*len(full_tokens[0])+[1]*len(full_tokens[1])
        attention_mask=[1]*len(token_type_ids)
        full_tokens=full_tokens[0]+full_tokens[1]
        full_labels=full_labels[0]+full_labels[1]
        full_plan_labels=full_plan_labels[0]+full_plan_labels[1]
        cur_len=len(full_labels)
        if cur_len<max_len:
            full_tokens+=[tokenizer.pad_token]*(max_len-cur_len)
            full_labels+=[padding_PAD]*(max_len-cur_len)
            full_plan_labels+=[padding_PAD]*(max_len-cur_len)
            attention_mask+=[0]*(max_len-cur_len)
            token_type_ids+=[0]*(max_len-cur_len)
            tag+=[pos2id_BIO[tokenizer.pad_token]]*(max_len-cur_len)
        full_ids=tokenizer.convert_tokens_to_ids(full_tokens)
        example={"input_ids":torch.tensor(full_ids,dtype=torch.long),"token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),\
                 "attention_mask":torch.tensor(attention_mask,dtype=torch.long),"labels":torch.tensor(full_labels,dtype=torch.long),
                "rel_label":F.one_hot(torch.tensor(rel),num_classes=len(id2rels)),"postag":torch.tensor(tag).long(),"feature":torch.tensor(tfidf_svd).float(),\
                 "plan_labels":torch.tensor(full_plan_labels,dtype=torch.long),"token2doc":token2doc}
        return example
def index_token(text_raw):
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.append(buff)
                buff = ""
            sub_text.append(char)
        else:
            buff += char
    if buff != "":
        sub_text.append(buff)
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    return tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index
def _convert_example_to_cls_plan_labels(example,
                               tokenizer):
    if example.__contains__('plan_spo_list'):
        spo_list = example['plan_spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.append(buff)
                buff = ""
            sub_text.append(char)
        else:
            buff += char
    if buff != "":
        sub_text.append(buff)
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels = [[0] * len(id2kglabels)
              for i in range(len(tokens))]  # initialize tag
    for spo in spo_list:
            label_subject = kglabels2id['B-'+spo['predicate']+'-SUB']
            label_object = kglabels2id['B-'+spo['predicate']+'-OB']
            subject_sub_tokens = tokenizer.tokenize(spo['subject'])
            object_sub_tokens = tokenizer.tokenize(spo['object'])
            forbidden_index = None
            if len(subject_sub_tokens) > len(object_sub_tokens):
                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        labels[index][label_subject]=1
                        for i in range(len(subject_sub_tokens) - 1):
                            labels[index+i+1][kglabels2id['I']]=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_object]=1
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break
                        # check if labeled already
                        elif index < forbidden_index or index >= forbidden_index + len(
                                subject_sub_tokens):
                            labels[index][label_object]=1
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break

            else:
                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        labels[index][label_object]=1
                        for i in range(len(object_sub_tokens) - 1):
                            labels[index + i + 1][kglabels2id['I']]=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_subject]=1
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break
                        elif index < forbidden_index or index >= forbidden_index + len(
                                object_sub_tokens):
                            labels[index][label_subject]=1
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break
    return labels
def _convert_example_to_plan_labels(example,
                               tokenizer):
    if example.__contains__('plan_spo_list'):
        spo_list = example['plan_spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.extend(cut_buff(buff))
                buff = ""
            sub_text.append(char)

        else:
            buff += char
    if buff != "":
        sub_text.extend(cut_buff(buff))
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels = [[0] * len(new_id2kglabels)
              for i in range(len(tokens))]  # initialize tag
    for spo in spo_list:
            label_subject = kglabels2id['B-'+spo['predicate']+'-SUB']
            label_object = kglabels2id['B-'+spo['predicate']+'-OB']
            subject_sub_tokens = special_tokenize(spo['subject'],tokenizer)
            object_sub_tokens = special_tokenize(spo['object'],tokenizer)
            forbidden_index = None
            if len(subject_sub_tokens) > len(object_sub_tokens):
                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        labels[index][label_subject]=1
                        for i in range(len(subject_sub_tokens) - 1):
                            labels[index+i+1][kglabels2id['I']]=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_object]=1
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break
                        # check if labeled already
                        elif index < forbidden_index or index >= forbidden_index + len(
                                subject_sub_tokens):
                            labels[index][label_object]=1
                            for i in range(len(object_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break

            else:
                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        labels[index][label_object]=1
                        for i in range(len(object_sub_tokens) - 1):
                            labels[index + i + 1][kglabels2id['I']]=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_subject]=1
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break
                        elif index < forbidden_index or index >= forbidden_index + len(
                                object_sub_tokens):
                            labels[index][label_subject]=1
                            for i in range(len(subject_sub_tokens) - 1):
                                labels[index + i + 1][kglabels2id['I']]=1
                            break
    return labels
def _convert_example_to_plan_labels_new(example,
                               tokenizer):
    if example.__contains__('plan_spo_list'):
        spo_list = example['plan_spo_list']
    else:
        spo_list = []
    text_raw = example['text']
    sub_text = []
    buff = ""
    for char in text_raw:
        if is_chinese_char(ord(char)) or is_punctuation(char):
            if buff != "":
                sub_text.extend(cut_buff(buff))
                buff = ""
            sub_text.append(char)

        else:
            buff += char
    if buff != "":
        sub_text.extend(cut_buff(buff))
    tok_to_orig_start_index = []
    tok_to_orig_end_index = []
    orig_to_tok_index = []
    tokens = []
    text_tmp = ''
    for (i, token) in enumerate(sub_text):
        orig_to_tok_index.append(len(tokens))
        sub_tokens = tokenizer.tokenize(token)
        text_tmp += token
        for sub_token in sub_tokens:
            tok_to_orig_start_index.append(len(text_tmp) - len(token))
            tok_to_orig_end_index.append(len(text_tmp) - 1)
            tokens.append(sub_token)
        else:
            continue
        break
    #  find all entities and tag them with corresponding "B"/"I" labels
    labels = [[0] * len(new_id2kglabels)
              for i in range(len(tokens))]  # initialize tag
    for spo in spo_list:
            label_subject = new_kglabels2id['B-'+spo['predicate']+'-SUB']
            label_object = new_kglabels2id['B-'+spo['predicate']+'-OB']
            subject_sub_tokens = special_tokenize(spo['subject'],tokenizer)
            object_sub_tokens = special_tokenize(spo['object'],tokenizer)
            forbidden_index = None
            if len(subject_sub_tokens) > len(object_sub_tokens):
                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        labels[index][label_subject]+=1
                        labels[index+len(subject_sub_tokens)-1][new_kglabels2id['E-'+spo['predicate']+'-SUB']]+=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_object]+=1
                            labels[index+len(object_sub_tokens)-1][new_kglabels2id['E-'+spo['predicate']+'-OB']]+=1
                            break
                        # check if labeled already
                        elif index < forbidden_index or index >= forbidden_index + len(
                                subject_sub_tokens):
                            labels[index][label_object]+=1
                            labels[index+len(object_sub_tokens)-1][new_kglabels2id['E-'+spo['predicate']+'-OB']]+=1
                            break

            else:
                for index in range(
                        len(tokens) - len(object_sub_tokens) + 1):
                    if tokens[index:index + len(
                            object_sub_tokens)] == object_sub_tokens:
                        labels[index][label_object]+=1
                        labels[index+len(object_sub_tokens)-1][new_kglabels2id['E-'+spo['predicate']+'-OB']]+=1
                        forbidden_index = index
                        break

                for index in range(
                        len(tokens) - len(subject_sub_tokens) + 1):
                    if tokens[index:index + len(
                            subject_sub_tokens)] == subject_sub_tokens:
                        if forbidden_index is None:
                            labels[index][label_subject]+=1
                            labels[index+len(subject_sub_tokens)-1][new_kglabels2id['E-'+spo['predicate']+'-SUB']]+=1
                            break
                        elif index < forbidden_index or index >= forbidden_index + len(
                                object_sub_tokens):
                            labels[index][label_subject]+=1
                            labels[index+len(subject_sub_tokens)-1][new_kglabels2id['E-'+spo['predicate']+'-SUB']]+=1
                            break
    return labels


## 加载模型

In [4]:
do_lower_case=True
max_len=256
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# bert_dir="./bert-pytorch-chinese/"
# vocab="bert-base-chinese-vocab.txt"
# config_file="bert_config.json"
bert_dir="./roberta-zh-wwm-pytorch/"
vocab="vocab.txt"
config_file="bert_config.json"
tokenizer=BertTokenizer.from_pretrained(os.path.join(bert_dir,vocab),do_lower_case=do_lower_case)


06/18/2020 21:17:29 - INFO - transformers.tokenization_utils -   Model name './roberta-zh-wwm-pytorch/vocab.txt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming './roberta-zh-wwm-pytorch/vocab.txt' is a path, a model identifier, or url to a directory containing tokenizer files.
06/18/2020 21:17:29 - WARNING - transformers.tokenization_utils -   Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
06/18/2020 21:17:29 - 

## 读取数据构建映射表

In [5]:
text_data=[]
with open("./dataset/train_data/new_train_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        text_data.append(json.loads(d))
test_text_data=[]
with open("./dataset/test1_data/new_test1_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        test_text_data.append(json.loads(d))
test2_text_data=[]
with open("./dataset/test2_data/new_test2_data.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        test2_text_data.append(json.loads(d))
schema=[]
with open("./dataset/schema.json","r") as r:
    raw_schema=r.readlines()
    for d in raw_schema:
        schema.append(json.loads(d))
rels=set()
special_rels=set()
for e in schema:
        if len(e['object_type'].keys())==1:
            rels.add(e["predicate"])
        else:
            special_rels.add(e["predicate"])
            for key in e['object_type'].keys():
                rels.add(e['predicate']+"_"+key)
if not os.path.exists("./dataset/dict.pk"):
    special_rels=list(special_rels)
    id2rels=list(rels)
    rels2id=dict([(rel,idx) for idx,rel in enumerate(id2rels)])
    id2labels=["O","B-OBJ","I-OBJ","B-SUB","I-SUB","[category]","[SEP]","[CLS]","[PAD]"]
    label2ids=dict([ (label,idx) for idx,label in enumerate(id2labels)])
    pk.dump([special_rels,id2rels,rels2id,id2labels,label2ids],open("./dataset/dict.pk","wb"))
else:
    print("loading dict...")
    special_rels,id2rels,rels2id,id2labels,label2ids=pk.load(open("./dataset/dict.pk","rb"))
id2kglabels=['O','I']+['B-'+e+"-SUB" for e in id2rels]+['B-'+e+"-OB" for e in id2rels]
kglabels2id=dict([ (label,idx) for idx,label in enumerate(id2kglabels)])
new_id2labels=["O","B-OBJ","E-OBJ","B-SUB","E-SUB","[category]","[SEP]","[CLS]","[PAD]"]
new_label2ids=dict([ (label,idx) for idx,label in enumerate(new_id2labels)])
new_id2kglabels=['O']+['B-'+e+"-SUB" for e in id2rels]+['B-'+e+"-OB" for e in id2rels]+['E-'+e+"-SUB" for e in id2rels]+['E-'+e+"-OB" for e in id2rels]
new_kglabels2id=dict([ (label,idx) for idx,label in enumerate(new_id2kglabels)])
id2reltype=[[] for i in range(len(id2rels))]
for e in schema:
    if len(e['object_type'].keys())==1:
        rel=e["predicate"]
        ids=rels2id[rel]
        id2reltype[ids].append(e)
    else:
        for key in e['object_type'].keys():
            rel=e['predicate']+"_"+key
            ids=rels2id[rel]
            temp_e=copy.deepcopy(e)
            poped_keys=[]
            for k in temp_e['object_type'].keys():
                if k!=key:
                    poped_keys.append(k)
            for k in poped_keys:
                 temp_e['object_type'].pop(k)
            id2reltype[ids].append(temp_e)
id2schema=[e[0] for e in id2reltype]
id2rel_text=[[] for i in range(len(id2rels))]
id2rel_rawtext=[[] for i in range(len(id2rels))]
id2rel_token2text=[[] for i in range(len(id2rels))]
for rel in range(len(id2rels)):
    if id2rels[rel].split("_")[0] not in special_rels:
        cls_text=id2schema[rel]['subject_type']+","+id2schema[rel]['predicate']+","+id2schema[rel]['object_type']['@value']
    else:
        cls_text=id2schema[rel]['subject_type']+","+id2schema[rel]['predicate']+","+id2schema[rel]['object_type'][id2rels[rel].split("_")[1]]
    id2rel_text[rel]=tokenizer.tokenize(cls_text)
    id2rel_rawtext[rel]=cls_text
    id2rel_token2text[rel]=index_token(cls_text)[0]
    assert len(id2rel_token2text[rel])==len(id2rel_text[rel])

loading dict...


In [5]:
#这一部分是生成所有关系及描述词对应的词性，所以需要在run过一遍下面的1.4建立词性tag映射表之后在run这一步。
if not os.path.exists("./middle_data/rel_data_postag.pk"):
    jieba.enable_paddle() 
    jieba.enable_parallel(8)
    rel_cut_words=[]
    rel_cut_tags=[]
    for idx in tqdm(range(len(id2rel_rawtext))):
        words = pseg.lcut(id2rel_rawtext[idx],use_paddle=False) #jieba默认模式
        new_words=[w for w,t in words]
        new_tags=[t for w,t in words]
        rel_cut_words.append([idx,new_words])
        rel_cut_tags.append([idx,new_tags])
    rel_cut_words=[e[1] for e in sorted(rel_cut_words,key=lambda x:x[0])]
    rel_cut_tags=[e[1] for e in sorted(rel_cut_tags,key=lambda x:x[0])]
    rel_data_postag=[]
    for idx in tqdm(range(len(id2rel_rawtext))):
        assert len(id2rel_rawtext[idx].strip())==len("".join(rel_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in rel_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        pos_label=np.zeros(len(id2rel_rawtext[idx])).astype(np.int8)
        for i,(b,e) in enumerate(indexs):
            assert (id2rel_rawtext[idx][b]==rel_cut_words[idx][i][0] or _is_whitespace(id2rel_rawtext[idx].strip()[b])\
                   or _is_whitespace(rel_cut_words[idx][i][0])) \
                    and (id2rel_rawtext[idx].strip()[e]==rel_cut_words[idx][i][-1] \
                         or _is_whitespace(id2rel_rawtext[idx].strip()[e])  or _is_whitespace(rel_cut_words[idx][i][-1]))
            pos_label[b+1:e+1]=pos2id_BIO['I-'+rel_cut_tags[idx][i]]
            pos_label[b]=pos2id_BIO['B-'+rel_cut_tags[idx][i]]
        rel_data_postag.append(pos_label)
    pk.dump(rel_data_postag,open("./middle_data/rel_data_postag.pk","wb"))
else:
    print("loading...")
    rel_data_postag=pk.load(open("./middle_data/rel_data_postag.pk","rb"))
special_major_idx=[2,4,22,32,54]
special_affilate_idx=[[] for i in range(len(id2rels))]
special_affilate_idx[2]=[5]
special_affilate_idx[4]=[0]
special_affilate_idx[22]=[51]
special_affilate_idx[32]=[8]
special_affilate_idx[54]=[6,11,14]
special_pass_idx=[0,5,6,8,11,14,51]
padding_O=[0]*len(new_id2labels)
padding_O[0]=1
padding_category=[0]*len(new_id2labels)
padding_category[5]=1
padding_SEP=[0]*len(new_id2labels)
padding_SEP[6]=1
padding_CLS=[0]*len(new_id2labels)
padding_CLS[7]=1
padding_PAD=[0]*len(new_id2labels)
padding_PAD[8]=1 #for pointer ner

loading...


## 建立词性tag

In [6]:
if not os.path.exists("./middle_data/train_postag.pk"):
    postag_set=set()
    jieba.enable_parallel(8)
    train_cut_words=[]
    train_cut_tags=[]
    for idx in tqdm(range(len(text_data))):
        words = pseg.lcut(text_data[idx]['text'],use_paddle=False) #jieba默认模式
        new_words=[w for w,t in words]
        new_tags=[t for w,t in words]
        postag_set.update(new_tags)
        train_cut_words.append([idx,new_words])
        train_cut_tags.append([idx,new_tags])
    train_cut_words=[e[1] for e in sorted(train_cut_words,key=lambda x:x[0])]
    train_cut_tags=[e[1] for e in sorted(train_cut_tags,key=lambda x:x[0])]
    id2pos=list(postag_set)
    pos2id=dict([(pos,idx) for idx,pos in enumerate(id2pos)])
    pk.dump([id2pos,pos2id,train_cut_words,train_cut_tags],open("./middle_data/train_postag.pk","wb"))
else:
    print("loading...")
    id2pos,pos2id,train_cut_words,train_cut_tags=pk.load(open("./middle_data/train_postag.pk","rb"))
if not os.path.exists("./middle_data/train_postag_BIO.pk"):
    id2pos_BIO=['B-'+e for e in id2pos]+['I-'+e for e in id2pos]
    id2pos_BIO.extend(['[CLS]','[SEP]','[PAD]'])
    pos2id_BIO=dict([(pos,idx) for idx,pos in enumerate(id2pos_BIO)])

    text_data_postag=[]
    for idx in tqdm(range(len(text_data))):
        assert len(text_data[idx]['text'])==len("".join(train_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in train_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        pos_label=np.zeros(len(text_data[idx]['text'])).astype(np.int8)
        for i,(b,e) in enumerate(indexs):
            assert (text_data[idx]['text'][b]==train_cut_words[idx][i][0] or _is_whitespace(text_data[idx]['text'][b])\
                   or _is_whitespace(train_cut_words[idx][i][0])) \
                    and (text_data[idx]['text'][e]==train_cut_words[idx][i][-1] \
                         or _is_whitespace(text_data[idx]['text'][e])  or _is_whitespace(train_cut_words[idx][i][-1]))
            pos_label[b+1:e+1]=pos2id_BIO['I-'+train_cut_tags[idx][i]]
            pos_label[b]=pos2id_BIO['B-'+train_cut_tags[idx][i]]
        text_data_postag.append(pos_label)
    pk.dump([id2pos_BIO,pos2id_BIO,text_data_postag],open("./middle_data/train_postag_BIO.pk","wb"))
else:
    print("loading...")
    id2pos_BIO,pos2id_BIO,text_data_postag=pk.load(open("./middle_data/train_postag_BIO.pk","rb"))

loading...
loading...


In [34]:
if not os.path.exists("./middle_data/test_postag.pk"):
    jieba.enable_paddle() 
    jieba.enable_parallel(8)
    test_cut_words=[]
    test_cut_tags=[]
    for idx in tqdm(range(len(test_text_data))):
        words = pseg.lcut(test_text_data[idx]['text'],use_paddle=False) #jieba默认模式
        new_words=[w for w,t in words]
        new_tags=[t for w,t in words]
        test_cut_words.append([idx,new_words])
        test_cut_tags.append([idx,new_tags])
    test_cut_words=[e[1] for e in sorted(test_cut_words,key=lambda x:x[0])]
    test_cut_tags=[e[1] for e in sorted(test_cut_tags,key=lambda x:x[0])]
    pk.dump([test_cut_words,test_cut_tags],open("./middle_data/test_postag.pk","wb"))
else:
    print("loading...")
    test_cut_words,test_cut_tags=pk.load(open("./middle_data/test_postag.pk","rb"))
if not os.path.exists("./middle_data/test_postag_BIO.pk"):

    test_text_data_postag=[]
    for idx in tqdm(range(len(test_text_data))):
        assert len(test_text_data[idx]['text'])==len("".join(test_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in test_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        pos_label=np.zeros(len(test_text_data[idx]['text'])).astype(np.int8)
        for i,(b,e) in enumerate(indexs):
            assert (test_text_data[idx]['text'][b]==test_cut_words[idx][i][0] or _is_whitespace(test_text_data[idx]['text'].strip()[b])\
                   or _is_whitespace(test_cut_words[idx][i][0])) \
                    and (test_text_data[idx]['text'][e]==test_cut_words[idx][i][-1] \
                         or _is_whitespace(test_text_data[idx]['text'][e])  or _is_whitespace(test_cut_words[idx][i][-1]))
            pos_label[b+1:e+1]=pos2id_BIO['I-'+test_cut_tags[idx][i]]
            pos_label[b]=pos2id_BIO['B-'+test_cut_tags[idx][i]]
        test_text_data_postag.append(pos_label)
    pk.dump(test_text_data_postag,open("./middle_data/test_postag_BIO.pk","wb"))
else:
    print("loading...")
    test_text_data_postag=pk.load(open("./middle_data/test_postag_BIO.pk","rb"))

loading...
loading...


In [16]:
if not os.path.exists("./middle_data/test2_postag.pk"):
    jieba.enable_paddle() 
    jieba.enable_parallel(8)
    test2_cut_words=[]
    test2_cut_tags=[]
    for idx in tqdm(range(len(test2_text_data))):
        words = pseg.lcut(test2_text_data[idx]['text'],use_paddle=False) #jieba默认模式
        new_words=[w for w,t in words]
        new_tags=[t for w,t in words]
        test2_cut_words.append([idx,new_words])
        test2_cut_tags.append([idx,new_tags])
    test2_cut_words=[e[1] for e in sorted(test2_cut_words,key=lambda x:x[0])]
    test2_cut_tags=[e[1] for e in sorted(test2_cut_tags,key=lambda x:x[0])]
    pk.dump([test2_cut_words,test2_cut_tags],open("./middle_data/test2_postag.pk","wb"))
else:
    print("loading...")
    test2_cut_words,test2_cut_tags=pk.load(open("./middle_data/test2_postag.pk","rb"))
if not os.path.exists("./middle_data/test2_postag_BIO.pk"):

    test2_text_data_postag=[]
    for idx in tqdm(range(len(test2_text_data))):
        assert len(test2_text_data[idx]['text'])==len("".join(test2_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in test2_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        pos_label=np.zeros(len(test2_text_data[idx]['text'])).astype(np.int8)
        for i,(b,e) in enumerate(indexs):
            assert (test2_text_data[idx]['text'][b]==test2_cut_words[idx][i][0] or _is_whitespace(test2_text_data[idx]['text'].strip()[b])\
                   or _is_whitespace(test2_cut_words[idx][i][0])) \
                    and (test2_text_data[idx]['text'][e]==test2_cut_words[idx][i][-1] \
                         or _is_whitespace(test2_text_data[idx]['text'][e])  or _is_whitespace(test2_cut_words[idx][i][-1]))
            pos_label[b+1:e+1]=pos2id_BIO['I-'+test2_cut_tags[idx][i]]
            pos_label[b]=pos2id_BIO['B-'+test2_cut_tags[idx][i]]
        test2_text_data_postag.append(pos_label)
    pk.dump(test2_text_data_postag,open("./middle_data/test2_postag_BIO.pk","wb"))
else:
    print("loading...")
    test2_text_data_postag=pk.load(open("./middle_data/test2_postag_BIO.pk","rb"))

loading...
loading...


## 读取词向量

In [6]:
if not os.path.exists("./Tencent_ChineseEmbedding/extra_embedding.npy"):
    word2vec_dims=wv_from_text.vectors.shape[-1]
    pad_vec=np.zeros([1,word2vec_dims],dtype=np.float32)
    other_token_vec=np.random.normal(scale=0.02,size=[3,word2vec_dims])
    extra_token_vec=np.concatenate([pad_vec,other_token_vec])
    np.save("./Tencent_ChineseEmbedding/extra_embedding.npy",extra_token_vec)
else:
    print("loading extra tokens vector file...")
    extra_token_vec=np.load("./Tencent_ChineseEmbedding/extra_embedding.npy")
word2vec=np.concatenate([extra_token_vec,wv_from_text.vectors])

05/20/2020 17:06:34 - INFO - gensim.utils -   loading Word2VecKeyedVectors object from ./Tencent_ChineseEmbedding/ChineseEmbedding.bin


loading...


05/20/2020 17:07:30 - INFO - gensim.utils -   loading vectors from ./Tencent_ChineseEmbedding/ChineseEmbedding.bin.vectors.npy with mmap=r
05/20/2020 17:07:30 - INFO - gensim.utils -   setting ignored attribute vectors_norm to None
05/20/2020 17:07:30 - INFO - gensim.utils -   loaded ./Tencent_ChineseEmbedding/ChineseEmbedding.bin


In [7]:
if not os.path.exists('./Tencent_ChineseEmbedding/usr_dict.txt'):
    keys=[]
    for key in tqdm(wv_from_text.vocab.keys()):
        keys.append(key)
    with open("./Tencent_ChineseEmbedding/usr_dict.txt","w") as w:
        for key in tqdm(keys):
            w.write(key+"\n")
else:
    print("loading...")
    keys=[]
    with open("./Tencent_ChineseEmbedding/usr_dict.txt","r") as r:
        for key in tqdm(r.readlines()):
            keys.append(key[:-1])
import jieba
jieba.load_userdict("./Tencent_ChineseEmbedding/usr_dict.txt")
if not os.path.exists("./Tencent_ChineseEmbedding/word_id.pk"):
    word2id={}
    id2word=[[] for i in range(word2vec.shape[0])]
    for i,key in tqdm(enumerate(keys)):
        assert (word2vec[4+i]==wv_from_text[key]).all()
        word2id[key]=4+i
        id2word[4+i]=key
    id2word[0]=tokenizer.pad_token
    word2id[tokenizer.pad_token]=0
    id2word[1]=tokenizer.unk_token
    word2id[tokenizer.unk_token]=1
    id2word[2]=tokenizer.cls_token
    word2id[tokenizer.cls_token]=2
    id2word[3]=tokenizer.sep_token
    word2id[tokenizer.sep_token]=3
    pk.dump([word2id,id2word],open("./Tencent_ChineseEmbedding/word_id.pk","wb"),protocol = 4)
else:
    print("loading...")
    word2id,id2word=pk.load(open("./Tencent_ChineseEmbedding/word_id.pk","rb"))

loading extra tokens vector file...
loading...


In [ ]:
if not os.path.exists("./middle_data/train_wordvec.pk"):
    jieba.enable_parallel(8)
    train_cut_words=[]
    train_cut_vecs=[]
    for idx in tqdm(range(len(text_data))):
        word_list=jieba.lcut(text_data[idx]['text'],HMM=False,use_paddle=False)
        wvid_list=[ word2id.get(e,1)  for e in word_list ]
        train_cut_words.append([idx,word_list])
        train_cut_vecs.append([idx,wvid_list])
    train_cut_words=[e[1] for e in sorted(train_cut_words,key=lambda x:x[0])]
    train_cut_vecs=[e[1] for e in sorted(train_cut_vecs,key=lambda x:x[0])]
#     train_cut_wvs=[]
#     for e in tqdm(train_cut_vecs):
#         train_cut_wvs.append(np.array([word2vec[t] for t in e]))
    pk.dump([train_cut_words,train_cut_vecs],open("./middle_data/train_wordvec.pk","wb"))
else:
    print("loading...")
    train_cut_words,train_cut_vecs=pk.load(open("./middle_data/train_wordvec.pk","rb"))
if not os.path.exists("./middle_data/train_tokenvec.pk"):
    text_data_tokenvec=[]
    for idx in tqdm(range(len(text_data))):
        assert len(text_data[idx]['text'])==len("".join(train_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in train_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        vec_label=np.zeros((len(text_data[idx]['text']))).astype(np.int)
        for i,(b,e) in enumerate(indexs):
            assert (text_data[idx]['text'][b]==train_cut_words[idx][i][0] or _is_whitespace(text_data[idx]['text'][b])\
                   or _is_whitespace(train_cut_words[idx][i][0])) \
                    and (text_data[idx]['text'][e]==train_cut_words[idx][i][-1] \
                         or _is_whitespace(text_data[idx]['text'][e])  or _is_whitespace(train_cut_words[idx][i][-1]))
            vec_label[b:e+1]=train_cut_vecs[idx][i]
        text_data_tokenvec.append(vec_label)
    pk.dump(text_data_tokenvec,open("./middle_data/train_tokenvec.pk","wb"))
else:
    print("loading...")
    text_data_tokenvec=pk.load(open("./middle_data/train_tokenvec.pk","rb"))


In [ ]:
if not os.path.exists("./middle_data/test_wordvec.pk"):
    jieba.enable_parallel(8)
    test_cut_words=[]
    test_cut_vecs=[]
    for idx in tqdm(range(len(test_text_data))):
        word_list=jieba.lcut(test_text_data[idx]['text'],HMM=False,use_paddle=False)
        wvid_list=[ word2id.get(e,1)  for e in word_list ]
        test_cut_words.append([idx,word_list])
        test_cut_vecs.append([idx,wvid_list])
    test_cut_words=[e[1] for e in sorted(test_cut_words,key=lambda x:x[0])]
    test_cut_vecs=[e[1] for e in sorted(test_cut_vecs,key=lambda x:x[0])]
    pk.dump([test_cut_words,test_cut_vecs],open("./middle_data/test_wordvec.pk","wb"))
else:
    print("loading...")
    test_cut_words,test_cut_vecs=pk.load(open("./middle_data/test_wordvec.pk","rb"))
if not os.path.exists("./middle_data/test_tokenvec.pk"):
    test_text_data_tokenvec=[]
    for idx in tqdm(range(len(test_text_data))):
        assert len(test_text_data[idx]['text'])==len("".join(test_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in test_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        vec_label=np.zeros((len(test_text_data[idx]['text']),)).astype(np.int)
        for i,(b,e) in enumerate(indexs):
            assert (test_text_data[idx]['text'][b]==test_cut_words[idx][i][0] or _is_whitespace(test_text_data[idx]['text'][b])\
                   or _is_whitespace(test_cut_words[idx][i][0])) \
                    and (test_text_data[idx]['text'][e]==test_cut_words[idx][i][-1] \
                         or _is_whitespace(test_text_data[idx]['text'][e])  or _is_whitespace(test_cut_words[idx][i][-1]))
            vec_label[b:e+1]=test_cut_vecs[idx][i]
        test_text_data_tokenvec.append(vec_label)
    pk.dump(test_text_data_tokenvec,open("./middle_data/test_tokenvec.pk","wb"))
else:
    print("loading...")
    test_text_data_tokenvec=pk.load(open("./middle_data/test_tokenvec.pk","rb"))


In [ ]:
if not os.path.exists("./middle_data/test2_wordvec.pk"):
    jieba.enable_parallel(8)
    test2_cut_words=[]
    test2_cut_vecs=[]
    for idx in tqdm(range(len(test2_text_data))):
        word_list=jieba.lcut(test2_text_data[idx]['text'],HMM=False,use_paddle=False)
        wvid_list=[ word2id.get(e,1)  for e in word_list ]
        test2_cut_words.append([idx,word_list])
        test2_cut_vecs.append([idx,wvid_list])
    test2_cut_words=[e[1] for e in sorted(test2_cut_words,key=lambda x:x[0])]
    test2_cut_vecs=[e[1] for e in sorted(test2_cut_vecs,key=lambda x:x[0])]
    pk.dump([test2_cut_words,test2_cut_vecs],open("./middle_data/test2_wordvec.pk","wb"))
else:
    print("loading...")
    test2_cut_words,test2_cut_vecs=pk.load(open("./middle_data/test2_wordvec.pk","rb"))
if not os.path.exists("./middle_data/test2_tokenvec.pk"):
    test2_text_data_tokenvec=[]
    for idx in tqdm(range(len(test2_text_data))):
        assert len(test2_text_data[idx]['text'])==len("".join(test2_cut_words[idx]))
        indexs=[]
        cur_length=0
        for e in test2_cut_words[idx]:
            indexs.append([cur_length,cur_length+len(e)-1])
            cur_length+=len(e)
        vec_label=np.zeros((len(test2_text_data[idx]['text']),)).astype(np.int)
        for i,(b,e) in enumerate(indexs):
            assert (test2_text_data[idx]['text'][b]==test2_cut_words[idx][i][0] or _is_whitespace(test2_text_data[idx]['text'][b])\
                   or _is_whitespace(test2_cut_words[idx][i][0])) \
                    and (test2_text_data[idx]['text'][e]==test2_cut_words[idx][i][-1] \
                         or _is_whitespace(test2_text_data[idx]['text'][e])  or _is_whitespace(test2_cut_words[idx][i][-1]))
            vec_label[b:e+1]=test2_cut_vecs[idx][i]
        test2_text_data_tokenvec.append(vec_label)
    pk.dump(test2_text_data_tokenvec,open("./middle_data/test2_tokenvec.pk","wb"))
else:
    print("loading...")
    test2_text_data_tokenvec=pk.load(open("./middle_data/test2_tokenvec.pk","rb"))


## TFIDF-SVD

In [9]:
if not os.path.exists("./middle_data/texts_tfidf_svd.pk"):
    texts_tfidf_svd,tfidf_vectorizer,svd_vectorizer=build_tfidf_svd_matrix(text_data,16)
    joblib.dump(tfidf_vectorizer,"./middle_data/tfidf_vectorizer.sk.model")
    joblib.dump(svd_vectorizer,"./middle_data/svd_vectorizer.sk.model")
    pk.dump(texts_tfidf_svd,open("./middle_data/texts_tfidf_svd.pk","wb"))
else:
    print("loading...")
    texts_tfidf_svd=pk.load(open("./middle_data/texts_tfidf_svd.pk","rb"))

loading...


In [10]:
if not os.path.exists("./middle_data/test_texts_tfidf_svd.pk"):
    tfidf_vectorizer=joblib.load("./middle_data/tfidf_vectorizer.sk.model")
    svd_vectorizer=joblib.load("./middle_data/svd_vectorizer.sk.model")
    test_texts_tfidf_svd,test_tfidf_vectorizer,test_svd_vectorizer=build_tfidf_svd_matrix(test_text_data,16,tfidf_vectorizer,svd_vectorizer)

    pk.dump(test_texts_tfidf_svd,open("./middle_data/test_texts_tfidf_svd.pk","wb"))
else:
    print("loading...")
    test_texts_tfidf_svd=pk.load(open("./middle_data/test_texts_tfidf_svd.pk","rb"))

loading...


In [11]:
if not os.path.exists("./middle_data/test2_texts_tfidf_svd.pk"):
    tfidf_vectorizer=joblib.load("./middle_data/tfidf_vectorizer.sk.model")
    svd_vectorizer=joblib.load("./middle_data/svd_vectorizer.sk.model")
    test2_texts_tfidf_svd,test2_tfidf_vectorizer,test2_svd_vectorizer=build_tfidf_svd_matrix(test2_text_data,16,tfidf_vectorizer,svd_vectorizer)

    pk.dump(test2_texts_tfidf_svd,open("./middle_data/test2_texts_tfidf_svd.pk","wb"))
else:
    print("loading...")
    test2_texts_tfidf_svd=pk.load(open("./middle_data/test2_texts_tfidf_svd.pk","rb"))

loading...


## 关系分类数据处理

In [20]:
if not os.path.exists("./middle_data/train_plan_labels.pk"):
    train_plan_labels=[]
    for e in tqdm(text_data):
        train_plan_labels.append(_convert_example_to_cls_plan_labels(e,tokenizer))
    for idx in tqdm(range(len(train_plan_labels))):
        for jdx in range(len(train_plan_labels[idx])):
            if sum(train_plan_labels[idx][jdx])==0:
                train_plan_labels[idx][jdx][0]=1
    pk.dump(train_plan_labels,open("./middle_data/train_plan_labels.pk","wb"))
else:
    print("loading...")
    train_plan_labels=pk.load(open("./middle_data/train_plan_labels.pk","rb"))

loading...


In [6]:
if not os.path.exists("./middle_data/doc_cls_info.pk"):
    cls_doc_tokens=[]
    cls_doc_rels=[]
    cls_token2doc=[]
    for e in tqdm(text_data):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,rel_labels=_convert_example_to_cls_record(e,tokenizer)
        cls_doc_tokens.append(tokens)
        cls_doc_rels.append(rel_labels)
        cls_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
    pk.dump([cls_doc_rels,cls_doc_tokens,cls_token2doc],open("./middle_data/doc_cls_info.pk","wb"))
else:
    print("loading...")
    cls_doc_rels,cls_doc_tokens,cls_token2doc=pk.load(open("./middle_data/doc_cls_info.pk","rb"))

loading...


In [7]:
if not os.path.exists("./middle_data/cls_examples.pk"):
    cls_examples,train_wordvecs,train_tokenvecs=[],[],[]
    #train_cut_vecs[idx],text_data_tokenvec[idx]
    for idx,(tokens,rel,token2doc,postag) in tqdm(enumerate(zip(cls_doc_tokens,cls_doc_rels,cls_token2doc,text_data_postag))):
                example=create_cls_example(tokens,rel,token2doc,postag,texts_tfidf_svd[idx],\
                                                       None,None,train_plan_labels[idx],tokenizer)
                cls_examples.append(example)
    pk.dump(cls_examples,open("./middle_data/cls_examples.pk","wb"))
else:
    print("loading example..")
    cls_examples=pk.load(open("./middle_data/cls_examples.pk","rb"))

loading example..


In [79]:
cls_examples=np.array(cls_examples)
spliter=KFold(n_splits=3,shuffle=True,random_state=20)
train_indexs=[]
val_indexs=[]
for idx,(train_index,val_index) in enumerate(spliter.split(cls_examples)):
    train_indexs.append(train_index)
    val_indexs.append(val_index)
cls_logits=[]
for idx in range(3):
    cls_logits.append(np.load("./cross_results/cls_"+str(idx)+".npy"))
pred_cls_logits=np.zeros((len(cls_examples),55))
for idx in range(3):
    pred_cls_logits[val_indexs[idx]]=cls_logits[idx]

In [49]:
cls_train_dataset=RelDataset(cls_examples[:10000])
cls_val_dataset=RelDataset(cls_examples[10000:20000])

In [9]:
if not os.path.exists("./middle_data/test_plan_labels.pk"):
    test_plan_labels=[]
    for e in tqdm(test_text_data):
        test_plan_labels.append(_convert_example_to_cls_plan_labels(e,tokenizer))
    for idx in tqdm(range(len(test_plan_labels))):
        for jdx in range(len(test_plan_labels[idx])):
            if sum(test_plan_labels[idx][jdx])==0:
                test_plan_labels[idx][jdx][0]=1
    pk.dump(test_plan_labels,open("./middle_data/test_plan_labels.pk","wb"))
else:
    print("loading...")
    test_plan_labels=pk.load(open("./middle_data/test_plan_labels.pk","rb"))

In [10]:
if not os.path.exists("./middle_data/test_doc_cls_info.pk"):
    test_cls_doc_tokens=[]
    test_cls_doc_rels=[]
    test_cls_token2doc=[]
    for e in tqdm(test_text_data):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,rel_labels=_convert_example_to_cls_record(e,tokenizer)
        test_cls_doc_tokens.append(tokens)
        test_cls_doc_rels.append(rel_labels)
        test_cls_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
    pk.dump([test_cls_doc_rels,test_cls_doc_tokens,test_cls_token2doc],open("./middle_data/test_doc_cls_info.pk","wb"))
else:
    print("loading...")
    test_cls_doc_rels,test_cls_doc_tokens,test_cls_token2doc=pk.load(open("./middle_data/test_doc_cls_info.pk","rb"))

In [12]:
if not os.path.exists("./middle_data/test_cls_examples.pk"):
    test_cls_examples=[]
    #test_cut_vecs[idx],test_text_data_tokenvec[idx]
    for idx,(tokens,rel,token2doc,postag) in tqdm(enumerate(zip(test_cls_doc_tokens,test_cls_doc_rels,test_cls_token2doc,test_text_data_postag))):   
            example=create_cls_example(tokens,rel,token2doc,postag,test_texts_tfidf_svd[idx],\
                                       None,None,test_plan_labels[idx],tokenizer)
            test_cls_examples.append(example)   
    pk.dump(test_cls_examples,open("./middle_data/test_cls_examples.pk","wb"))
else:
    print("loading example..")
    test_cls_examples=pk.load(open("./middle_data/test_cls_examples.pk","rb"))

loading example..


In [13]:
cls_test_dataset=RelDataset(test_cls_examples[:])

In [6]:
#model ensemble 部分
preds=[]

for idx in range(3):
    preds.append(np.load("./cross_results/test_cls_"+str(idx)+".npy"))
preds.append(np.load("./cross_results/test_cls_full.npy"))
preds=np.stack(preds,axis=2)
preds=preds.mean(axis=-1)
# preds=(preds>0.5).astype(np.int)
pred_cls_logits=preds
# cls_logits=[]
# for idx in range(3):
#     cls_logits.append(np.load("./cross_results/cls_"+str(idx)+".npy"))
# pred_cls_logits=np.zeros((len(cls_examples),55))
# for idx in range(3):
#     pred_cls_logits[val_indexs[idx]]=cls_logits[idx]

In [ ]:
if not os.path.exists("./middle_data/test2_plan_labels.pk"):
    test2_plan_labels=[]
    for e in tqdm(test2_text_data):
        test2_plan_labels.append(_convert_example_to_cls_plan_labels(e,tokenizer))
    for idx in tqdm(range(len(test2_plan_labels))):
        for jdx in range(len(test2_plan_labels[idx])):
            if sum(test2_plan_labels[idx][jdx])==0:
                test2_plan_labels[idx][jdx][0]=1
    pk.dump(test2_plan_labels,open("./middle_data/test2_plan_labels.pk","wb"))
else:
    print("loading...")
    test2_plan_labels=pk.load(open("./middle_data/test2_plan_labels.pk","rb"))

In [ ]:
if not os.path.exists("./middle_data/test2_doc_cls_info.pk"):
    test2_cls_doc_tokens=[]
    test2_cls_doc_rels=[]
    test2_cls_token2doc=[]
    for e in tqdm(test2_text_data):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,rel_labels=_convert_example_to_cls_record(e,tokenizer)
        test2_cls_doc_tokens.append(tokens)
        test2_cls_doc_rels.append(rel_labels)
        test2_cls_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
    pk.dump([test2_cls_doc_rels,test2_cls_doc_tokens,test2_cls_token2doc],open("./middle_data/test2_doc_cls_info.pk","wb"))
else:
    print("loading...")
    test2_cls_doc_rels,test2_cls_doc_tokens,test2_cls_token2doc=pk.load(open("./middle_data/test2_doc_cls_info.pk","rb"))

In [9]:
if not os.path.exists("./middle_data/test2_cls_examples.pk"):
    test2_cls_examples=[]
    for idx,(tokens,rel,token2doc,postag) in tqdm(enumerate(zip(test2_cls_doc_tokens,test2_cls_doc_rels,test2_cls_token2doc,test2_text_data_postag))):   
            example=create_cls_example(tokens,rel,token2doc,postag,test2_texts_tfidf_svd[idx],\
                                       test2_cut_vecs[idx],test2_text_data_tokenvec[idx],test2_plan_labels[idx],tokenizer)
            test2_cls_examples.append(example)   
    pk.dump(test2_cls_examples,open("./middle_data/test2_cls_examples.pk","wb"))
else:
    print("loading example..")
    test2_cls_examples=pk.load(open("./middle_data/test2_cls_examples.pk","rb"))

In [10]:
preds=[]

for idx in range(3):
    preds.append(np.load("./cross_results/test2_cls_"+str(idx)+".npy"))
preds.append(np.load("./cross_results/test2_cls_full.npy"))
preds=np.stack(preds,axis=2)
preds=preds.mean(axis=-1)
# preds=(preds>0.5).astype(np.int)
pred_cls_logits=preds

In [25]:
cls_test2_dataset=RelDataset(test2_cls_examples[:])

## 关系实体识别数据处理

In [7]:
if not os.path.exists("./middle_data/doc_token_info_ner.pk"):
    ner_doc_tokens=[]
    ner_doc_token_labels=[]
    ner_doc_rels=[]
    ner_token2doc=[]
    ner_doc2doc=[]
    for idx,example in tqdm(enumerate(text_data)):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels=_convert_example_to_record(example,tokenizer)
        rel_list=[e['predicate'] for e in example['spo_list']]
#         if len(set(rel_list))!=len(rel_list):
#                  print(idx)
        for i in range(len(rel_labels)):
            ner_doc_tokens.append(tokens)
            ner_doc_rels.append(rel_labels[i])
            ner_doc_token_labels.append(labels_list[rel_labels[i]])
            ner_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
            ner_doc2doc.append(idx)
    pk.dump([ner_doc2doc,ner_doc_rels,ner_doc_tokens,ner_doc_token_labels,ner_token2doc],open("./middle_data/doc_token_info_ner.pk","wb"))
else:
    print("loading doc..")
    ner_doc2doc,ner_doc_rels,ner_doc_tokens,ner_doc_token_labels,ner_token2doc=pk.load(open("./middle_data/doc_token_info_ner.pk","rb"))

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [12]:

if not os.path.exists("./middle_data/ner_examples_ner.pk"):
    ner_examples=[]
    example2doc=[]
    doc2example=[[] for i in range(len(ner_doc_tokens))]
    for idx,(tokens,rel,labels,token2doc) in tqdm(enumerate(zip(ner_doc_tokens,ner_doc_rels,ner_doc_token_labels,ner_token2doc))):
        tfidf_svd=texts_tfidf_svd[ner_doc2doc[idx]]
        postag=text_data_postag[ner_doc2doc[idx]]
        cur_idx=0
        rel_text=id2rel_text[rel]
        rel_postag=rel_data_postag[rel]
        rel_token2doc=id2rel_token2text[rel]
        while len(tokens)-cur_idx> (max_len-3-len(rel_text)):
            temp_tokens=tokens[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_labels=labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
            #训练时把这些label里面妹有 3 4 1 2 的样本去掉
            if (1 in  temp_labels) or (2 in  temp_labels) or (3 in  temp_labels) or (4 in  temp_labels):
                example2doc.append(idx)
                ner_examples.append(create_example(temp_tokens,rel,temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))
                doc2example[idx].append(len(ner_examples)-1)
            cur_idx+=max_len-3-len(rel_text)
        if cur_idx<len(tokens):
            temp_tokens=tokens[cur_idx:]
            temp_labels=labels[cur_idx:]
            temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
            if (1 in  temp_labels) or (2 in  temp_labels) or (3 in  temp_labels) or (4 in  temp_labels):
                example2doc.append(idx)
                ner_examples.append(create_example(temp_tokens,rel,temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))    
                doc2example[idx].append(len(ner_examples)-1)

    pk.dump([doc2example,example2doc,ner_examples],open("./middle_data/ner_examples_ner.pk","wb"))
else:
    print("loading example..")
    doc2example,example2doc,ner_examples=pk.load(open("./middle_data/ner_examples_ner.pk","rb"))

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [10]:
#交叉ensemble部分
ner_examples=np.array(ner_examples)
spliter=KFold(n_splits=3,shuffle=True,random_state=20)
train_indexs=[]
val_indexs=[]
for idx,(train_index,val_index) in enumerate(spliter.split(ner_examples)):
    train_indexs.append(train_index)
    val_indexs.append(val_index)
preds=[]
logits=[]
rel_logits=[]
for idx in range(3):
    preds.append(np.load("./cross_results/pred_"+str(idx)+".npy"))
    logits.append(np.load("./cross_results/logit_"+str(idx)+".npy"))
    rel_logits.append(np.load("./cross_results/rel_"+str(idx)+".npy"))
pred_token_pred=np.zeros((len(ner_examples),256))
pred_token_logits=np.zeros((len(ner_examples),256,9))
pred_rel_logits=np.zeros((len(ner_examples),55))
for idx in range(3):
    val_index=val_indexs[idx]
    pred_token_pred[val_index]=preds[idx]
    pred_token_logits[val_index]=logits[idx]
    pred_rel_logits[val_index]=rel_logits[idx]

In [13]:
# for i in range(len(example2doc)):
#     if ner_doc2doc[example2doc[i]]==180000:
#         print(i)
ner_train_dataset=NerDataset(ner_examples[:10000])
ner_val_dataset=NerDataset(ner_examples[268432:])

## 使用关系分类得到的结果进行序列标注(验证整体pipline模型效果及预测 测试集的时候使用)

### CRF

#### val

In [38]:
preds=(preds>0.5).astype(np.int)

In [39]:
preds.shape,out_label_ids.shape

NameError: name 'out_label_ids' is not defined

In [ ]:
val_index

In [ ]:
if not os.path.exists("./middle_data/pred_token_info_ner_0.pk"):
    pred_doc_tokens=[]
    pred_doc_token_labels=[]
    pred_doc_rels=[]
    pred_token2doc=[]
    pred_doc2doc=[]
    for idx,ind in tqdm(enumerate(val_index)):
        example=text_data[ind]
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels=_convert_example_to_record(example,tokenizer)
        rels=(preds[idx].astype(np.int8)*np.arange(preds.shape[1]))
        rels=rels[rels>0]
        rel_labels=rels
        if preds[idx][0]==1:
            rel_labels=np.array([0]+list(rel_labels))
        for i in range(len(rel_labels)):
            pred_doc_tokens.append(tokens)
            pred_doc_rels.append(rel_labels[i])
            pred_doc_token_labels.append(labels_list[rel_labels[i]])
            pred_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
            pred_doc2doc.append(ind)
    pk.dump([pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc],open("./middle_data/pred_token_info_ner_0.pk","wb"))
else:
    print("loading doc..")
    pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc=pk.load(open("./middle_data/pred_token_info_ner_0.pk","rb"))


In [ ]:
if not os.path.exists("./middle_data/pred_examples_ner_0.pk"):
    pred_examples=[]
    pred_example2doc=[]
    pred_doc2example=[[] for i in range(len(pred_doc_tokens))]
    for idx,(tokens,rel,labels,token2doc) in tqdm(enumerate(zip(pred_doc_tokens,pred_doc_rels,pred_doc_token_labels,pred_token2doc))):
                tfidf_svd=texts_tfidf_svd[pred_doc2doc[idx]]
                postag=text_data_postag[pred_doc2doc[idx]]
                cur_idx=0
                rel_text=id2rel_text[rel]
                rel_postag=rel_data_postag[rel]
                rel_token2doc=id2rel_token2text[rel]
                while len(tokens)-cur_idx> (max_len-3-len(rel_text)):
                    temp_tokens=tokens[cur_idx:cur_idx+max_len-3-len(rel_text)]
                    temp_labels=labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
                    temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
                    pred_example2doc.append(idx)
                    pred_examples.append(create_example(temp_tokens,int(rel),temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))
                    pred_doc2example[idx].append(len(pred_examples)-1)
                    cur_idx+=max_len-3-len(rel_text)
                if cur_idx<len(tokens):
                    temp_tokens=tokens[cur_idx:]
                    temp_labels=labels[cur_idx:]
                    temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
                    pred_example2doc.append(idx)
                    pred_examples.append(create_example(temp_tokens,int(rel),temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))    
                    pred_doc2example[idx].append(len(pred_examples)-1)
    pk.dump([pred_doc2example,pred_example2doc,pred_examples],open("./middle_data/pred_examples_ner_0.pk","wb"))
else:
    print("loading example..")
    pred_doc2example,pred_example2doc,pred_examples=pk.load(open("./middle_data/pred_examples_ner_0.pk","rb"))

In [ ]:
ner_pred_dataset=NerDataset(pred_examples[:])
len(ner_pred_dataset) #21678

#### test

In [28]:
preds=np.random.rand(10468,55)

In [30]:
preds.shape,len(test_text_data)

((10468, 55), 10468)

In [31]:
preds=(preds>0.5).astype(np.int)

In [32]:
if not os.path.exists("./middle_data/test_pred_token_info_ner.pk"):
    pred_doc_tokens=[]
    pred_doc_token_labels=[]
    pred_doc_rels=[]
    pred_token2doc=[]
    pred_doc2doc=[]
    for idx,example in tqdm(enumerate(test_text_data)):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels=_convert_example_to_record(example,tokenizer)
        rels=(preds[idx].astype(np.int8)*np.arange(preds.shape[1]))
        rels=rels[rels>0]
        rel_labels=rels
        if preds[idx][0]==1:
            rel_labels=np.array([0]+list(rel_labels))
        for i in range(len(rel_labels)):
            pred_doc_tokens.append(tokens)
            pred_doc_rels.append(rel_labels[i])
            pred_doc_token_labels.append(labels_list[rel_labels[i]])
            pred_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
            pred_doc2doc.append(idx)
    pk.dump([pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc],open("./middle_data/test_pred_token_info_ner.pk","wb"))
else:
    print("loading doc..")
    pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc=pk.load(open("./middle_data/test_pred_token_info_ner.pk","rb"))


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [35]:
if not os.path.exists("./middle_data/test_pred_examples_ner.pk"):
    pred_examples=[]
    pred_example2doc=[]
    pred_doc2example=[[] for i in range(len(pred_doc_tokens))]
    for idx,(tokens,rel,labels,token2doc) in tqdm(enumerate(zip(pred_doc_tokens,pred_doc_rels,pred_doc_token_labels,pred_token2doc))):
                tfidf_svd=test_texts_tfidf_svd[pred_doc2doc[idx]]
                postag=test_text_data_postag[pred_doc2doc[idx]]
                cur_idx=0
                rel_text=id2rel_text[rel]
                rel_postag=rel_data_postag[rel]
                rel_token2doc=id2rel_token2text[rel]
                while len(tokens)-cur_idx> (max_len-3-len(rel_text)):
                    temp_tokens=tokens[cur_idx:cur_idx+max_len-3-len(rel_text)]
                    temp_labels=labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
                    temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
                    pred_example2doc.append(idx)
                    pred_examples.append(create_example(temp_tokens,int(rel),temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))
                    pred_doc2example[idx].append(len(pred_examples)-1)
                    cur_idx+=max_len-3-len(rel_text)
                if cur_idx<len(tokens):
                    temp_tokens=tokens[cur_idx:]
                    temp_labels=labels[cur_idx:]
                    temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
                    pred_example2doc.append(idx)
                    pred_examples.append(create_example(temp_tokens,int(rel),temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))    
                    pred_doc2example[idx].append(len(pred_examples)-1)
    pk.dump([pred_doc2example,pred_example2doc,pred_examples],open("./middle_data/test_pred_examples_ner.pk","wb"))
else:
    print("loading example..")
    pred_doc2example,pred_example2doc,pred_examples=pk.load(open("./middle_data/test_pred_examples_ner.pk","rb"))

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [36]:
ner_pred_dataset=NerDataset(pred_examples)
len(ner_pred_dataset)

288440

In [14]:
#交叉 ensemble 部分
pred_examples=np.array(pred_examples)
pred_token_pred=np.zeros((len(pred_examples),256))
pred_token_logits=np.zeros((len(pred_examples),256,9))
pred_rel_logits=np.zeros((len(pred_examples),55))
pred_token_pred=np.load("./cross_results/test_pred_full.npy")
pred_token_logits=np.load("./cross_results/test_logit_full.npy")
pred_rel_logits=np.load("./cross_results/test_rel_full.npy")
pred_rel_logits.shape,len(pred_examples)

((18646, 55), 0)

In [12]:
if not os.path.exists("./middle_data/test2_pred_token_info_ner.pk"):
    pred_doc_tokens=[]
    pred_doc_token_labels=[]
    pred_doc_rels=[]
    pred_token2doc=[]
    pred_doc2doc=[]
    for idx,example in tqdm(enumerate(test2_text_data)):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels=_convert_example_to_record(example,tokenizer)
        rels=(preds[idx].astype(np.int8)*np.arange(preds.shape[1]))
        rels=rels[rels>0]
        rel_labels=rels
        if preds[idx][0]==1:
            rel_labels=np.array([0]+list(rel_labels))
        for i in range(len(rel_labels)):
            pred_doc_tokens.append(tokens)
            pred_doc_rels.append(rel_labels[i])
            pred_doc_token_labels.append(labels_list[rel_labels[i]])
            pred_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
            pred_doc2doc.append(idx)
    pk.dump([pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc],open("./middle_data/test2_pred_token_info_ner.pk","wb"))
else:
    print("loading doc..")
    pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc=pk.load(open("./middle_data/test2_pred_token_info_ner.pk","rb"))


loading doc..


In [13]:
if not os.path.exists("./middle_data/test2_pred_examples_ner.pk"):
    pred_examples=[]
    pred_example2doc=[]
    pred_doc2example=[[] for i in range(len(pred_doc_tokens))]
    for idx,(tokens,rel,labels,token2doc) in tqdm(enumerate(zip(pred_doc_tokens,pred_doc_rels,pred_doc_token_labels,pred_token2doc))):
                tfidf_svd=test2_texts_tfidf_svd[pred_doc2doc[idx]]
                postag=test2_text_data_postag[pred_doc2doc[idx]]
                cur_idx=0
                rel_text=id2rel_text[rel]
                rel_postag=rel_data_postag[rel]
                rel_token2doc=id2rel_token2text[rel]
                while len(tokens)-cur_idx> (max_len-3-len(rel_text)):
                    temp_tokens=tokens[cur_idx:cur_idx+max_len-3-len(rel_text)]
                    temp_labels=labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
                    temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
                    pred_example2doc.append(idx)
                    pred_examples.append(create_example(temp_tokens,int(rel),temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))
                    pred_doc2example[idx].append(len(pred_examples)-1)
                    cur_idx+=max_len-3-len(rel_text)
                if cur_idx<len(tokens):
                    temp_tokens=tokens[cur_idx:]
                    temp_labels=labels[cur_idx:]
                    temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
                    pred_example2doc.append(idx)
                    pred_examples.append(create_example(temp_tokens,int(rel),temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc))    
                    pred_doc2example[idx].append(len(pred_examples)-1)
    pk.dump([pred_doc2example,pred_example2doc,pred_examples],open("./middle_data/test2_pred_examples_ner.pk","wb"))
else:
    print("loading example..")
    pred_doc2example,pred_example2doc,pred_examples=pk.load(open("./middle_data/test2_pred_examples_ner.pk","rb"))

loading example..


In [14]:
#交叉 ensemble 部分
pred_examples=np.array(pred_examples)
pred_token_pred=np.zeros((len(pred_examples),256))
pred_token_logits=np.zeros((len(pred_examples),256,9))
pred_rel_logits=np.zeros((len(pred_examples),55))
pred_token_pred=np.load("./cross_results/test2_pred_full.npy")
pred_token_logits=np.load("./cross_results/test2_logit_full.npy")
pred_rel_logits=np.load("./cross_results/test2_rel_full.npy")
pred_rel_logits.shape,len(pred_examples)

((214785, 55), 214785)

In [15]:
ner_pred_dataset=NerDataset(pred_examples)
len(ner_pred_dataset)

214785

### POINTER

#### val

In [ ]:
if not os.path.exists("./middle_data/train_plan_labels_pointer.pk"):
    train_plan_labels=[]
    for e in tqdm(text_data):
        train_plan_labels.append(_convert_example_to_plan_labels_new(e,tokenizer))
    for idx in tqdm(range(len(train_plan_labels))):
        for jdx in range(len(train_plan_labels[idx])):
            if sum(train_plan_labels[idx][jdx])==0:
                train_plan_labels[idx][jdx][0]=1
    pk.dump(train_plan_labels,open("./middle_data/train_plan_labels_pointer.pk","wb"))
else:
    print("loading...")
    train_plan_labels=pk.load(open("./middle_data/train_plan_labels_pointer.pk","rb"))

In [ ]:
if not os.path.exists("./middle_data/pred_token_info_pointer.pk"):
    pred_doc_tokens=[]
    pred_doc_token_labels=[]
    pred_doc_rels=[]
    pred_token2doc=[]
    pred_doc2doc=[]
    for idx,example in tqdm(enumerate(text_data[180000:])):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels=_convert_example_to_record_new(example,tokenizer)
        rels=(preds[idx].astype(np.int8)*np.arange(preds.shape[1]))
        rels=rels[rels>0]
        rel_labels=rels
        if preds[idx][0]==1:
            rel_labels=np.array([0]+list(rel_labels))
        for i in range(len(rel_labels)):
            pred_doc_tokens.append(tokens)
            pred_doc_rels.append(rel_labels[i])
            pred_doc_token_labels.append(labels_list[rel_labels[i]])
            pred_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
            pred_doc2doc.append(idx+180000)
    for idx in tqdm(range(len(pred_doc_token_labels))):
        for jdx in range(len(pred_doc_token_labels[idx])):
            if sum(pred_doc_token_labels[idx][jdx])==0:
                pred_doc_token_labels[idx][jdx][0]=1
    pk.dump([pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc],open("./middle_data/pred_token_info_pointer.pk","wb"))
else:
    print("loading doc..")
    pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc=pk.load(open("./middle_data/pred_token_info_pointer.pk","rb"))

In [ ]:
len(pred_doc_token_labels)

In [ ]:
if not os.path.exists("./middle_data/pred_examples_pointer.pk"):
    pred_examples=[]
    pred_example2doc=[]
    pred_doc2example=[[] for i in range(len(pred_doc_tokens))]
    for idx,(tokens,rel,labels,token2doc) in tqdm(enumerate(zip(pred_doc_tokens,pred_doc_rels,pred_doc_token_labels,pred_token2doc))):
        tfidf_svd=texts_tfidf_svd[pred_doc2doc[idx]]
        postag=text_data_postag[pred_doc2doc[idx]]
        plan_labels=train_plan_labels[pred_doc2doc[idx]]
        index=[0,new_kglabels2id['B-'+id2rels[rel]+"-OB"],new_kglabels2id['E-'+id2rels[rel]+"-OB"],\
           new_kglabels2id['B-'+id2rels[rel]+"-SUB"],new_kglabels2id['E-'+id2rels[rel]+"-SUB"]]
        plan_labels=np.concatenate([np.array(plan_labels)[:,index],np.zeros((len(plan_labels),4)).astype(np.int64)],axis=1).tolist()
        cur_idx=0
        rel_text=id2rel_text[rel]
        rel_postag=rel_data_postag[rel]
        rel_token2doc=id2rel_token2text[rel]
        while len(tokens)-cur_idx> (max_len-3-len(rel_text)):
            temp_tokens=tokens[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_labels=labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_plan_labels=plan_labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
            #训练时把这些label里面妹有 3 4 1 2 的样本去掉
            compare_labels=np.array(temp_labels).argmax(-1)
#             if (1 in  compare_labels) or (2 in  compare_labels) or (3 in  compare_labels) or (4 in  compare_labels):
            pred_example2doc.append(idx)
            pred_examples.append(create_example_new(temp_tokens,rel,temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc,temp_plan_labels))
            pred_doc2example[idx].append(len(pred_examples)-1)
            cur_idx+=max_len-3-len(rel_text)
        if cur_idx<len(tokens):
            temp_tokens=tokens[cur_idx:]
            temp_labels=labels[cur_idx:]
            temp_plan_labels=plan_labels[cur_idx:]
            temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
            compare_labels=np.array(temp_labels).argmax(-1)
#             if (1 in  compare_labels) or (2 in  compare_labels) or (3 in  compare_labels) or (4 in  compare_labels):
            pred_example2doc.append(idx)
            pred_examples.append(create_example_new(temp_tokens,rel,temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc,temp_plan_labels))    
            pred_doc2example[idx].append(len(pred_examples)-1)
    pk.dump([pred_doc2example,pred_example2doc,pred_examples],open("./middle_data/pred_examples_pointer.pk","wb"))
else:
    print("loading example..")
    pred_doc2example,pred_example2doc,pred_examples=pk.load(open("./middle_data/pred_examples_pointer.pk","rb"))

In [ ]:
len(pred_examples)

In [ ]:
ner_pred_dataset=NerDataset_pointer(pred_examples[:])
len(ner_pred_dataset) #21678

#### test

In [ ]:
if not os.path.exists("./middle_data/test_plan_labels_pointer.pk"):
    test_plan_labels=[]
    for e in tqdm(test_text_data):
        test_plan_labels.append(_convert_example_to_plan_labels_new(e,tokenizer))
    for idx in tqdm(range(len(test_plan_labels))):
        for jdx in range(len(test_plan_labels[idx])):
            if sum(test_plan_labels[idx][jdx])==0:
                test_plan_labels[idx][jdx][0]=1
    pk.dump(test_plan_labels,open("./middle_data/test_plan_labels_pointer.pk","wb"))
else:
    print("loading...")
    test_plan_labels=pk.load(open("./middle_data/test_plan_labels_pointer.pk","rb"))

In [ ]:
if not os.path.exists("./middle_data/test_pred_token_info_pointer.pk"):
    pred_doc_tokens=[]
    pred_doc_token_labels=[]
    pred_doc_rels=[]
    pred_token2doc=[]
    pred_doc2doc=[]
    for idx,example in tqdm(enumerate(test_text_data)):
        tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index,tokens,labels_list,rel_labels=_convert_example_to_record_new(example,tokenizer)
        rels=(preds[idx].astype(np.int8)*np.arange(preds.shape[1]))
        rels=rels[rels>0]
        rel_labels=rels
        if preds[idx][0]==1:
            rel_labels=np.array([0]+list(rel_labels))
        for i in range(len(rel_labels)):
            pred_doc_tokens.append(tokens)
            pred_doc_rels.append(rel_labels[i])
            pred_doc_token_labels.append(labels_list[rel_labels[i]])
            pred_token2doc.append([tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index])
            pred_doc2doc.append(idx)
    for idx in tqdm(range(len(pred_doc_token_labels))):
        for jdx in range(len(pred_doc_token_labels[idx])):
            if sum(pred_doc_token_labels[idx][jdx])==0:
                pred_doc_token_labels[idx][jdx][0]=1
    pk.dump([pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc],open("./middle_data/test_pred_token_info_pointer.pk","wb"))
else:
    print("loading doc..")
    pred_doc2doc,pred_doc_rels,pred_doc_tokens,pred_doc_token_labels,pred_token2doc=pk.load(open("./middle_data/test_pred_token_info_pointer.pk","rb"))

In [ ]:
if not os.path.exists("./middle_data/test_pred_examples_pointer.pk"):
    pred_examples=[]
    pred_example2doc=[]
    pred_doc2example=[[] for i in range(len(pred_doc_tokens))]
    for idx,(tokens,rel,labels,token2doc) in tqdm(enumerate(zip(pred_doc_tokens,pred_doc_rels,pred_doc_token_labels,pred_token2doc))):
        tfidf_svd=test_texts_tfidf_svd[pred_doc2doc[idx]]
        postag=test_text_data_postag[pred_doc2doc[idx]]
        plan_labels=test_plan_labels[pred_doc2doc[idx]]
        index=[0,new_kglabels2id['B-'+id2rels[rel]+"-OB"],new_kglabels2id['E-'+id2rels[rel]+"-OB"],\
           new_kglabels2id['B-'+id2rels[rel]+"-SUB"],new_kglabels2id['E-'+id2rels[rel]+"-SUB"]]
        plan_labels=np.concatenate([np.array(plan_labels)[:,index],np.zeros((len(plan_labels),4)).astype(np.int64)],axis=1).tolist()
        cur_idx=0
        rel_text=id2rel_text[rel]
        rel_postag=rel_data_postag[rel]
        rel_token2doc=id2rel_token2text[rel]
        while len(tokens)-cur_idx> (max_len-3-len(rel_text)):
            temp_tokens=tokens[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_labels=labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_plan_labels=plan_labels[cur_idx:cur_idx+max_len-3-len(rel_text)]
            temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
            #训练时把这些label里面妹有 3 4 1 2 的样本去掉
            compare_labels=np.array(temp_labels).argmax(-1)
#             if (1 in  compare_labels) or (2 in  compare_labels) or (3 in  compare_labels) or (4 in  compare_labels):
            pred_example2doc.append(idx)
            pred_examples.append(create_example_new(temp_tokens,rel,temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc,temp_plan_labels))
            pred_doc2example[idx].append(len(pred_examples)-1)
            cur_idx+=max_len-3-len(rel_text)
        if cur_idx<len(tokens):
            temp_tokens=tokens[cur_idx:]
            temp_labels=labels[cur_idx:]
            temp_plan_labels=plan_labels[cur_idx:]
            temp_token2doc=[e[cur_idx:cur_idx+max_len-3-len(rel_text)] if ind<2 else e for ind,e in enumerate(token2doc)]
            compare_labels=np.array(temp_labels).argmax(-1)
#             if (1 in  compare_labels) or (2 in  compare_labels) or (3 in  compare_labels) or (4 in  compare_labels):
            pred_example2doc.append(idx)
            pred_examples.append(create_example_new(temp_tokens,rel,temp_labels,temp_token2doc,tokenizer,rel_text,tfidf_svd,postag,rel_postag,rel_token2doc,temp_plan_labels))    
            pred_doc2example[idx].append(len(pred_examples)-1)
    pk.dump([pred_doc2example,pred_example2doc,pred_examples],open("./middle_data/test_pred_examples_pointer.pk","wb"))
else:
    print("loading example..")
    pred_doc2example,pred_example2doc,pred_examples=pk.load(open("./middle_data/test_pred_examples_pointer.pk","rb"))

In [ ]:
len(pred_examples)

In [ ]:
ner_pred_dataset=NerDataset_pointer(pred_examples[:])
len(ner_pred_dataset) #21678

# 关系分类

In [50]:
hidden_dropout_prob = 0.1
num_rel_labels = len(id2rels)
num_token_labels=len(id2labels)
learning_rate = 5e-5
weight_decay = 0
epochs = 3
batch_size = 16
adam_epsilon=1e-8

In [51]:
output_dir="./output_test/"
device=torch.device("cuda:2")
args=ARG(train_batch_size=batch_size,eval_batch_size=batch_size,weight_decay=weight_decay,learning_rate=learning_rate,
         adam_epsilon=adam_epsilon,num_train_epochs=epochs,warmup_steps=0,gradient_accumulation_steps=1,save_steps=313,
         max_grad_norm=1.0,model_name_or_path=output_dir,output_dir=output_dir,seed=42,device=device,n_gpu=1,
        max_steps=0,output_mode="classification",fp16=False,fp16_opt_level='O1',card_list=[0,1,2,3])

## model

In [ ]:
# class BertMulticlass_DGCNN(nn.Module):
#     def __init__(self,bert_dir,config_file,num_rel_labels,hidden_dropout_prob,embed_dim,loss_type=True,use_feature=True,alpha=1):
#         super(BertMulticlass_DGCNN,self).__init__()
#         self.num_rel_labels = num_rel_labels
#         self.alpha=alpha
#         self.bert =BertModel.from_pretrained(bert_dir,config=os.path.join(bert_dir,config_file), \
#                                              hidden_dropout_prob=hidden_dropout_prob,output_hidden_states=True, output_attentions=True)
#         self.dropout = nn.Dropout(hidden_dropout_prob)
#         self.embed=nn.Embedding(len(id2pos_BIO),embed_dim)
#         if use_feature:
#             hidden_size=self.bert.pooler.dense.out_features+embed_dim
#         else:
#             hidden_size=self.bert.pooler.dense.out_features         
#         self.DGCNN=nn.Sequential(GCNN_block(hidden_size,hidden_size,1,padding=0,dilation=1),
#                    GCNN_block(hidden_size,hidden_size,3,padding=1,dilation=1),
#                    GCNN_block(hidden_size,hidden_size,3,padding=2,dilation=2),
#                    GCNN_block(hidden_size,hidden_size,3,padding=4,dilation=4))
# #         self.dense_feature=nn.Sequential(nn.Linear(16,16),nn.ReLU())
#         if use_feature:
#             self.classifier=nn.Linear(hidden_size,num_rel_labels)
#         else:
#             self.classifier=nn.Linear(hidden_size,num_rel_labels)
#         self.loss_type=loss_type
#         self.use_feature=use_feature
#     def forward(
#         self,
#         input_ids=None,
#         attention_mask=None,
#         token_type_ids=None,
#         rel_label=None,
#         feature=None,
#         postag=None,
        
#     ):

#         outputs = self.bert(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#         )

#         sequence_output = outputs[0]
#         if self.use_feature:
#             postag_output=self.embed(postag)
#             sequence_output=torch.cat([sequence_output,postag_output],dim=-1)
#         sequence_output=F.adaptive_avg_pool1d(self.DGCNN([sequence_output.permute(0,2,1),attention_mask])[0],1).squeeze(-1)
#         sequence_output = self.dropout(sequence_output)
#         if self.use_feature:
# #             feature=self.dense_feature(feature)
# #             logits = self.classifier(torch.cat([feature,sequence_output],dim=-1))
#             logits = self.classifier(sequence_output)
#         else:
#             logits = self.classifier(sequence_output)
#         outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

#         if rel_label is not None:
#             if self.num_rel_labels == 1:
#                 #  We are doing regression
#                 loss_fct = MSELoss()
#                 loss = loss_fct(logits.view(-1), labels.view(-1))
#             else:
#                 if self.loss_type:
#                     loss_fct = clsLoss(0.3,self.alpha)
#                     loss = loss_fct(logits.view(-1, self.num_rel_labels), rel_label.view(-1,self.num_rel_labels))
#                 else:
#                     loss_fct = nn.BCEWithLogitsLoss(reduction="none")
#                     loss = ((loss_fct(logits.view(-1, self.num_rel_labels), rel_label.view(-1,self.num_rel_labels).float())).sum(dim=-1)).mean()
#             outputs = (loss,) + outputs

#         return outputs  # (loss), logits, (hidden_states), (attentions)
# def train(args, train_dataset,val_dataset, model, tokenizer):
# #     """ Train the model """
# #     if args.local_rank in [-1, 0]:
# #         tb_writer = SummaryWriter()

# #     args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
#     train_sampler = RandomSampler(train_dataset) # if args.local_rank == -1 else DistributedSampler(train_dataset)
#     train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

#     if args.max_steps > 0:
#         t_total = args.max_steps
#         args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
#     else:
#         t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

#     # Prepare optimizer and schedule (linear warmup and decay)
#     no_decay = ["bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {
#             "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#             "weight_decay": args.weight_decay,
#         },
#         {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
#     ]

#     optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
#     scheduler = get_linear_schedule_with_warmup(
#         optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
#     )

#     # Check if saved optimizer or scheduler states exist
#     if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
#         os.path.join(args.model_name_or_path, "scheduler.pt")
#     ):
#         logger.info("  loading optimizer and scheduler...")
#         # Load in optimizer and scheduler states
#         optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
# #         scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
#     else:
#         logger.info("  No optimizer and scheduler,we build a new one")        

#     if args.fp16:
#         try:
#             from apex import amp
#         except ImportError:
#             raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
#         model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

#     # multi-gpu training (should be after apex fp16 initialization)
#     if args.n_gpu > 1:
#         model = torch.nn.DataParallel(model,device_ids=args.card_list)


#     # Train!
#     logger.info("***** Running training *****")
#     logger.info("  Num examples = %d", len(train_dataset))
#     logger.info("  Num Epochs = %d", args.num_train_epochs)
# #     logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
#     logger.info(
#         "  Total train batch size (w. parallel, distributed & accumulation) = %d",
#         args.train_batch_size
#         * args.gradient_accumulation_steps
#     )
#     logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
#     logger.info("  Total optimization steps = %d", t_total)

#     global_step = 0
#     epochs_trained = 0
#     steps_trained_in_current_epoch = 0
#     # Check if continuing training from a checkpoint
#     if os.path.exists(args.model_name_or_path):
#         # set global_step to global_step of last saved checkpoint from model path
#         try:
#             global_step = int(args.model_name_or_path.split("-")[-1].split("/")[0])
#         except ValueError:
#             global_step = 0
#         epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
#         steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

#         logger.info("  Continuing training from checkpoint, will skip to saved global_step")
#         logger.info("  Continuing training from epoch %d", epochs_trained)
#         logger.info("  Continuing training from global step %d", global_step)
#         logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)

#     tr_loss, logging_loss = 0.0, 0.0
#     model.zero_grad()
#     train_iterator = tqdm(range(
#         epochs_trained, int(args.num_train_epochs)), desc="Epoch")
#     set_seed(args)  # Added here for reproductibility
#     for _ in train_iterator:
#         start=time.time()
#         epoch_iterator = tqdm(train_dataloader, desc="Iteration")

#         for step, batch in enumerate(epoch_iterator):

#             # Skip past any already trained steps if resuming training
#             if steps_trained_in_current_epoch > 0:
#                 if  (step + 1) % args.gradient_accumulation_steps == 0: 
#                         steps_trained_in_current_epoch -= 1
#                 continue

#             model.train()
#             batch = tuple(t.to(args.device) for t in batch[:-1])
#             inputs = {"input_ids": batch[0], "attention_mask": batch[1], "rel_label": batch[3],\
#                      "postag":batch[4],"feature":batch[5]}
# #             inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
#             inputs["token_type_ids"]=batch[2]
# #             if args.model_type != "distilbert":
# #                 inputs["token_type_ids"] = (
# #                     batch[2] if args.model_type in ["bert", "xlnet", "albert"] else None
# #                 )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
#             outputs = model(**inputs)
#             loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

#             if args.n_gpu > 1:
#                 loss = loss.mean()  # mean() to average on multi-gpu parallel training
#             if args.gradient_accumulation_steps > 1:
#                 loss = loss / args.gradient_accumulation_steps

#             if args.fp16:
#                 with amp.scale_loss(loss, optimizer) as scaled_loss:
#                     scaled_loss.backward()
#             else:
#                 loss.backward()
#             logger.info("  step:%d loss %.3f", step,loss.item())

#             tr_loss += loss.item()
#             if (step + 1) % args.gradient_accumulation_steps == 0:
#                 if args.fp16:
#                     torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
#                 else:
#                     torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

#                 optimizer.step()
#                 scheduler.step()  # Update learning rate schedule
#                 model.zero_grad()
#                 global_step += 1


#                 if  args.save_steps > 0 and global_step % args.save_steps == 0:
#                     # Save model checkpoint
#                     results = evaluate(args, val_dataset,model, tokenizer)
#                     output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
#                     if not os.path.exists(output_dir):
#                         os.makedirs(output_dir)
#                     model_to_save = (
#                         model.module if hasattr(model, "module") else model
#                     )  # Take care of distributed/parallel training
#                     torch.save(model_to_save.state_dict(),os.path.join(output_dir,"model.pt"))
#                     tokenizer.save_pretrained(output_dir)

#                     torch.save(args, os.path.join(output_dir, "training_args.bin"))
#                     logger.info("Saving model checkpoint to %s", output_dir)

#                     torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
#                     torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
#                     if args.fp16:
#                         torch.save(amp.state_dict(),os.path.join(output_dir, "amp.pt"))
#                     logger.info("Saving optimizer and scheduler states to %s", output_dir)

#             if args.max_steps > 0 and global_step > args.max_steps:
#                 epoch_iterator.close()
#                 break
#         print(time.time()-start)
#         if args.max_steps > 0 and global_step > args.max_steps:
#             train_iterator.close()
#             break


#     return global_step, tr_loss / global_step
# def evaluate(args, eval_dataset,model, tokenizer,thre=0.5):
#     eval_output_dir = args.output_dir 

#     results = {}
# #         eval_dataset = load_and_cache_examples(args, eval_task, tokenizer, evaluate=True)

#     if not os.path.exists(eval_output_dir) :
#         os.makedirs(eval_output_dir)

# #         args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
#     # Note that DistributedSampler samples randomly
#     eval_sampler = SequentialSampler(eval_dataset)
#     eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

#     # multi-gpu eval
#     if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
#         model = torch.nn.DataParallel(model,device_ids=args.card_list)

#     # Eval!
#     logger.info("***** Running evaluation  *****")
#     logger.info("  Num examples = %d", len(eval_dataset))
#     logger.info("  Batch size = %d", args.eval_batch_size)
#     eval_loss = 0.0
#     nb_eval_steps = 0
#     preds = None
#     out_label_ids = None
#     rights,num=0,0
#     recall_num,full_num=0,0
#     precision_num,full_num_pre=0,0
#     for batch in tqdm(eval_dataloader, desc="Evaluating"):
#         model.eval()
#         batch = tuple(t.to(args.device) for t in batch[:-1])

#         with torch.no_grad():
#             inputs = {"input_ids": batch[0], "attention_mask": batch[1], "rel_label": batch[3],\
#                      "postag":batch[4],"feature":batch[5]}
# #             inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
#             inputs["token_type_ids"]=batch[2]
# #                 if args.model_type != "distilbert":
# #                     inputs["token_type_ids"] = (
# #                         batch[2] if args.model_type in ["bert", "xlnet", "albert"] else None
# #                     )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
#             outputs = model(**inputs)
#             tmp_eval_loss, logits = outputs[:2]

#             eval_loss += tmp_eval_loss.mean().item()
#         rights+=( (F.sigmoid(logits).detach()>thre).long()==batch[3]).all(dim=-1).sum().detach().cpu().item()
#         num+=batch[3].shape[0]
#         recall_num+=(( (F.sigmoid(logits).detach()>thre).long()==1) & (batch[3]==1)).sum().detach().cpu().item()
#         precision_num+=(( (F.sigmoid(logits).detach()>thre).long()==1) & (batch[3]==1)).sum().detach().cpu().item()
#         full_num+=batch[3].sum().detach().cpu().item()
#         full_num_pre+= (F.sigmoid(logits).detach()>thre).long().sum().detach().cpu().item()
#         nb_eval_steps += 1
#         if preds is None:
#             preds = F.sigmoid(logits).detach().cpu().numpy()
#             out_label_ids = inputs["rel_label"].detach().cpu().numpy()
# #             out_label_ids=inputs["labels"].detach().cpu().numpy()
#         else:
#             preds = np.append(preds, F.sigmoid(logits).detach().cpu().numpy(), axis=0)
#             out_label_ids = np.append(out_label_ids, inputs["rel_label"].detach().cpu().numpy(), axis=0)
# #             out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

#     eval_loss = eval_loss / nb_eval_steps
#     if args.output_mode == "classification":
#         preds = (preds>thre).astype(np.int8)
#         out_label_ids=out_label_ids.astype(np.int8)
#     elif args.output_mode == "regression":
#         preds = np.squeeze(preds)
#     result = {"acc_class":(preds==out_label_ids).sum(axis=0)/preds.shape[0],"acc_sample":rights/(num+0.001),\
#               "recall_sample":recall_num/(full_num+0.001),"precision_sample":precision_num/(full_num_pre+0.001)}
#     results.update(result)

#     output_eval_file = os.path.join(eval_output_dir,  "eval_results.txt")
#     with open(output_eval_file, "a") as writer:
#         logger.info("***** Eval results  *****")
#         for key in sorted(result.keys()):
#             logger.info("  %s = %s", key, str(result[key]))
#             writer.write("%s = %s\n" % (key, str(result[key])))

#     return results,preds,out_label_ids
# epoch_iterator=Data.DataLoader(cls_val_dataset,batch_size=2,shuffle=False)
# for step, batch in enumerate(epoch_iterator):
#     model_cls.train()
#     tokenvec=batch[6].detach().cpu().numpy()
#     batch[6]=torch.tensor(word2vec[tokenvec]).float()
#     wordvec=batch[7].detach().cpu().numpy()
#     batch[7]=torch.tensor(word2vec[wordvec]).float()
# #     batch = tuple(t.to(args.device) for t in batch[:-1])
#     inputs = {"input_ids": batch[0], "attention_mask": batch[1], "rel_label": batch[3],\
#              "postag":batch[4],"feature":batch[5],"tokenvec":batch[6],"wordvec":batch[7],"wordmask":batch[8]}
#     inputs["token_type_ids"]=batch[2]
#     outputs = model_cls(**inputs)
#     outputs[0].backward()
#     break

In [25]:
class clsLoss(nn.Module):
    def __init__(self,m,alpha):
        super(clsLoss,self).__init__()
        self.m=m
        self.alpha=alpha
    def forward(self,logits,targets):
        y_hat=torch.sigmoid(logits)
        y=targets
        theta=(((y_hat-0.5)*(y.float()-0.5))<0) | ((y_hat>0.5-self.m) & (y_hat<0.5+self.m))
        loss_p=-(theta.float()*y.float()*torch.log(y_hat+1e-5)).sum(dim=-1)/y.float().sum(dim=-1)
        loss_n=-(theta.float()*(1-y).float()*torch.log((1-y_hat)+1e-5)).sum(dim=-1)/(1-y.float()).sum(dim=-1)
#         loss_n=-torch.log(1-(theta.float()*(1-y).float()*y_hat).sum(dim=-1)/(1-y).float().sum(dim=-1))
        loss=loss_p+self.alpha*loss_n
        return loss.mean()
class GCNN_block(nn.Module):
    def __init__(self,input_channel,output_channel,kernel_size,padding,dilation=1):
        super(GCNN_block,self).__init__()
        self.input_channel=input_channel
        self.output_channel=output_channel
        self.conv1=nn.Conv1d(input_channel,output_channel,kernel_size,padding=padding,dilation=dilation)
        self.conv2=nn.Conv1d(input_channel,output_channel,kernel_size,padding=padding,dilation=dilation)
        if input_channel !=output_channel:
            self.trans=nn.Conv1d(input_channel,output_channel,1)
    def forward(self,args):
        X,attention_mask=args[0],args[1]
        X=X*attention_mask.unsqueeze(1).float()
        gate=torch.sigmoid(self.conv2(X))
        if self.input_channel==self.output_channel:
            Y=X*(1-gate)+self.conv1(X)*gate
        else:
            Y=self.trans(X)*(1-gate)+self.conv1(X)*gate
        Y=Y*attention_mask.unsqueeze(1).float()
        return Y,attention_mask
class Conditional_LayerNorm(nn.Module):
    def __init__(self,features,conditional_dim,eps=1e-6):
        super(Conditional_LayerNorm,self).__init__()
        self.gamma=nn.Parameter(torch.ones(features))
        self.beta=nn.Parameter(torch.zeros(features))
        self.trans_gamma=nn.Linear(conditional_dim,self.gamma.shape[-1])
        self.trans_beta=nn.Linear(conditional_dim,self.beta.shape[-1])
        torch.nn.init.constant_(self.trans_gamma.weight,val=0)
        torch.nn.init.constant_(self.trans_gamma.bias,val=0)
        torch.nn.init.constant_(self.trans_beta.weight,val=0)
        torch.nn.init.constant_(self.trans_beta.bias,val=0)
        self.eps=eps
    def forward(self,X,condition):
        mean=X.mean(-1,keepdim=True)
        std=X.std(-1,keepdim=True)
        cond_gamma=self.trans_gamma(condition)
        cond_beta=self.trans_beta(condition)
        if condition.dim()<X.dim(): #condition是固定维度
            return (self.gamma+cond_gamma).unsqueeze(1)*(X-mean)/(std+self.eps)+(self.beta+cond_beta).unsqueeze(1)
        else:#condition是sequence
            return (self.gamma+cond_gamma)*(X-mean)/(std+self.eps)+(self.beta+cond_beta)
class BertMulticlass_DGCNN(nn.Module):
    def __init__(self,bert_dir,config_file,num_rel_labels,hidden_dropout_prob,embed_dim,                 loss_type=True,use_feature=True,use_vec=False,use_word=False,use_pool=False,use_plan=False,alpha=1):
        super(BertMulticlass_DGCNN,self).__init__()
        self.num_rel_labels = num_rel_labels
        self.alpha=alpha
        self.bert =BertModel.from_pretrained(bert_dir,config=os.path.join(bert_dir,config_file),                                              hidden_dropout_prob=hidden_dropout_prob,output_hidden_states=True, output_attentions=True)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.embed=nn.Embedding(len(id2pos_BIO),embed_dim)
        if use_feature:
            hidden_size=self.bert.pooler.dense.out_features+embed_dim
        else:
            hidden_size=self.bert.pooler.dense.out_features      
        if use_vec and use_feature:
#             hidden_size+=200
            self.trans=nn.Linear(200,hidden_size-embed_dim)
        if use_vec and not use_feature:
            self.trans=nn.Linear(200,hidden_size)
        self.DGCNN=nn.Sequential(GCNN_block(hidden_size,hidden_size,1,padding=0,dilation=1),
                   GCNN_block(hidden_size,hidden_size,3,padding=1,dilation=1),
                   GCNN_block(hidden_size,hidden_size,3,padding=2,dilation=2),
                   GCNN_block(hidden_size,hidden_size,3,padding=4,dilation=4))
#         self.dense_feature=nn.Sequential(nn.Linear(16,16),nn.ReLU())
        logit_size=hidden_size
        if use_word:
            self.DGCNN_vec=nn.Sequential(GCNN_block(200,200,1,padding=0,dilation=1),
                       GCNN_block(200,200,3,padding=1,dilation=1),
                       GCNN_block(200,200,3,padding=2,dilation=2),
                       GCNN_block(200,200,3,padding=4,dilation=4))
            logit_size+=200
        if use_plan:
            self.cond_ln=Conditional_LayerNorm(hidden_size,112)
#             self.DGCNN_plan=nn.Sequential(GCNN_block(112,112,1,padding=0,dilation=1),
#                        GCNN_block(112,112,3,padding=1,dilation=1),
#                        GCNN_block(112,112,3,padding=2,dilation=2),
#                        GCNN_block(112,112,3,padding=4,dilation=4))
#             logit_size+=112
        if use_pool:#cat or +
            logit_size+=self.bert.pooler.dense.out_features 
        self.classifier=nn.Linear(logit_size,num_rel_labels)
        self.loss_type=loss_type
        self.use_feature=use_feature
        self.use_vec=use_vec
        self.use_word=use_word
        self.use_pool=use_pool
        self.use_plan=use_plan
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        rel_label=None,
        feature=None,
        postag=None,
        tokenvec=None,
        wordvec=None,
        wordmask=None,
        plan_label=None
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs[0]
        pool_output=outputs[1]
        if self.use_vec:
            tokenvec=self.trans(tokenvec)
            sequence_output+=tokenvec
#             sequence_output=torch.cat([sequence_output,tokenvec],dim=-1)
        if self.use_feature:
            postag_output=self.embed(postag)
            sequence_output=torch.cat([sequence_output,postag_output],dim=-1)
        if self.use_plan:
#             plan_output=F.adaptive_avg_pool1d(self.DGCNN_plan([plan_label.permute(0,2,1),attention_mask])[0],1).squeeze(-1)
            sequence_output=self.cond_ln(sequence_output,plan_label)
        sequence_output=F.adaptive_avg_pool1d(self.DGCNN([sequence_output.permute(0,2,1),attention_mask])[0],1).squeeze(-1)
        sequence_output = self.dropout(sequence_output)
        if self.use_word:
            word_output=F.adaptive_avg_pool1d(self.DGCNN_vec([wordvec.permute(0,2,1),wordmask])[0],1).squeeze(-1)
            word_output=self.dropout(word_output)
#         if self.use_plan:
#             plan_output=F.adaptive_avg_pool1d(self.DGCNN_plan([plan_label.permute(0,2,1),attention_mask])[0],1).squeeze(-1)
#             plan_output=self.dropout(plan_output)
        logits_input=sequence_output
        if self.use_word:
            logits_input=torch.cat([logits_input,word_output],dim=-1)
#         if self.use_plan:
#             logits_input=torch.cat([logits_input,plan_output],dim=-1)    
        if self.use_pool:
            pool_output=self.dropout(pool_output)
            logits_input=torch.cat([logits_input,pool_output],dim=-1) 
        logits = self.classifier(logits_input)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if rel_label is not None:
            if self.num_rel_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                if self.loss_type:
                    loss_fct = clsLoss(0.3,self.alpha)
                    loss = loss_fct(logits.view(-1, self.num_rel_labels), rel_label.view(-1,self.num_rel_labels))
                else:
                    loss_fct = nn.BCEWithLogitsLoss(reduction="none")
                    loss = ((loss_fct(logits.view(-1, self.num_rel_labels), rel_label.view(-1,self.num_rel_labels).float())).sum(dim=-1)).mean()
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)
class BertMulticlass(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

        self.init_weights()
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.BCEWithLogitsLoss(reduction="none")
                loss = ((loss_fct(logits.view(-1, self.num_labels), labels.view(-1,self.num_labels).float())).sum(dim=-1)).mean()
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)
def train_cls(args, train_dataset,val_dataset, model, tokenizer):
#     """ Train the model """
#     if args.local_rank in [-1, 0]:
#         tb_writer = SummaryWriter()

#     args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) # if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)
    model.train()
    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
        os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        logger.info("  loading optimizer and scheduler...")
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
#         scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
    else:
        logger.info("  No optimizer and scheduler,we build a new one")        

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model,device_ids=args.card_list)


    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
#     logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        # set global_step to global_step of last saved checkpoint from model path
        try:
            global_step = int(args.model_name_or_path.split("-")[-1].split("/")[0])
        except ValueError:
            global_step = 0
        epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
        steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

        logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        logger.info("  Continuing training from epoch %d", epochs_trained)
        logger.info("  Continuing training from global step %d", global_step)
        logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = tqdm(range(
        epochs_trained, int(args.num_train_epochs)), desc="Epoch")
    set_seed(args)  # Added here for reproductibility
    for _ in train_iterator:
        start=time.time()
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")

        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                if  (step + 1) % args.gradient_accumulation_steps == 0: 
                        steps_trained_in_current_epoch -= 1
                continue

            model.train()
            if batch[6].sum()!=0:
                tokenvec=batch[6].detach().cpu().numpy()
                batch[6]=torch.tensor(word2vec[tokenvec]).float()
                wordvec=batch[7].detach().cpu().numpy()
                batch[7]=torch.tensor(word2vec[wordvec]).float()
            else:
                batch[6]=torch.tensor(batch[6])
                batch[7]=torch.tensor(batch[7])
            batch = tuple(t.to(args.device) for t in batch[:-1])
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "rel_label": batch[3],\
                      "postag":batch[4],"feature":batch[5],"tokenvec":batch[6],"wordvec":batch[7],\
                      "wordmask":batch[8],"plan_label":batch[9].float()}
            inputs["token_type_ids"]=batch[2]
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            logger.info("  step:%d loss %.3f", step,loss.item())

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1


                if  args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    results = evaluate_cls(args, val_dataset,model, tokenizer)
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    torch.save(model_to_save.state_dict(),os.path.join(output_dir,"model.pt"))
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    if args.fp16:
                        torch.save(amp.state_dict(),os.path.join(output_dir, "amp.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        print(time.time()-start)
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break


    return global_step, tr_loss / global_step
def evaluate_cls(args, eval_dataset,model, tokenizer, thre=0.5):
    eval_output_dir = args.output_dir 

    results = {}
#         eval_dataset = load_and_cache_examples(args, eval_task, tokenizer, evaluate=True)

    if not os.path.exists(eval_output_dir) :
        os.makedirs(eval_output_dir)

#         args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # multi-gpu eval
#         if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
#             model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    full_logits=[]
    out_label_ids = None
    rights,num=0,0
    recall_num,full_num=0,0
    precision_num,full_num_pre=0,0
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        tokenvec=batch[6].detach().cpu().numpy()
        batch[6]=torch.tensor(word2vec[tokenvec]).float()
        wordvec=batch[7].detach().cpu().numpy()
        batch[7]=torch.tensor(word2vec[wordvec]).float()
        batch = tuple(t.to(args.device) for t in batch[:-1])


        with torch.no_grad():

            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "rel_label": batch[3],\
                      "postag":batch[4],"feature":batch[5],"tokenvec":batch[6],"wordvec":batch[7],\
                      "wordmask":batch[8],"plan_label":batch[9].float()}
            inputs["token_type_ids"]=batch[2]
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        rights+=((torch.sigmoid(logits.detach())>thre).long()==batch[3]).all(dim=-1).sum().detach().cpu().item()
        num+=batch[3].shape[0]
        recall_num+=(((torch.sigmoid(logits.detach())>thre).long()==1) & (batch[3]==1)).sum().detach().cpu().item()
        precision_num+=(((torch.sigmoid(logits.detach())>thre).long()==1) & (batch[3]==1)).sum().detach().cpu().item()
        full_num+=batch[3].sum().detach().cpu().item()
        full_num_pre+=(torch.sigmoid(logits.detach())>thre).long().sum().detach().cpu().item()
        nb_eval_steps += 1
        full_logits.append((torch.sigmoid(logits.detach())).detach().cpu().numpy())
        if preds is None:
            preds = (torch.sigmoid(logits.detach())>thre).detach().cpu().numpy()
            out_label_ids = inputs["rel_label"].detach().cpu().numpy()
#             out_label_ids=inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, (torch.sigmoid(logits.detach())>thre).detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["rel_label"].detach().cpu().numpy(), axis=0)
#             out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args.output_mode == "classification":
        preds = preds.astype(np.int8)
        out_label_ids=out_label_ids.astype(np.int8)
    elif args.output_mode == "regression":
        preds = np.squeeze(preds)
    result = {"acc_class":(preds==out_label_ids).sum(axis=0)/preds.shape[0],"acc_sample":rights/(num+0.001),\
              "recall_sample":recall_num/(full_num+0.001),"precision_sample":precision_num/(full_num_pre+0.001)}
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "a") as writer:
        logger.info("***** Eval results *****")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return results,preds,out_label_ids,full_logits

In [53]:
model_cls=BertMulticlass_DGCNN(bert_dir,config_file,num_rel_labels=num_rel_labels,
                  hidden_dropout_prob=hidden_dropout_prob,embed_dim=128,loss_type=True,use_feature=True,use_vec=False,use_word=False,use_pool=True,use_plan=True,alpha=1)

06/18/2020 15:42:25 - INFO - transformers.configuration_utils -   loading configuration file ./roberta-zh-wwm-pytorch/bert_config.json
06/18/2020 15:42:25 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_return_sequences": 1,
  "output_attentions": true,
 

In [64]:
set_seed(args)
model_cls.to(args.device)
global_step, tr_loss = train(args,cls_train_dataset,cls_val_dataset, model_cls, tokenizer)

In [27]:
# ckpt=torch.load("./output_1_1_0_1_1_1_new/checkpoint-35988/model.pt", map_location=lambda storage, loc: storage)
# model_cls.load_state_dict(ckpt)
# model_cls=model_cls.to(args.device)
results,preds,out_label_ids,full_logits=evaluate(args,cls_test_dataset,model_cls,tokenizer,thre=0.5)
#获取test集/val集(在进行两阶段模型综合性能测试时)的关系分类结果后，进入1.9构建结合句子和本阶段模型输出的分类结果的关系实体识别数据集ner_pred_dataset
#用于二阶段模型

# full_logits=np.concatenate(full_logits,axis=0)
# np.save("./cross_results/test_cls_new.npy",full_logits)
# full_logits.shape

In [19]:
#交叉ensemble部分
preds=[]

for idx in range(3):
    preds.append(np.load("./cross_results/test_cls_"+str(idx)+".npy"))
preds.append(np.load("./cross_results/test_cls_full.npy"))
preds.append(np.load("./cross_results/test_cls_new.npy"))
preds=np.stack(preds,axis=2)
preds=preds.mean(axis=-1)
preds=(preds>0.5).astype(np.int)

# 序列标注

## models set

In [14]:
hidden_dropout_prob = 0.1
num_rel_labels = len(id2rels)
num_token_labels=len(id2labels)
learning_rate = 5e-5
weight_decay = 0
epochs = 3
batch_size = 16
adam_epsilon=1e-8

In [21]:
class BERT_BiLSTM_CRF(nn.Module):
    def __init__(self,bert_dir,config_file,num_rel_labels,num_token_labels,hidden_dropout_prob,need_birnn=True,need_cnn=False,rnn_dim=128):
        super(BERT_BiLSTM_CRF, self).__init__()
        self.num_rel_labels = num_rel_labels
        self.num_token_labels= num_token_labels
        self.bert =BertModel.from_pretrained(bert_dir,config=os.path.join(bert_dir,config_file), \
                                             hidden_dropout_prob=hidden_dropout_prob,output_hidden_states=True, output_attentions=True)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        out_dim =self.bert.pooler.dense.out_features
        hidden_size=out_dim
        self.rel_classifier = nn.Linear(out_dim, self.num_rel_labels)
        self.need_birnn = need_birnn
        self.need_cnn=need_cnn
        # 如果为False，则不要BiLSTM层
        if need_birnn:
            self.birnn = nn.LSTM(out_dim, rnn_dim, num_layers=1, bidirectional=True, batch_first=True)
            out_dim = rnn_dim*2
        if need_cnn:
            self.DGCNN=nn.Sequential(GCNN_block(hidden_size,hidden_size,1,padding=0,dilation=1),
                       GCNN_block(hidden_size,hidden_size,3,padding=1,dilation=1),
                       GCNN_block(hidden_size,hidden_size,3,padding=2,dilation=2),
                       GCNN_block(hidden_size,hidden_size,3,padding=4,dilation=4))
            out_dim=hidden_size
            #attention/batch norm+relu/ pooling/什么都不用
        
        self.hidden2tag = nn.Linear(out_dim, self.num_token_labels)
        self.crf = CRF(self.num_token_labels, batch_first=True)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        label_tokens=None
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        seq_output,pooled_output = outputs[0],outputs[1]
        pooled_output = self.dropout(pooled_output)
        emissions=self.tag_outputs(seq_output,attention_mask)
        rel_logits = self.rel_classifier(pooled_output)
        seq_logits = emissions
        outputs = (rel_logits,seq_logits) + outputs[2:]  # add hidden states and attention if they are here
        loss_full=None
        if labels is not None:
            loss_rel = nn.BCEWithLogitsLoss(reduction="none")
            l_r = ((loss_rel(rel_logits.view(-1, self.num_rel_labels), labels.view(-1,self.num_rel_labels).float())).sum(dim=-1)).mean()
            loss_full=l_r

        if label_tokens is not None:
            l_t = -1*self.crf(emissions, label_tokens, mask=attention_mask.byte(),reduction='mean')
            loss_full+=l_t
        if loss_full is not None:
            outputs = (loss_full,) + outputs
        return outputs  # (loss), logits, (hidden_states), (attentions)

    def tag_outputs(self, sequence_output,attention_mask):
        if self.need_birnn:
            sequence_output, _ = self.birnn(sequence_output)
        if self.need_cnn:
            sequence_output=self.DGCNN([sequence_output.permute(0,2,1),attention_mask])[0].permute(0,2,1)
        sequence_output = self.dropout(sequence_output)
        emissions = self.hidden2tag(sequence_output)
        return emissions
    
    def predict(self, input_ids,attention_mask=None,token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask,token_type_ids=token_type_ids )
        sequence_output = outputs[0]
        emissions = self.tag_outputs(sequence_output,attention_mask)
        return self.crf.decode(emissions, attention_mask.byte())
    def logits_predict(self, emissions,attention_mask=None):
        return self.crf.decode(emissions, attention_mask.byte())
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    def __init__(self):
            super(MaskedSoftmaxCELoss,self).__init__(reduction='none')
    def forward(self,pred,label,mask=None):
            loss=super(MaskedSoftmaxCELoss,self).forward((pred).contiguous().view(-1,pred.shape[-1]),(label).contiguous().view(-1).long()).view(label.shape)
            if mask is not None:
                loss*=mask.float().contiguous()
            return loss
class BertSeqLabeling(nn.Module):
    def __init__(self,bert_dir,config_file,num_rel_labels,num_token_labels,hidden_dropout_prob):
        super(BertSeqLabeling,self).__init__()
        self.num_rel_labels = num_rel_labels
        self.num_token_labels= num_token_labels
        self.bert =BertModel.from_pretrained(bert_dir,config=os.path.join(bert_dir,config_file), \
                                             hidden_dropout_prob=hidden_dropout_prob,output_hidden_states=True, output_attentions=True)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.rel_classifier = nn.Linear(self.bert.pooler.dense.out_features, self.num_rel_labels)
        self.seq_classifier=nn.Linear(self.bert.pooler.dense.out_features,self.num_token_labels)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        label_tokens=None,
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        seq_output,pooled_output = outputs[0],outputs[1]
        pooled_output = self.dropout(pooled_output)
        seq_output=self.dropout(seq_output)
        rel_logits = self.rel_classifier(pooled_output)
        seq_logits = self.seq_classifier(seq_output)
        outputs = (rel_logits,seq_logits) + outputs[2:]  # add hidden states and attention if they are here
        loss_full=None
        if labels is not None:
            loss_rel = nn.BCEWithLogitsLoss(reduction="none")
            l_r = ((loss_rel(rel_logits.view(-1, self.num_rel_labels), labels.view(-1,self.num_rel_labels).float())).sum(dim=-1)).mean()
            loss_full=0.5*l_r

        if label_tokens is not None:
            loss_token =MaskedSoftmaxCELoss()
            # Only keep active parts of the loss
            if attention_mask is not None:
                l_t = (loss_token(seq_logits, label_tokens,attention_mask)).sum(dim=-1).mean()
            else:
                l_t = loss_token(seq_logits, label_tokens).sum(dim=-1).mean()
            loss_full+=l_t
        if loss_full is not None:
            outputs = (loss_full,) + outputs
        return outputs  # (loss), logits, (hidden_states), (attentions)
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)
def train_ner(args, train_dataset,val_dataset, model, tokenizer):
#     """ Train the model """
#     if args.local_rank in [-1, 0]:
#         tb_writer = SummaryWriter()

#     args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) # if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
        os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        logger.info("  loading optimizer and scheduler...")
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
#         scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
    else:
        logger.info("  No optimizer and scheduler,we build a new one")        

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model,device_ids=args.card_list)


    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
#     logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
#         * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        # set global_step to global_step of last saved checkpoint from model path
        try:
            global_step = int(args.model_name_or_path.split("-")[-1].split("/")[0])
        except ValueError:
            global_step = 0
        epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
        steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

        logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        logger.info("  Continuing training from epoch %d", epochs_trained)
        logger.info("  Continuing training from global step %d", global_step)
        logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = tqdm(range(
        epochs_trained, int(args.num_train_epochs)), desc="Epoch")
    set_seed(args)  # Added here for reproductibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                if  (step + 1) % args.gradient_accumulation_steps == 0: 
                        steps_trained_in_current_epoch -= 1
                continue

            model.train()
            batch = tuple(t.to(args.device) for t in batch[:-1])
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3],"label_tokens":batch[4]}
            inputs["token_type_ids"]=batch[2]
#             if args.model_type != "distilbert":
#                 inputs["token_type_ids"] = (
#                     batch[2] if args.model_type in ["bert", "xlnet", "albert"] else None
#                 )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            logger.info("  step:%d loss %.3f", step,loss.item())

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if  args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    results = evaluate_ner(args, val_dataset,model, tokenizer)
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    torch.save(model_to_save.state_dict(),os.path.join(output_dir,"model.pt"))
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)
                    if args.fp16:
                        torch.save(amp.state_dict(),os.path.join(output_dir, "amp.pt"))
                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break


    return global_step, tr_loss / global_step
def evaluate_ner(args, eval_dataset,model, tokenizer, prefix=""):
    eval_output_dir = args.output_dir 

    results = {}

    if not os.path.exists(eval_output_dir) :
        os.makedirs(eval_output_dir)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=0)

    # multi-gpu eval
#         if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
#             model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    rel_preds=None
    rel_labels=None
    token_preds=None
    token_labels=None
    full_token_logits=[]
    full_rel_logits=[]
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch[:-1])

        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3],"label_tokens":batch[4]}
            inputs["token_type_ids"]=batch[2]
            temp_inputs={"input_ids": batch[0], "attention_mask": batch[1],"token_type_ids":batch[2]}
#                 if args.model_type != "distilbert":
#                     inputs["token_type_ids"] = (
#                         batch[2] if args.model_type in ["bert", "xlnet", "albert"] else None
#                     )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
            outputs = model(**inputs)
            model_to_predict = (
                model.module if hasattr(model, "module") else model
            )  # Take care of distributed/parallel training
            token_pred=model_to_predict.predict(**temp_inputs)
            tmp_eval_loss, rel_logits, token_logits = outputs[:3]
            
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        mask=batch[1].view(-1).detach().cpu().numpy()
        rel_logits = F.softmax(rel_logits,dim=-1)
        rel_pred =torch.argmax(rel_logits,dim=-1).detach().cpu().numpy()
        rel_label=torch.argmax(inputs["labels"],dim=-1).detach().cpu().numpy()
        full_rel_logits.append(rel_logits.detach().cpu().numpy())
        if rel_preds is None:
            rel_preds=rel_pred
        else:
            rel_preds=np.append(rel_preds,rel_pred,axis=0)
        if rel_labels is None:
            rel_labels=rel_label
        else:
            rel_labels=np.append(rel_labels,rel_label,axis=0)     

            
        #token classify
        token_logits = F.softmax(token_logits,dim=-1)
        token_logits =token_logits.view(-1).detach().cpu().numpy()
        full_token_logits.append(token_logits.reshape(-1,256,9))
        for i in range(len(token_pred)):
            token_pred[i]+=[label2ids['[PAD]']]*(max_len-len(token_pred[i]))
        token_pred=np.array(token_pred).reshape(-1)*mask
        token_label=inputs["label_tokens"].view(-1).detach().cpu().numpy()*mask
        if token_preds is None:
            token_preds=token_pred
        else:
            token_preds=np.append(token_preds,token_pred,axis=0)
        if token_labels is None:
            token_labels=token_label
        else:
            token_labels=np.append(token_labels,token_label,axis=0) 
        


    eval_loss = eval_loss / nb_eval_steps
    print("准召报告：")
    df_precison_recall_f1=pd.DataFrame(metrics.precision_recall_fscore_support(rel_labels,rel_preds),index=['precison','recall','f1','number'])
    display(HTML(df_precison_recall_f1.to_html()))
    df_precison_recall_f2=pd.DataFrame(metrics.precision_recall_fscore_support(token_labels,token_preds),index=['precison','recall','f1','number'])
    display(HTML(df_precison_recall_f2.to_html()))
    result = {"cls":    df_precison_recall_f1,'ner':    df_precison_recall_f2}
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "a") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return rel_labels,rel_preds,token_preds,token_labels,full_token_logits,full_rel_logits
class GCNN_block(nn.Module):
    def __init__(self,input_channel,output_channel,kernel_size,padding,dilation=1):
        super(GCNN_block,self).__init__()
        self.input_channel=input_channel
        self.output_channel=output_channel
        self.conv1=nn.Conv1d(input_channel,output_channel,kernel_size,padding=padding,dilation=dilation)
        self.conv2=nn.Conv1d(input_channel,output_channel,kernel_size,padding=padding,dilation=dilation)
        if input_channel !=output_channel:
            self.trans=nn.Conv1d(input_channel,output_channel,1)
    def forward(self,args):
        X,attention_mask=args[0],args[1]
        X=X*attention_mask.unsqueeze(1).float()
        gate=torch.sigmoid(self.conv2(X))
        if self.input_channel==self.output_channel:
            Y=X*(1-gate)+self.conv1(X)*gate
        else:
            Y=self.trans(X)*(1-gate)+self.conv1(X)*gate
        Y=Y*attention_mask.unsqueeze(1).float()
        return Y,attention_mask
class Conditional_LayerNorm(nn.Module):
    def __init__(self,features,conditional_dim,eps=1e-6):
        super(Conditional_LayerNorm,self).__init__()
        self.gamma=nn.Parameter(torch.ones(features))
        self.beta=nn.Parameter(torch.zeros(features))
        self.trans_gamma=nn.Linear(conditional_dim,self.gamma.shape[-1])
        self.trans_beta=nn.Linear(conditional_dim,self.beta.shape[-1])
        torch.nn.init.constant_(self.trans_gamma.weight,val=0)
        torch.nn.init.constant_(self.trans_gamma.bias,val=0)
        torch.nn.init.constant_(self.trans_beta.weight,val=0)
        torch.nn.init.constant_(self.trans_beta.bias,val=0)
        self.eps=eps
    def forward(self,X,condition):
        mean=X.mean(-1,keepdim=True)
        std=X.std(-1,keepdim=True)
        cond_gamma=self.trans_gamma(condition)
        cond_beta=self.trans_beta(condition)
        if condition.dim()<X.dim(): #condition是固定维度
            return (self.gamma+cond_gamma).unsqueeze(1)*(X-mean)/(std+self.eps)+(self.beta+cond_beta).unsqueeze(1)
        else:#condition是sequence
            return (self.gamma+cond_gamma)*(X-mean)/(std+self.eps)+(self.beta+cond_beta)
class BERT_BiLSTM_POINTER(nn.Module):
    def __init__(self,bert_dir,config_file,num_rel_labels,num_token_labels,hidden_dropout_prob,\
                 need_birnn=True,need_cnn=False,rnn_dim=128,use_plan=False,p=1):
        super(BERT_BiLSTM_POINTER, self).__init__()
        self.num_rel_labels = num_rel_labels
        self.num_token_labels= 4
        self.bert =BertModel.from_pretrained(bert_dir,config=os.path.join(bert_dir,config_file), \
                                             hidden_dropout_prob=hidden_dropout_prob,output_hidden_states=True, output_attentions=True)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        out_dim =self.bert.pooler.dense.out_features
        hidden_size=out_dim
        self.rel_classifier = nn.Linear(out_dim, self.num_rel_labels)
        self.need_birnn = need_birnn
        self.need_cnn=need_cnn
        self.p=p
        self.use_plan=use_plan
        if use_plan:
            self.cond_ln=Conditional_LayerNorm(hidden_size,9)
        if need_birnn:
            self.birnn = nn.LSTM(out_dim, rnn_dim, num_layers=1, bidirectional=True, batch_first=True)
            out_dim = rnn_dim*2
        if need_cnn:
            self.DGCNN=nn.Sequential(GCNN_block(hidden_size,hidden_size,1,padding=0,dilation=1),
                       GCNN_block(hidden_size,hidden_size,3,padding=1,dilation=1),
                       GCNN_block(hidden_size,hidden_size,3,padding=2,dilation=2),
                       GCNN_block(hidden_size,hidden_size,3,padding=4,dilation=4))
            out_dim=hidden_size
        self.hidden2tag = nn.Linear(out_dim, self.num_token_labels)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        label_tokens=None,
        plan_label=None
    ):
        label_tokens=(label_tokens[:,:,1:5]!=0).long()
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        seq_output,pooled_output = outputs[0],outputs[1]
        pooled_output = self.dropout(pooled_output)
        if self.use_plan:
            plan_output=F.adaptive_avg_pool1d(plan_label.permute(0,2,1),1).squeeze(-1)
            seq_output=self.cond_ln(seq_output,plan_output)
        emissions=self.tag_outputs(seq_output,attention_mask)
        rel_logits = self.rel_classifier(pooled_output)
        seq_logits = emissions
        outputs = (rel_logits,seq_logits) + outputs[2:]  # add hidden states and attention if they are here
        loss_full=None
        if labels is not None:
            loss_rel = nn.BCEWithLogitsLoss(reduction="none")
            l_r = ((loss_rel(rel_logits.view(-1, self.num_rel_labels), labels.view(-1,self.num_rel_labels).float())).sum(dim=-1)).mean()
            loss_full=l_r

        if label_tokens is not None:
            #(batch_size,seq_len,num_token_labels)
            loss_token=nn.BCELoss(reduction='none')
            l_t=(torch.sum((loss_token(torch.sigmoid(seq_logits)**self.p, label_tokens.float()))\
                 *attention_mask.float().unsqueeze(-1)))/input_ids.shape[0]
            loss_full+=l_t
        if loss_full is not None:
            outputs = (loss_full,) + outputs
        return outputs  # (loss), logits, (hidden_states), (attentions)

    def tag_outputs(self, sequence_output,attention_mask):
        if self.need_birnn:
            sequence_output, _ = self.birnn(sequence_output)
        if self.need_cnn:
            sequence_output=self.DGCNN([sequence_output.permute(0,2,1),attention_mask])[0].permute(0,2,1)
        sequence_output = self.dropout(sequence_output)
        emissions = self.hidden2tag(sequence_output)
        return emissions
def train_pointer(args, train_dataset,val_dataset, model, tokenizer):
#     """ Train the model """
#     if args.local_rank in [-1, 0]:
#         tb_writer = SummaryWriter()

#     args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset) # if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
        os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        logger.info("  loading optimizer and scheduler...")
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
#         scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
    else:
        logger.info("  No optimizer and scheduler,we build a new one")        

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model,device_ids=args.card_list)


    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
#     logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
#         * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        # set global_step to global_step of last saved checkpoint from model path
        try:
            global_step = int(args.model_name_or_path.split("-")[-1].split("/")[0])
        except ValueError:
            global_step = 0
        epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
        steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

        logger.info("  Continuing training from checkpoint, will skip to saved global_step")
        logger.info("  Continuing training from epoch %d", epochs_trained)
        logger.info("  Continuing training from global step %d", global_step)
        logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = tqdm(range(
        epochs_trained, int(args.num_train_epochs)), desc="Epoch")
    set_seed(args)  # Added here for reproductibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                if  (step + 1) % args.gradient_accumulation_steps == 0: 
                        steps_trained_in_current_epoch -= 1
                continue

            model.train()
            batch = tuple(t.to(args.device) for t in batch[:-1])
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3],"label_tokens":batch[4],"plan_label":batch[7].float()}
            inputs["token_type_ids"]=batch[2]
#             if args.model_type != "distilbert":
#                 inputs["token_type_ids"] = (
#                     batch[2] if args.model_type in ["bert", "xlnet", "albert"] else None
#                 )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            logger.info("  step:%d loss %.3f", step,loss.item())

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if  args.save_steps > 0 and global_step % args.save_steps == 0:
                    # Save model checkpoint
                    results = evaluate_pointer(args, val_dataset,model, tokenizer)
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    torch.save(model_to_save.state_dict(),os.path.join(output_dir,"model.pt"))
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)
                    if args.fp16:
                        torch.save(amp.state_dict(),os.path.join(output_dir, "amp.pt"))
                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break


    return global_step, tr_loss / global_step
def evaluate_pointer(args, eval_dataset,model, tokenizer, thre=0.5,prefix=""):
    eval_output_dir = args.output_dir 

    results = {}

    if not os.path.exists(eval_output_dir) :
        os.makedirs(eval_output_dir)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=0)

    # multi-gpu eval
#         if args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
#             model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    rel_preds=None
    rel_labels=None
    token_preds=None
    token_labels=None
    recall_num=np.zeros((4,))
    precision_num=np.zeros((4,))
    full_num=np.zeros((4,))
    full_num_pre=np.zeros((4,))
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch[:-1])

        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3],"label_tokens":batch[4],"plan_label":batch[7].float()}
            inputs["token_type_ids"]=batch[2]
#             temp_inputs={"input_ids": batch[0], "attention_mask": batch[1],"token_type_ids":batch[2]}
#                 if args.model_type != "distilbert":
#                     inputs["token_type_ids"] = (
#                         batch[2] if args.model_type in ["bert", "xlnet", "albert"] else None
#                     )  # XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, rel_logits, token_logits = outputs[:3]
            
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        mask=batch[1].unsqueeze(-1)
        rel_pred = F.softmax(rel_logits,dim=-1)
        rel_pred =torch.argmax(rel_pred,dim=-1).detach().cpu().numpy()
        rel_label=torch.argmax(inputs["labels"],dim=-1).detach().cpu().numpy()
        if rel_preds is None:
            rel_preds=rel_pred
        else:
            rel_preds=np.append(rel_preds,rel_pred,axis=0)
        if rel_labels is None:
            rel_labels=rel_label
        else:
            rel_labels=np.append(rel_labels,rel_label,axis=0)     

            

        recall_num+=((((torch.sigmoid(token_logits.detach())>thre).long()==1) & (batch[4].detach()[:,:,1:5]!=0)).long()*mask).sum(dim=[0,1]).detach().cpu().numpy()
        precision_num+=((((torch.sigmoid(token_logits.detach())>thre).long()==1) & (batch[4].detach()[:,:,1:5]!=0)).long()*mask).sum(dim=[0,1]).detach().cpu().numpy()
        full_num+=((batch[4].detach()[:,:,1:5]!=0).long()*mask).sum(dim=[0,1]).detach().cpu().numpy()
        full_num_pre+=((torch.sigmoid(token_logits.detach())>thre).long()*mask).sum(dim=[0,1]).detach().cpu().numpy()
#         token_pred=((torch.sigmoid(token_logits.detach())>thre).long()*mask).cpu().numpy().reshape(-1,4)
        token_pred=((torch.sigmoid(token_logits.detach())*mask.float())).cpu().numpy().reshape(-1,4)
        token_label=((batch[4].detach()[:,:,1:5]!=0).long()*mask).cpu().numpy().reshape(-1,4)
        if token_preds is None:
            token_preds=token_pred
        else:
            token_preds=np.append(token_preds,token_pred,axis=0)
        if token_labels is None:
            token_labels=token_label
        else:
            token_labels=np.append(token_labels,token_label,axis=0) 
    result = {"recall_sample":recall_num/(full_num+0.001),"precision_sample":precision_num/(full_num_pre+0.001)}
    results.update(result)


    eval_loss = eval_loss / nb_eval_steps
    print("准召报告：")
    df_precison_recall_f1=pd.DataFrame(metrics.precision_recall_fscore_support(rel_labels,rel_preds),index=['precison','recall','f1','number'])
    display(HTML(df_precison_recall_f1.to_html()))
    result = {"cls":    df_precison_recall_f1}
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "a") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(results.keys()):
            logger.info("  %s = %s", key, str(results[key]))
            writer.write("%s = %s\n" % (key, str(results[key])))

    return rel_labels,rel_preds,token_preds,token_labels

In [16]:
# model=BertSeqLabeling(bert_dir,config_file,num_rel_labels=num_rel_labels,num_token_labels=num_token_labels,
#                                      hidden_dropout_prob=hidden_dropout_prob)
model_ner=BERT_BiLSTM_CRF(bert_dir,config_file,num_rel_labels=num_rel_labels,num_token_labels=num_token_labels,
                                     hidden_dropout_prob=hidden_dropout_prob, need_birnn=False,need_cnn=True)
# model_ner=BERT_BiLSTM_POINTER(bert_dir,config_file,num_rel_labels=num_rel_labels,num_token_labels=num_token_labels,
#                          hidden_dropout_prob=hidden_dropout_prob,need_birnn=False,need_cnn=True\
#                           ,use_plan=True,p=4)

06/18/2020 16:11:26 - INFO - transformers.configuration_utils -   loading configuration file ./roberta-zh-wwm-pytorch/bert_config.json
06/18/2020 16:11:26 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_return_sequences": 1,
  "output_attentions": true,
 

In [17]:
output_dir="./output_test/"
device=torch.device("cuda:2")
args=ARG(train_batch_size=batch_size*2,eval_batch_size=batch_size*4,weight_decay=weight_decay,learning_rate=learning_rate,
         adam_epsilon=adam_epsilon,num_train_epochs=epochs,warmup_steps=0,gradient_accumulation_steps=1,save_steps=313,
         max_grad_norm=1.0,model_name_or_path=output_dir,output_dir=output_dir,seed=42,device=device,n_gpu=2,
        max_steps=0,output_mode="classification",fp16=False,fp16_opt_level='O1',card_list=[2,3])

In [24]:
model_ner.to(args.device)
train_ner(args, ner_train_dataset,ner_val_dataset, model_ner, tokenizer)

In [32]:
#交叉ensemble部分

ckpt=torch.load("./output_ner_0_1_old/checkpoint-67112/model.pt", map_location=lambda storage, loc: storage)
# full "./output_ner_0_1_old/checkpoint-67112/model.pt"
# 0  "./output_ner_0_1_0/checkpoint-23872/model.pt"
# 1 "./output_ner_0_1_1/checkpoint-29840/model.pt"
#2  "./output_ner_0_1_2/checkpoint-17904/model.pt"
model_ner.load_state_dict(ckpt)
model_ner=model_ner.to(args.device)

## 单model检验（只检验二阶段命名实体识别的准确度，跟一阶段model无关）

In [ ]:
results=evaluate(args,ner_val_dataset,model,tokenizer)

In [ ]:
val_token_pred=results[2].reshape(-1,256)

In [ ]:
val_token_label=results[3].reshape(-1,256)

In [ ]:
text_labels=[np.zeros((len(text_data[ner_doc2doc[i]]['text']))) for i in range(len(ner_doc_token_labels))]
for idx in tqdm(range(len(text_labels))):
    token_labels=ner_doc_token_labels[idx]
    tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index=ner_token2doc[idx]
    for i in range(len(token_labels)):
        begin,end=tok_to_orig_start_index[i],tok_to_orig_end_index[i]
        text_labels[idx][begin:end+1]=token_labels[i]

In [ ]:
for t,l in zip(text_data[ner_doc2doc[example2doc[360000+12]]]['text'],text_labels[example2doc[360000+12]]):
        print(t," ",id2labels[int(l)])

In [ ]:
text_preds=[np.zeros((len(text_data[ner_doc2doc[i]]['text']),len(id2labels)))for i in range(len(ner_doc_token_labels))]
for idx in tqdm(range(len(ner_val_dataset))):
    text=ner_val_dataset[idx][0].numpy()
    attention_mask=ner_val_dataset[idx][1].numpy()
    token_labels=ner_val_dataset[idx][4].numpy()
    tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index=ner_val_dataset[idx][5]
    token_pred=val_token_pred[idx]
    token_pred=token_pred[1:1+len(tok_to_orig_start_index)]
    for i in range(len(token_pred)):
        begin,end=tok_to_orig_start_index[i],tok_to_orig_end_index[i]
        text_preds[example2doc[360000+idx]][begin:end+1,token_pred[i]]+=1
text_preds=[e.argmax(axis=-1) for e in text_preds]

In [ ]:
for t,l in zip(text_data[ner_doc2doc[example2doc[360000]]]['text'],text_preds[example2doc[360000]]):
        print(t," ",id2labels[l])

In [ ]:

with open("val_output.txt","w") as w:
    for idx in tqdm(range(example2doc[360000],len(text_preds))):
        for token,label,pred in zip(text_data[ner_doc2doc[idx]]['text'],text_labels[idx],text_preds[idx]):
            if _is_whitespace(token):
                continue
            w.write(token+" "+id2labels[int(label)]+" "+id2labels[int(pred)]+"\n")
        w.write("\n")

## predict（使用一阶段输出的关系分类结果和句子一起构建ner_pred_dataset,再使用二阶段模型识别关系实体，完成关系抽取）

### POINTER preprocess

In [ ]:
len(ner_pred_dataset)

In [ ]:
pred_results=evaluate_pointer(args,ner_pred_dataset,model_ner,tokenizer)

In [ ]:
pred_token_pred=pred_results[2].reshape(-1,256,4)

In [ ]:
pred_results[2].shape,pred_token_pred.shape

In [ ]:
pred_token_pred[0][0:10]

In [ ]:
len(pred_doc_token_labels),pred_example2doc[-1]

In [ ]:

def find_entities(token_label_pred,text,B_index,E_index,thre=0.5,max_count=2):
    probs=[]
    for i in range(len(text)):
        counts=0
        for j in range(i,len(text)):
            if B_index==0 and j-i+1>20:
                continue
            if B_index==2 and j-i+1>25:
                continue
            if token_label_pred[i][B_index]*token_label_pred[j][E_index]>thre and counts<max_count:
                probs.append([i,j,token_label_pred[i][B_index]*token_label_pred[j][E_index]])
                counts+=1
    probs=list(sorted(probs,key=lambda x:x[1]-x[0],reverse=True))
    contains=[[] for i in range(len(probs))]
    crosses=[]
    for idx,e in enumerate(probs):
        cross=[]
        contained=[]
        for jdx,t in enumerate(probs):
            if jdx==idx:
                continue
            if t[0]<=e[0] and t[1]>=e[1]:
                contained.append([jdx,t[1]-t[0]])
            elif ((t[1]>=e[0] and t[0]<=e[0]) or (t[0]<=e[1] and t[1]>=e[1])) and not (t[0]>=e[0] and t[1]<=e[1]):
                cross.append(jdx)
        crosses.append(cross)
        if len(contained)>0:
            father=min(contained,key=lambda x:x[1]) 
            contains[father[0]].append(idx)
    saved=np.ones((len(contains),))
    for idx in range(len(contains)):
        e=probs[idx]
#         print([(text[int(e[0]):int(e[1])+1]),e[2]])
#         print("contains")
        full_prob=probs[idx][2]
        seperate_prob=0
        for jdx in contains[idx]:
            t=probs[jdx]
#             print([(text[int(t[0]):int(t[1])+1]),t[2]])   
            seperate_prob+=probs[jdx][2]
        if seperate_prob>full_prob:
            saved[idx]=0
        else:
            saved[contains[idx]]=0
    for idx in range(len(crosses)):
        full_prob=probs[idx][2]
        cross_prob=0
        for jdx in crosses[idx]:
            if probs[jdx][2]>full_prob:
                saved[idx]=0
            else:
                saved[jdx]=0 
#     print([ (text[int(e[0]):int(e[1])+1],e[2],e[0],e[1])for e in probs]) #覆盖不可以 相交可以
    probs=np.array(probs)[saved.astype(np.bool)]
    entities=[text[int(e[0]):int(e[1])+1] for e in probs]
    offsets=[[int(e[0]),int(e[1])+1] for e in probs]
    return entities,offsets
def find_subs_obs_pointer(token_label_pred,text,sub_thre=0.7,ob_thre=0.8,max_count=2):
    obs,ob_offsets=find_entities(token_label_pred,text,0,1,ob_thre,max_count)
    subs,sub_offsets=find_entities(token_label_pred,text,2,3,sub_thre,max_count-1)
    return subs,obs,sub_offsets,ob_offsets
def make_output_pointer(doc2pred_ner_doc,simple_thre,complex_thre,text_data,pred_info,soinfos):
    for ind,info in tqdm(enumerate(doc2pred_ner_doc)):
        rels={}
        for idx in (info):
            rels[pred_doc_rels[idx]]=idx
        for idx in info:
            token_label_pred=text_preds[idx]
            label_pred=pred_doc_rels[idx]
            text=text_data[ind]['text']
            if label_pred in special_pass_idx:
                continue
            elif label_pred not in special_major_idx:
                subs,obs,sub_offsets,ob_offsets=soinfos[idx]
                if ind==337:
                    print(subs)
                    print(obs)
                if (len(subs)*len(obs)<simple_thre) or len(subs)==1 or len(obs)==1:
                    for i,sub in enumerate(subs):
                        for j,ob in enumerate(obs):
                            sub=sub.strip()
                            ob=ob.strip()
                            if sub=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':sub}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            pred_info[ind]['spo_list'].append(rel_dict)
                else:
                    for j,ob in enumerate(obs):
    #                     sub_offset=sub_offsets[i]
    #                     min_dis,ob_index=100000,-1
                        ob_offset=ob_offsets[j]
                        min_dis,sub_index=100000,-1
                        for i,sub in enumerate(subs):
                            sub=sub.strip()
                            ob=ob.strip()
                            if sub=='' or ob=='':
                                continue
                            if abs(sub_offsets[i][0]-ob_offset[0])<min_dis:
                                sub_index=i
                                min_dis=abs(sub_offsets[i][0]-ob_offset[0])
                        if sub_index!=-1:
                            ob=ob.strip()
                            if subs[sub_index]=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':subs[sub_index].strip()}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            pred_info[ind]['spo_list'].append(rel_dict)
                    #如果搞完存在sub没有ob对应上那就再来一轮
            else:
                affilate_labels=special_affilate_idx[label_pred]
                subs,obs,sub_offsets,ob_offsets=soinfos[idx]
                subs2id=dict([(sub,i) for i,sub in enumerate(subs)])
                subs_af=[[] for i in range(len(subs))]
                for f in affilate_labels:
                    if f in rels:
                        af_idx=rels[f]
                        af_token_label_pred=text_preds[af_idx]
                        af_text=text_data[ind]['text']
                        af_subs,af_obs,af_sub_offsets,af_ob_offsets=soinfos[af_idx]
                        for sub in af_subs:
                            if sub in subs:
                                subs_af[subs2id[sub]].append([af_obs,f,af_ob_offsets])
                #现在只取第一个
                if len(subs)*len(obs)<complex_thre or len(subs)==1 or len(obs)==1 :
                    #这个时候ob附属找离ob主体最近的
                    for i,sub in enumerate(subs):
                        sub_offset=sub_offsets[i]
                        for j,ob in enumerate(obs):
                            sub=sub.strip()
                            ob=ob.strip()
                            if sub=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':sub}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            if len(subs_af[i])==0:
                                    pred_info[ind]['spo_list'].append(rel_dict)
                            if len(subs_af[i])>=1:
                                ob_offset=ob_offsets[j]
                                for af_obs in subs_af[i]:
                                    af_label_pred=af_obs[1]
                                    af_min_dis,af_ob_index=100000,-1
                                    for k,(af_ob,af_offset) in enumerate(zip(af_obs[0],af_obs[2])):
                                        af_ob=af_ob.strip()
                                        if af_ob=='':
                                            continue
#                                         rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_ob})
#                                         rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))
#                                         pred_info[ind]['spo_list'].append(copy.deepcopy(rel_dict))
                                        if abs(ob_offset[0]-af_offset[0])<af_min_dis:
                                            af_ob_index=k
                                            min_dis=abs(ob_offset[0]-af_offset[0])
                                    if af_ob_index!=-1:
                                        rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_obs[0][af_ob_index].strip()})
                                        rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))

                                pred_info[ind]['spo_list'].append(rel_dict)
                else:
                    #这个时候还是直接取的第一个
                    for i,sub in enumerate(subs):

                        sub_offset=sub_offsets[i]
                        min_dis,ob_index=100000,-1
                        for j,ob in enumerate(obs):
                            if sub=='' or ob=='':
                                continue
                            if abs(sub_offset[0]-ob_offsets[j][0])<min_dis:
                                ob_index=j
                                min_dis=abs(sub_offset[0]-ob_offsets[j][0])
                        if ob_index!=-1:
                            j=ob_index
                            if subs[i]=='' or obs[j]=='':
                                continue
                            rel_dict={'object':{'@value':obs[j]},'subject':subs[i]}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            if len(subs_af[i])==0:
                                    pred_info[ind]['spo_list'].append(rel_dict)
                            if len(subs_af[i])>=1:
#                                 ob_offset=ob_offsets[j]
#                                 for af_obs in subs_af[i]:
#                                     af_label_pred=af_obs[1]
#                                     af_min_dis,af_ob_index=100000,-1
#                                     for k,(af_ob,af_offset) in enumerate(zip(af_obs[0],af_obs[2])):
#                                         if af_ob=='':
#                                             continue
#                                         if abs(ob_offset[0]-af_offset[0])<af_min_dis:
#                                             af_ob_index=k
#                                             min_dis=abs(ob_offset[0]-af_offset[0])
#                                     if af_ob_index!=-1:
#                                         rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_obs[0][af_ob_index]})
#                                         rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))

#                                 pred_info[ind]['spo_list'].append(rel_dict)
                                for af_obs in subs_af[i]:
                                    af_label_pred=af_obs[1]
                                    rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))
                                    for af_ob in af_obs[0]:
                                        rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_ob})
                                        break
                                pred_info[ind]['spo_list'].append(rel_dict) 
                    #如果结束了存在ob没对上，那么在把没对上的ob也塞进去

#### test

In [ ]:
text_preds=[np.zeros((len(test_text_data[pred_doc2doc[i]]['text']),4))for i in range(len(pred_doc_token_labels))]
for idx in tqdm(range(len(ner_pred_dataset))):
    text=ner_pred_dataset[idx][0].numpy()
    attention_mask=ner_pred_dataset[idx][1].numpy()
    token_labels=ner_pred_dataset[idx][4].numpy()
    tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index=ner_pred_dataset[idx][-1]
    token_pred=pred_token_pred[idx]
    token_pred=token_pred[1:1+len(tok_to_orig_start_index)]
    for i in range(len(token_pred)):
        begin,end=tok_to_orig_start_index[i],tok_to_orig_end_index[i]
        text_preds[pred_example2doc[idx]][begin:end+1,:]+=token_pred[i] #求和还是取平均？

In [ ]:
soinfos=[]
for idx,token_label_pred in tqdm(enumerate(text_preds)):
    soinfos.append(find_subs_obs_pointer(token_label_pred,test_text_data[pred_doc2doc[idx]]['text']))
new_soinfos=[]
for e in tqdm(soinfos):
    new_sub=[]
    new_sub_offset=[]
    for idx,t in enumerate(e[0]):
        if idx==0:
            new_sub.append(t)
            new_sub_offset.append(e[2][idx])
            continue
        if t not in e[0][:idx]:
            new_sub.append(t)
            new_sub_offset.append(e[2][idx])
    new_ob=[]
    new_ob_offset=[]
    for idx,t in enumerate(e[1]):
        if idx==0:
            new_ob.append(t)
            new_ob_offset.append(e[3][idx])
            continue
        if t not in e[1][:idx]:
            new_ob.append(t)
            new_ob_offset.append(e[3][idx])
    new_soinfos.append([new_sub,new_ob,new_sub_offset,new_ob_offset])

In [ ]:
# idx=240
# token_label_pred=text_preds[idx]
# text=test_text_data[pred_doc2doc[idx]]['text']
# print(id2rels[pred_doc_rels[idx]])
# print(find_entities(token_label_pred,text,2,3,0.2,1))
# print(find_entities(token_label_pred,text,0,1,0.3,2))
# for t,l in zip(text,token_label_pred):
#     print(t,np.array(new_id2labels)[((l>0.5)*np.arange(1,5))[np.where(((l>0.5)*np.arange(1,5)))]],l)

In [ ]:
special_rels

In [ ]:
for e in pred_info:
    flag=False
    for spo in e['spo_list']:
        if spo['predicate'] in special_rels:
            flag=True
            break
    if flag:
        print(e)

In [ ]:
doc2pred_ner_doc=[[] for i in range(len(test_text_data))]
pred_info=[{} for i in range(len(test_text_data[:]))]
for idx,e in enumerate(pred_info):
    e['text']=test_text_data[idx]['text']
    e['spo_list']=[]
for idx in range(len(pred_doc_rels[:])):
    ind=pred_doc2doc[idx]
    doc2pred_ner_doc[ind].append(idx)
flag=False
simple_thre=4
complex_thre=2
make_output_pointer(doc2pred_ner_doc,simple_thre,complex_thre,test_text_data,pred_info,new_soinfos)



#### val

In [ ]:
text_preds=[np.zeros((len(text_data[pred_doc2doc[i]]['text']),4))for i in range(len(pred_doc_token_labels))]
for idx in tqdm(range(len(ner_pred_dataset))):
    text=ner_pred_dataset[idx][0].numpy()
    attention_mask=ner_pred_dataset[idx][1].numpy()
    token_labels=ner_pred_dataset[idx][4].numpy()
    tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index=ner_pred_dataset[idx][-1]
    token_pred=pred_token_pred[idx]
    token_pred=token_pred[1:1+len(tok_to_orig_start_index)]
    for i in range(len(token_pred)):
        begin,end=tok_to_orig_start_index[i],tok_to_orig_end_index[i]
        text_preds[pred_example2doc[idx]][begin:end+1,:]+=token_pred[i] #求和还是取平均？

##### 数据分析

In [ ]:
max_sub=''
max_sub_spo=''
max_ob=''
max_ob_spo=''
sub_len=[]
ob_len=[]
subs=set()
obs=set()
for example in tqdm(text_data):
    for spo in example['spo_list']:
        sub_len.append(len(spo['subject']))
        subs.add(spo['subject'])
        obs.add(spo['object']['@value'])
        ob_len.append(len(spo['object']['@value']))
        if len(spo['subject'])>len(max_sub):
            max_sub=spo['subject']
            max_sub_spo=spo
        if len(spo['object']['@value'])>len(max_ob):
            max_ob=spo['object']['@value']
            max_ob_spo=spo
len(subs),len(obs)
# for t,l in zip(text,token_label_pred):
#     print(t,np.array(new_id2labels)[((l>0.5)*np.arange(1,5))[np.where(((l>0.5)*np.arange(1,5)))]],l)
idx=0
token_label_pred=text_preds[idx]
text=text_data[pred_doc2doc[idx]]['text']
print(id2rels[pred_doc_rels[idx]])
print(find_entities(token_label_pred,text,2,3,0.2,1))
print(find_entities(token_label_pred,text,0,1,0.3,2))
# find_subs_obs_pointer(token_label_pred,text)

In [ ]:
soinfos=[]
for idx,token_label_pred in tqdm(enumerate(text_preds)):
    soinfos.append(find_subs_obs_pointer(token_label_pred,text_data[pred_doc2doc[idx]]['text']))
new_soinfos=[]
for e in tqdm(soinfos):
    new_sub=[]
    new_sub_offset=[]
    for idx,t in enumerate(e[0]):
        if idx==0:
            new_sub.append(t)
            new_sub_offset.append(e[2][idx])
            continue
        if t not in e[0][:idx]:
            new_sub.append(t)
            new_sub_offset.append(e[2][idx])
    new_ob=[]
    new_ob_offset=[]
    for idx,t in enumerate(e[1]):
        if idx==0:
            new_ob.append(t)
            new_ob_offset.append(e[3][idx])
            continue
        if t not in e[1][:idx]:
            new_ob.append(t)
            new_ob_offset.append(e[3][idx])
    new_soinfos.append([new_sub,new_ob,new_sub_offset,new_ob_offset])

In [ ]:

doc2pred_ner_doc=[[] for i in range(len(text_data[:])-180000)]
pred_info=[{} for i in range(len(text_data[:])-180000)]
for idx,e in enumerate(pred_info):
    e['text']=text_data[180000+idx]['text']
    e['spo_list']=[]
for idx in range(len(pred_doc_rels[:])):
    ind=pred_doc2doc[idx]
    doc2pred_ner_doc[ind-180000].append(idx)
flag=False
simple_thre=4
complex_thre=2
make_output_pointer(doc2pred_ner_doc,simple_thre,complex_thre,text_data[180000:],pred_info,new_soinfos)



In [ ]:
rel_nums,pred_nums=0,0
right_num=0
for idx,(pred,label) in enumerate(zip(pred_info,text_data[180000:])):
        pred_nums+=len(pred['spo_list'])
        rel_nums+=len(label['spo_list'])
        print(idx)
        print("text:",label['text'])
        print("real_spo_list:",label['spo_list'])
        print("pred_spo_list:",pred['spo_list'])
        for e in pred['spo_list']:
            if e in label['spo_list']:
                right_num+=1
#         for e in label['spo_list']:
#             if not e in pred['spo_list']:
#                 print(idx)
#                 print(e)
#                 print(label['text'])
#                 for t in label['spo_list']:
#                     if t['predicate']==e['predicate']:
#                         print(t)
#                 print("=================================")
#                 for t in pred['spo_list']:
#                     if t['predicate']==e['predicate']:
#                         print(t)
         

In [ ]:
right_num,pred_nums,rel_nums

In [ ]:
precision=right_num/pred_nums
recall=right_num/rel_nums
f1=2*precision*recall/(precision+recall)
precision,recall,f1

### CRF preprocess

In [33]:
len(ner_pred_dataset)

18727

In [23]:
pred_results=evaluate_ner(args,ner_pred_dataset,model_ner,tokenizer)

In [35]:
#交叉ensemble 部分
pred_rel_logits=np.concatenate(pred_results[-1],axis=0)
pred_token_logits=np.concatenate(pred_results[-2],axis=0)
pred_token_pred=pred_results[2].reshape(-1,256)
pred_token_logits.shape,pred_token_pred.shape,pred_rel_logits.shape
np.save("./cross_results/test_pred_new.npy",pred_token_pred)
np.save("./cross_results/test_logit_new.npy",pred_token_logits)
np.save("./cross_results/test_rel_new.npy",pred_rel_logits)
np.save("./cross_results/pred_0.npy",pred_token_pred)
np.save("./cross_results/logit_0.npy",pred_token_logits)
np.save("./cross_results/rel_0.npy",pred_rel_logits)

((18727, 256, 9), (18727, 256), (18727, 55))

In [20]:
def split_and_idx(text,token):
    lists=[""]
    indexs=[]
    for idx,c in enumerate(text):
        if c==token:
            lists.append("")
        else:
            if lists[-1]=='':
                indexs.append(idx)
            lists[-1]+=c
    lists=list(filter(lambda x:x!="",lists))
    final_lists=[[lists[idx],indexs[idx]] for idx in range(len(lists))]
    return final_lists
def throw_part(text,index):
#CEO,子，王，等，哥 姐
#     if "校长" in text:
#         new_text,new_index=split_and_idx(text,'长')[-1]
#         return new_text,new_index+index
#     if "巨头" in text:
#         new_text,new_index=split_and_idx(text,'头')[-1]
#         return new_text,new_index+index
#     if "嘉宾" in text:
#         new_text,new_index=split_and_idx(text,'宾')[-1]
#         return new_text,new_index+index
#     if "人" in text:
#         new_text,new_index=split_and_idx(text,'人')[-1]
#         return new_text,new_index+index
#     if "经理" in text:
#         new_text,new_index=split_and_idx(text,'理')[-1]
#         return new_text,new_index+index
#     if "主席" in text:
#         new_text,new_index=split_and_idx(text,'席')[-1]
#         return new_text,new_index+index
#     if "总裁" in text:
#         new_text,new_index=split_and_idx(text,'裁')[-1]
#         return new_text,new_index+index
    if "CEO" in text:
        new_text,new_index=split_and_idx(text,'O')[-1]
        return new_text,new_index+index
    if "子" in text and "子" not in text[:2]:
        new_text,new_index=split_and_idx(text,'子')[-1]
        return new_text,new_index+index
    if "王" in text:
        new_text,new_index=split_and_idx(text,'王')[-1]
        return new_text,new_index+index
    if "哥" in text:
        new_text,new_index=split_and_idx(text,'哥')[-1]
        return new_text,new_index+index
    if "姐" in text:
        new_text,new_index=split_and_idx(text,'姐')[-1]
        return new_text,new_index+index
    if "等" in text[1:]:
        new_text,new_index=split_and_idx(text,'等')[0]
        return new_text,index
    if "身边" in text[1:]:
        new_text,new_index=split_and_idx(text,'身')[0]
        return new_text,index
    if "悉数" in text[1:]:
        new_text,new_index=split_and_idx(text,'悉')[0]
        return new_text,index
    return text,index
def find_token(text,token):
    if token=="《":
        for idx in range(len(text)-1,-1,-1):
            if text[idx]=="》":
                return -1
            if text[idx]==token:
                return idx
    if token=="》":
        for idx in range(len(text)):
            if text[idx]=="《":
                return -1
            if text[idx]==token:
                return idx
    return -1
def find_subs_obs(token_label_pred,text,logits_pred,find_sub=False,find_ob=False,find=False):
    subs=[]
    sub_offsets=[]
    obs=[]
    ob_offsets=[]
    index=0
    flag=False
    while index<token_label_pred.shape[0]:
        if token_label_pred[index]==3:
            subs.append(text[index])
            index+=1
            while(index<token_label_pred.shape[0] and (token_label_pred[index]==4 or \
                                                       (token_label_pred[index]==3 and token_label_pred[index-1]==3))):
                subs[-1]+=(text[index])
                index+=1
            sub_offsets.append([index-len(subs[-1]),index])
            if  find_sub and find_token(text[sub_offsets[-1][0]-10:sub_offsets[-1][0]],"《")!=-1 and \
                find_token(text[sub_offsets[-1][1]:sub_offsets[-1][1]+10],"》")!=-1:
                    begin_index=find_token(text[sub_offsets[-1][0]-10:sub_offsets[-1][0]],"《")+ sub_offsets[-1][0]-10
                    end_index=find_token(text[sub_offsets[-1][1]:sub_offsets[-1][1]+10],"》")+sub_offsets[-1][1]
                    if (text[begin_index:end_index+1][1:-1].strip()!=subs[-1]):
                        subs[-1]=text[begin_index:end_index+1][1:-1].strip()
                        sub_offsets[-1]=[begin_index+1,end_index]
            if find and index<token_label_pred.shape[0] and text[index]=='、':
                sub=split_and_idx(text[index+1:],"、")
                if len(sub)>1:
                    flag=True
                    for e in sub[:-1]:
                        assert (e[0]==text[index+1+e[1]:index+1+e[1]+len(e[0])])

                        if len(e[0])>20:
                            flag=False
                            break
                        for c in e[0]:
                             if is_punctuation(c):
                                flag=False
                                break
                        if not flag:
                            break
                    if flag:
                        for i,e in enumerate(sub):
                            info='校长 巨头 嘉宾 人 经理 主席 总裁'.split()
                            tag=True
                            for t in info:
                                if t in e[0]:
                                    tag=False
                                    break
                            if not tag:
                                continue

                            if e[0].startswith("等人") or "母亲" in e[0] or "妻子" in e[0]:
                                continue

                            if "等" in e[0][1:]:
                             e[0]=e[0].split("等")[0]
                            if i==len(sub)-1:
                                if len(e[0])<10 and sum(token_label_pred[index+1+e[1]:index+1+e[1]+len(e[0])])==0\
                                    and "和" not in e[0] and "之" not in e[0]:
                                    new_sub,ind=throw_part(e[0],e[1])
                                    assert new_sub==text[index+1+ind:index+1+ind+len(new_sub)]
                                    if new_sub not in subs:
                                        subs.append(new_sub)
                                        sub_offsets.append([index+1+ind,index+1+ind+len(new_sub)])

                            elif sum(token_label_pred[index+1+e[1]:index+1+e[1]+len(e[0])])==0 and "和" not in e[0]\
                                and "还有" not in e[0] and "第一" not in e[0]:

                                new_sub,ind=throw_part(e[0],e[1])
                                assert new_sub==text[index+1+ind:index+1+ind+len(new_sub)]
                                if new_sub not in subs:
                                    subs.append(new_sub)
                                    sub_offsets.append([index+1+ind,index+1+ind+len(new_sub)])        
            index-=1
        if index<token_label_pred.shape[0] and token_label_pred[index]==1:
            obs.append(text[index])
            index+=1
            while(index<token_label_pred.shape[0] and (token_label_pred[index]==2 or \
                                                       (token_label_pred[index]==1 and token_label_pred[index-1]==1))):
                obs[-1]+=(text[index])
                index+=1
            ob_offsets.append([index-len(obs[-1]),index])
            if  find_ob and find_token(text[ob_offsets[-1][0]-10:ob_offsets[-1][0]],"《")!=-1 and \
                find_token(text[ob_offsets[-1][1]:ob_offsets[-1][1]+10],"》")!=-1:
                    begin_index=find_token(text[ob_offsets[-1][0]-10:ob_offsets[-1][0]],"《")+ ob_offsets[-1][0]-10
                    end_index=find_token(text[ob_offsets[-1][1]:ob_offsets[-1][1]+10],"》")+ob_offsets[-1][1]
                    if (text[begin_index:end_index+1][1:-1].strip()!=obs[-1]):
                        obs[-1]=text[begin_index:end_index+1][1:-1].strip()
                        ob_offsets[-1]=[begin_index+1,end_index]
            if find and index<token_label_pred.shape[0] and text[index]=='、':
                sub=split_and_idx(text[index+1:],"、")
                if len(sub)>1:
                    flag=True
                    for e in sub[:-1]:
                        assert (e[0]==text[index+1+e[1]:index+1+e[1]+len(e[0])])

                        if len(e[0])>20:
                            flag=False
                            break
                        for c in e[0]:
                             if is_punctuation(c):
                                flag=False
                                break
                        if not flag:
                            break
                    if flag:

                        for i,e in enumerate(sub):
                            info='校长 巨头 嘉宾 人 经理 主席 总裁 导演'.split()
                            tag=True
                            for t in info:
                                if t in e[0]:
                                    tag=False
                                    break
                            if not tag:
                                continue

                            if e[0].startswith("等人") or "母亲" in e[0] or "妻子" in e[0]:
                                continue
                            if "等" in e[0][1:]:
                                 e[0]=e[0].split("等")[0]
                            if i==len(sub)-1:

                                if len(e[0])<10 and sum(token_label_pred[index+1+e[1]:index+1+e[1]+len(e[0])])==0:
                                    new_ob,ind=throw_part(e[0],e[1])
                                    assert new_ob==text[index+1+ind:index+1+ind+len(new_ob)]
                                    if new_ob not in obs:
                                        obs.append(new_ob)
                                        ob_offsets.append([index+1+ind,index+1+ind+len(new_ob)])

                            elif sum(token_label_pred[index+1+e[1]:index+1+e[1]+len(e[0])])==0 and len(e[0])<10:
                                new_ob,ind=throw_part(e[0],e[1])
                                assert new_ob==text[index+1+ind:index+1+ind+len(new_ob)]
                                if new_ob not in obs:
                                    obs.append(new_ob)
                                    ob_offsets.append([index+1+ind,index+1+ind+len(new_ob)])

#                         print("==================")
            index-=1
        index+=1
    new_subs=[]
    new_sub_offsets=[]
    sub_logits=[]
    for idx,sub in enumerate(subs):
        if sub in new_subs:
            continue
        new_subs.append(sub)
        new_sub_offsets.append(sub_offsets[idx])
        sub_logits.append(logits_pred[sub_offsets[idx][0]:sub_offsets[idx][1]])
    new_obs=[]
    new_ob_offsets=[]
    ob_logits=[]
    for idx,ob in enumerate(obs):
        if ob in new_obs:
            continue
        new_obs.append(ob)
        new_ob_offsets.append(ob_offsets[idx])
        ob_logits.append(logits_pred[ob_offsets[idx][0]:ob_offsets[idx][1]])
    subs=new_subs
    sub_offsets=new_sub_offsets
    obs=new_obs
    ob_offsets=new_ob_offsets
    if len(set(subs))!=len(subs) or len(set(obs))!=len(obs):
            print("subs",subs)
            print(sub_offsets)
            print("obs",obs)
            print(ob_offsets)
    if len(subs)==0:
        subs.append('')
        sub_offsets.append([0,0])
    if len(obs)==0:
        obs.append('')
        ob_offsets.append([0,0])
    return subs,obs,sub_offsets,ob_offsets,sub_logits,ob_logits
def make_output(doc2pred_ner_doc,simple_thre,complex_thre,text_data,pred_info,pred_doc_rels):
    for ind,info in tqdm(enumerate(doc2pred_ner_doc)):
        rels={}
        pred_info[ind]['spo_logits']=[]
        for idx in (info):
            rels[pred_doc_rels[idx]]=idx
        for idx in info:
            token_label_pred=text_preds[idx]
            label_pred=pred_doc_rels[idx]
            logits_pred=text_logits[idx]
            text=text_data[ind]['text']
            if label_pred in special_pass_idx:
                continue
            elif label_pred not in special_major_idx:
                find_sub_hat,find_ob_hat=False,False
                sub_type=id2schema[label_pred]['subject_type']
                for key,value in id2schema[label_pred]['object_type'].items():
                    ob_type=id2schema[label_pred]['object_type'][key]
                if sub_type in '影视作品 音乐专辑 歌曲 图书作品 电视综艺 作品 文学作品'.split(" "):
                    find_sub_hat=True
                if ob_type in '影视作品 音乐专辑 歌曲 图书作品 电视综艺 作品 文学作品'.split(" "):
                    find_ob_hat=True     

                subs,obs,sub_offsets,ob_offsets,sub_logits,ob_logits=find_subs_obs(token_label_pred,text,logits_pred,find_sub_hat,find_ob_hat)
#                 if ind==980:
#                     print(subs)
#                     print(obs)
                if (len(subs)*len(obs)<simple_thre) or len(subs)==1 or len(obs)==1:
                    for i,sub in enumerate(subs):
                        for j,ob in enumerate(obs):
                            sub=sub.strip()
                            ob=ob.strip()
                            if sub=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':sub}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            pred_info[ind]['spo_list'].append(rel_dict)
                            logits_dict={'object':ob_logits[j],'subject':sub_logits[i]}
                            pred_info[ind]['spo_logits'].append(logits_dict)
                else:
                    for j,ob in enumerate(obs):
    #                     sub_offset=sub_offsets[i]
    #                     min_dis,ob_index=100000,-1
                        ob_offset=ob_offsets[j]
                        min_dis,sub_index=100000,-1
                        for i,sub in enumerate(subs):
                            sub=sub.strip()
                            ob=ob.strip()
                            if sub=='' or ob=='':
                                continue
                            if abs(sub_offsets[i][0]-ob_offset[0])<min_dis:
                                sub_index=i
                                min_dis=abs(sub_offsets[i][0]-ob_offset[0])
                        if sub_index!=-1:
                            ob=ob.strip()
                            if subs[sub_index]=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':subs[sub_index].strip()}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            pred_info[ind]['spo_list'].append(rel_dict)
                            logits_dict={'object':ob_logits[j],'subject':sub_logits[sub_index]}
                            pred_info[ind]['spo_logits'].append(logits_dict)
                    #如果搞完存在sub没有ob对应上那就再来一轮
            else:
                affilate_labels=special_affilate_idx[label_pred]
                find_sub_hat,find_ob_hat=False,False
                sub_type=id2schema[label_pred]['subject_type']
                for key,value in id2schema[label_pred]['object_type'].items():
                    ob_type=id2schema[label_pred]['object_type'][key]
                if sub_type in '影视作品 音乐专辑 歌曲 图书作品 电视综艺 作品 文学作品'.split(" "):
                    find_sub_hat=True
                if ob_type in '影视作品 音乐专辑 歌曲 图书作品 电视综艺 作品 文学作品'.split(" "):
                    find_ob_hat=True     

                subs,obs,sub_offsets,ob_offsets,sub_logits,ob_logits=find_subs_obs(token_label_pred,text,logits_pred,find_sub_hat,find_ob_hat)
                subs2id=dict([(sub,i) for i,sub in enumerate(subs)])
                subs_af=[[] for i in range(len(subs))]
                for f in affilate_labels:
                    if f in rels:
                        af_idx=rels[f]
                        af_token_label_pred=text_preds[af_idx]
                        af_text=text_data[ind]['text']
                        sub_type=id2schema[f]['subject_type']
                        for key,value in id2schema[f]['object_type'].items():
                            ob_type=id2schema[f]['object_type'][key]
                        if sub_type in '影视作品 音乐专辑 歌曲 图书作品 电视综艺 作品 文学作品'.split(" "):
                            find_sub_hat=True
                        if ob_type in '影视作品 音乐专辑 歌曲 图书作品 电视综艺 作品 文学作品'.split(" "):
                            find_ob_hat=True     
                        af_subs,af_obs,af_sub_offsets,af_ob_offsets,af_sub_logits,af_ob_logits=find_subs_obs(af_token_label_pred,af_text,logits_pred,find_sub_hat,find_ob_hat)
                        for sub in af_subs:
                            if sub in subs:
                                subs_af[subs2id[sub]].append([af_obs,f,af_ob_offsets])
                #现在只取第一个
                if len(subs)*len(obs)<complex_thre or len(subs)==1 or len(obs)==1 :
                    #这个时候ob附属找离ob主体最近的
                    for i,sub in enumerate(subs):
                        sub_offset=sub_offsets[i]
                        for j,ob in enumerate(obs):
                            sub=sub.strip()
                            ob=ob.strip()
                            if sub=='' or ob=='':
                                continue
                            rel_dict={'object':{'@value':ob},'subject':sub}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            logits_dict={'object':ob_logits[j],'subject':sub_logits[i]}
                            if len(subs_af[i])==0:
                                    pred_info[ind]['spo_list'].append(rel_dict)
                                    pred_info[ind]['spo_logits'].append(logits_dict)
                            if len(subs_af[i])>=1:
                                ob_offset=ob_offsets[j]
                                for af_obs in subs_af[i]:
                                    af_label_pred=af_obs[1]
                                    af_min_dis,af_ob_index=100000,-1
                                    for k,(af_ob,af_offset) in enumerate(zip(af_obs[0],af_obs[2])):
                                        af_ob=af_ob.strip()
                                        if af_ob=='':
                                            continue
#                                         rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_ob})
#                                         rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))
#                                         pred_info[ind]['spo_list'].append(copy.deepcopy(rel_dict))
                                        if abs(ob_offset[0]-af_offset[0])<af_min_dis:
                                            af_ob_index=k
                                            min_dis=abs(ob_offset[0]-af_offset[0])
                                    if af_ob_index!=-1:
                                        rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_obs[0][af_ob_index].strip()})
                                        rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))

                                pred_info[ind]['spo_list'].append(rel_dict)
                                pred_info[ind]['spo_logits'].append(logits_dict)
                else:
                    #这个时候还是直接取的第一个
                    for i,sub in enumerate(subs):

                        sub_offset=sub_offsets[i]
                        min_dis,ob_index=100000,-1
                        for j,ob in enumerate(obs):
                            if sub=='' or ob=='':
                                continue
                            if abs(sub_offset[0]-ob_offsets[j][0])<min_dis:
                                ob_index=j
                                min_dis=abs(sub_offset[0]-ob_offsets[j][0])
                        if ob_index!=-1:
                            j=ob_index
                            if subs[i]=='' or obs[j]=='':
                                continue
                            rel_dict={'object':{'@value':obs[j]},'subject':subs[i]}
                            rel_dict.update(copy.deepcopy(id2schema[label_pred]))
                            logits_dict={'object':ob_logits[j],'subject':sub_logits[i]}
                            if len(subs_af[i])==0:
                                    pred_info[ind]['spo_list'].append(rel_dict)
                                    pred_info[ind]['spo_logits'].append(logits_dict)
                            if len(subs_af[i])>=1:
#                                 ob_offset=ob_offsets[j]
#                                 for af_obs in subs_af[i]:
#                                     af_label_pred=af_obs[1]
#                                     af_min_dis,af_ob_index=100000,-1
#                                     for k,(af_ob,af_offset) in enumerate(zip(af_obs[0],af_obs[2])):
#                                         if af_ob=='':
#                                             continue
#                                         if abs(ob_offset[0]-af_offset[0])<af_min_dis:
#                                             af_ob_index=k
#                                             min_dis=abs(ob_offset[0]-af_offset[0])
#                                     if af_ob_index!=-1:
#                                         rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_obs[0][af_ob_index]})
#                                         rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))

#                                 pred_info[ind]['spo_list'].append(rel_dict)
                                for af_obs in subs_af[i]:
                                    af_label_pred=af_obs[1]
                                    rel_dict['object_type'].update(copy.deepcopy(id2schema[af_label_pred]['object_type']))
                                    for af_ob in af_obs[0]:
                                        rel_dict['object'].update({id2rels[af_label_pred].split("_")[1]:af_ob})
                                        break
                                pred_info[ind]['spo_list'].append(rel_dict) 
                                pred_info[ind]['spo_logits'].append(logits_dict)
                    #如果结束了存在ob没对上，那么在把没对上的ob也塞进去

#### val

In [54]:
text_preds=[np.zeros((len(text_data[ner_doc2doc[i]]['text']),len(id2labels)))for i in range(len(ner_doc_token_labels))]
text_logits=[np.zeros((len(text_data[ner_doc2doc[i]]['text']),len(id2labels)))for i in range(len(ner_doc_token_labels))]
for idx in tqdm(range(len(ner_examples))):
    example=ner_examples[idx]
    text=example['input_ids'].numpy()
    attention_mask=example['attention_mask'].numpy()
    token_labels=example['labels'].numpy()
    tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index=example['token2doc']
    token_pred=pred_token_pred[idx].astype(np.int)
    token_pred=token_pred[1:1+len(tok_to_orig_start_index)]
    logits_pred=pred_token_logits[idx][1:1+len(tok_to_orig_start_index)]
    for i in range(len(token_pred)):
        begin,end=tok_to_orig_start_index[i],tok_to_orig_end_index[i]
        text_logits[example2doc[idx]][begin:end+1]=logits_pred[i]
        if token_pred[i]!=0:
            text_preds[example2doc[idx]][begin:end+1,token_pred[i]]+=1
            if token_pred[i]==2 or token_pred[i]==4:
                B_index=-1
                for k in range(0,-begin-1,-1):
                    if text_preds[example2doc[idx]][k+begin,token_pred[i]-1]!=0:
                        B_index=k+begin
                        break
#                 if idx==489:
#                     print(i,B_index,text_data[180321]['text'][B_index])
                if B_index!=-1:
                    for e in range(B_index+1,begin):
                        if text_preds[example2doc[idx]][e,token_pred[i]]==0:
                            text_preds[example2doc[idx]][e,token_pred[i]]+=1
                else:
                    print(idx)
#         if idx==489:
#             print(text_preds[487])
text_preds=[(e!=0).astype(np.int32).argmax(axis=-1) for e in text_preds]

In [99]:
text_data=np.array(text_data)
ind2idx=dict([(e,idx) for idx,e in enumerate(range(len(text_data)))])
doc2pred_ner_doc=[[] for i in range(len(text_data))]
pred_info=[{} for i in range(len(text_data))]
for idx,e in tqdm(enumerate(pred_info)):
    e['text']=text_data[idx]['text']
    e['spo_list']=[]
for idx in range(len(ner_doc_rels[:])):
    ind=ner_doc2doc[idx]
    doc2pred_ner_doc[ind].append(idx)
flag=False
simple_thre=1000
complex_thre=1000   #现在是要尽可能地多生成实体对
make_output(doc2pred_ner_doc,simple_thre,complex_thre,text_data,pred_info,ner_doc_rels)



/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:250: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [35]:
for idx in tqdm(range(len(pred_info))):
    pred_info[idx]['cls_logits']=list(pred_cls_logits[idx])
    for spo in pred_info[idx]['spo_logits']:
        spo['object']=spo['object'].tolist()
        spo['subject']=spo['subject'].tolist()
    pred_info[idx]['plan_spo_list']=text_data[idx]['plan_spo_list']

In [104]:
with open("./cross_results/train_info.json","w") as w:
    for e in tqdm(pred_info):
        w.write(json.dumps(e, ensure_ascii=False)+'\n')


In [139]:
rel_nums,pred_nums=0,0
right_num=0
for idx,(pred,label) in tqdm(enumerate(zip(pred_info,text_data))):
        pred_nums+=len(pred['spo_list'])
        rel_nums+=len(label['spo_list'])
#         print(idx)
#         print("text:",label['text'])
#         print("real_spo_list:",label['spo_list'])
#         print("pred_spo_list:",pred['spo_list'])
        for e in pred['spo_list']:
            if e in label['spo_list']:
                right_num+=1
#         for e in label['spo_list']:
#             if not e in pred['spo_list']:
#                 print(idx)
#                 print(e)
#                 print(label['text'])
#                 for t in label['spo_list']:
#                     if t['predicate']==e['predicate']:
#                         print(t)
#                 print("=================================")
#                 for t in pred['spo_list']:
#                     if t['predicate']==e['predicate']:
#                         print(t)
         

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [140]:
right_num,pred_nums,rel_nums

(278336, 401495, 348534)

In [141]:
precision=right_num/pred_nums
recall=right_num/rel_nums
f1=2*precision*recall/(precision+recall)
precision,recall,f1

(0.6932489819300365, 0.7985906683422564, 0.7422006349087835)

#### test

In [246]:
#交叉ensemble 部分
# preds=[]
# logits=[]
# rel_logits=[]
# for idx in range(3):
#     preds.append(np.load("./cross_results/test_pred_"+str(idx)+".npy"))
#     logits.append(np.load("./cross_results/test_logit_"+str(idx)+".npy"))
#     rel_logits.append(np.load("./cross_results/test_rel_"+str(idx)+".npy"))
# preds.append(np.load("./cross_results/test_pred_full.npy"))
# logits.append(np.load("./cross_results/test_logit_full.npy"))
# rel_logits.append(np.load("./cross_results/test_rel_full.npy"))
# preds=np.stack(preds,axis=2)

# pred_token_pred=np.zeros((len(pred_examples),256))
# for idx,pred in tqdm(enumerate(preds)):
#     for j,token in enumerate(pred):
#         token_pred=np.argmax(np.bincount(token))
#         if token_pred==0 and (token>0).sum()>=2 and ((not (np.bincount(token)>1).any()) or (np.bincount(token)==2).sum()==2):
#             if (not (np.bincount(token)>1).any()):
#                 print(token)
#                 token_pred=token[-1]
#                 print(token_pred)
#             else:
#                 token_pred=token.sum()//2

#         pred_token_pred[idx][j]=token_pred
# pred_token_pred=pred_token_pred.astype(np.int)  

In [43]:
text_preds=[np.zeros((len(test_text_data[pred_doc2doc[i]]['text']),len(id2labels)))for i in range(len(pred_doc_token_labels[:]))]
text_logits=[np.zeros((len(test_text_data[pred_doc2doc[i]]['text']),len(id2labels)))for i in range(len(pred_doc_token_labels))]
for idx in tqdm(range(len(ner_pred_dataset))):
    text=ner_pred_dataset[idx][0].numpy()
    attention_mask=ner_pred_dataset[idx][1].numpy()
    token_labels=ner_pred_dataset[idx][4].numpy()
    tok_to_orig_start_index,tok_to_orig_end_index,orig_to_tok_index=ner_pred_dataset[idx][-1]
    token_pred=pred_token_pred[idx]
    token_pred=token_pred[1:1+len(tok_to_orig_start_index)]
    logits_pred=pred_token_logits[idx][1:1+len(tok_to_orig_start_index)]
    for i in range(len(token_pred)):
        begin,end=tok_to_orig_start_index[i],tok_to_orig_end_index[i]
        text_logits[pred_example2doc[idx]][begin:end+1]=logits_pred[i]
        if token_pred[i]!=0:
            text_preds[pred_example2doc[idx]][begin:end+1,token_pred[i]]+=1
            if token_pred[i]==2 or token_pred[i]==4:
                B_index=-1
                for k in range(0,-begin-1,-1):
                    if text_preds[pred_example2doc[idx]][k+begin,token_pred[i]-1]!=0:
                        B_index=k+begin
                        break
#                 if idx==489:
#                     print(i,B_index,text_data[180321]['text'][B_index])
                if B_index!=-1:
                    for e in range(B_index+1,begin):
                        if text_preds[pred_example2doc[idx]][e,token_pred[i]]==0:
                            text_preds[pred_example2doc[idx]][e,token_pred[i]]+=1
#                 else:
#                     print(idx)
#         if idx==489:
#             print(text_preds[487])
text_preds=[(e!=0).astype(np.int32).argmax(axis=-1) for e in text_preds]

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:

doc2pred_ner_doc=[[] for i in range(len(test_text_data))]
pred_info=[{} for i in range(len(test_text_data))]
for idx,e in enumerate(pred_info):
    e['text']=test_text_data[idx]['text']
    e['spo_list']=[]
for idx in range(len(pred_doc_rels)):
    ind=pred_doc2doc[idx]
    doc2pred_ner_doc[ind].append(idx)
flag=False
simple_thre=4
complex_thre=4
make_output(doc2pred_ner_doc,simple_thre,complex_thre,test_text_data,pred_info,pred_doc_rels)

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:250: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [46]:
for idx in tqdm(range(len(pred_info))):
#     pred_info[idx]['cls_logits']=list(pred_cls_logits[idx])
    for spo in pred_info[idx]['spo_logits']:
        spo['object']=spo['object'].tolist()
        spo['subject']=spo['subject'].tolist()
    pred_info[idx]['plan_spo_list']=test_text_data[idx]['plan_spo_list']

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


##### 分类器筛选

In [33]:
import lightgbm as lgb
lgb_clf=lgb.Booster(model_file="./model.txt")

In [48]:
train_data=pred_info

In [49]:
train_X=[]
train_Y=[]
full_rel_nums=np.zeros((len(train_data)))
rel_index=[]
for i,e in tqdm(enumerate(train_data)):
#     cls_logits=np.array(e['cls_logits'])
    triple_num=len(e['spo_list'])
    rels=set()
    entities_num=0
    for spo in e['spo_list']:
        rels.add(spo['predicate'])
        entities_num+=(1+len(spo['object']))
    rel_num=len(rels)
    for idx in range(len(e['spo_list'])):
        spo=e['spo_list'][idx]
        spo_logits=e['spo_logits'][idx]
        sub_ob_logits=np.concatenate([np.array(spo_logits['subject'])[0],np.array(spo_logits['subject'])[-1],\
                              np.array(spo_logits['object'])[0],np.array(spo_logits['object'])[-1]])
        predicate=spo['predicate'] if spo['predicate'] not in special_rels else spo['predicate']+'_@value'
        sub=spo['subject']
        sub_len=len(sub)
        ob=spo['object']['@value']
        ob_len=len(ob)
        show_time=0
        for plan_spo in e['plan_spo_list']:
            if plan_spo['predicate']==predicate and plan_spo['subject']==sub and plan_spo['object']==ob:
                show_time+=1
                break
        feature=np.concatenate([sub_ob_logits,np.array([show_time,rel_num,entities_num,triple_num]),np.array([sub_len,ob_len])])
        label=1
        train_X.append(feature)
        train_Y.append(label)
        rel_index.append([i,spo])
train_X=np.array(train_X)
train_Y=np.array(train_Y)

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [50]:
train_X.shape

(23083, 42)

In [51]:
val_pre=lgb_clf.predict(train_X[:])

In [52]:
((val_pre[:]>0.1)==1).sum(),(train_Y[:]==1).sum() #P

(22326, 23083)

In [53]:
num=0
for idx in tqdm(range(len(val_pre))):
    if val_pre[idx]<=0.1:
        text_idx=rel_index[idx][0]
        spo=rel_index[idx][1]
        pred_info[text_idx]['spo_list'].remove(spo)
#         print(spo)
#         print(pred_info[text_idx]['text'])
#         num+=1
#         if num>=20:
#             break
#         print("="*10)

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [55]:
for idx in range(len(pred_info)):
    pred_info[idx].pop('spo_logits')
#     pred_info[idx].pop('cls_logits')
    pred_info[idx].pop('plan_spo_list')

### test后处理

In [56]:
nums=0
for e in pred_info:
    nums+=len(e['spo_list'])
nums

22326

In [57]:
with open("./predict_res.json","w") as w:
    for e in pred_info:
        w.write(json.dumps(e, ensure_ascii=False)+'\n')


In [81]:
pred_data=[]
pred_num=0
with open("./frzhu/test2_res.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        pred_data.append(json.loads(d))
        pred_num+=len(pred_data[-1]['spo_list'])
pred_num

194917

##### 知识库匹配

In [ ]:
# similarity_matrix=np.zeros((len(id2rels),len(id2rels)))
# for i in range(len(id2rels)):
#     for j in range(len(id2rels)):
#         similarity_matrix[i][j]=wv_from_text.similarity(id2rels[i].split("_")[0],id2rels[j].split("_")[0])
# df=pd.DataFrame(similarity_matrix,index=id2rels,columns=id2rels)
# infos=[]
# for idx,col in enumerate(df.columns):
#     temp=df.iloc[:,idx:idx+1].sort_values(by=col,ascending=False)
#     new_temp=temp[temp[col]<1-1e-3]
#     infos.append(new_temp)

In [78]:
id2pair_pre='编剧 导演 主演 作词 作曲 制片人 父亲 母亲 妻子 丈夫'.split(" ")
pair_pre_list=[['编剧','导演'],['编剧','导演','主演','制片人'],['导演','主演'],['作词', '作曲'],['作词', '作曲'],['导演','制片人'],\
              ['父亲','母亲'],['父亲','母亲'],['妻子','丈夫'],['妻子','丈夫']]
pair_pre2id=dict([(e,idx) for idx,e in enumerate(id2pair_pre)])
subs_predicate=dict()
obs_predicate=dict()
for idx,e in tqdm(enumerate(text_data)):
    for spo in e['spo_list']:
        pred_list=subs_predicate.get(spo['subject'],dict([(e,0) for idx,e in enumerate(id2pair_pre)]))
        if spo['predicate'] in pred_list:
            pred_list[spo['predicate']]+=1
            subs_predicate[spo['subject']]=pred_list
        pred_list=obs_predicate.get(spo['object']['@value'],dict([(e,0) for idx,e in enumerate(id2pair_pre)]))
        if spo['predicate'] in pred_list:
            pred_list[spo['predicate']]+=1
            obs_predicate[spo['object']['@value']]=pred_list

/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [60]:
if not os.path.exists("./dataset/test1_data/new_test1_data.json"):
    spo_corpus=[]
    for idx,e in tqdm(enumerate(text_data)):
        for spo in e['spo_list']:
            if spo['predicate'] in special_rels:
                for key,value in spo['object'].items():
                    rel=spo['predicate']+"_"+key
                    ob=value
                    sub=spo['subject']
                    sample={'predicate':rel,'object':ob,'subject':sub}
                    spo_corpus.append(sample)
            else:
                for key,value in spo['object'].items():
                    rel=spo['predicate']
                    ob=value
                    sub=spo['subject']
                    sample={'predicate':rel,'object':ob,'subject':sub}
                    spo_corpus.append(sample)
    for idx,e in tqdm(enumerate(test_text_data)):
        self_corpus=[]
        for spo in spo_corpus:
            if spo['subject'] in e['text']  and spo['object'] in e['text'] :
                if (spo['subject']!='' or spo['object']!='') and spo not in self_corpus:
                    self_corpus.append(spo)
        e['plan_spo_list']=e.get('plan_spo_list',[])+self_corpus
    with open("./dataset/test1_data/new_test1_data.json","w") as w:
        for e in test_text_data:
            w.write(json.dumps(e, ensure_ascii=False)+'\n')
else:
    print("loading...")
    new_test_text_data=[]
    with open("./dataset/test1_data/new_test1_data.json","r") as r:
        raw_data=r.readlines()
        for d in raw_data:
            new_test_text_data.append(json.loads(d))


loading...


In [79]:
if not os.path.exists("./dataset/test2_data/new_test2_data.json"):
    spo_corpus=[]
    for idx,e in tqdm(enumerate(text_data)):
        for spo in e['spo_list']:
            if spo['predicate'] in special_rels:
                for key,value in spo['object'].items():
                    rel=spo['predicate']+"_"+key
                    ob=value
                    sub=spo['subject']
                    sample={'predicate':rel,'object':ob,'subject':sub}
                    spo_corpus.append(sample)
            else:
                for key,value in spo['object'].items():
                    rel=spo['predicate']
                    ob=value
                    sub=spo['subject']
                    sample={'predicate':rel,'object':ob,'subject':sub}
                    spo_corpus.append(sample)
    for idx,e in tqdm(enumerate(test2_text_data)):
        self_corpus=[]
        for spo in spo_corpus:
            if spo['subject'] in e['text']  and spo['object'] in e['text'] :
                if (spo['subject']!='' or spo['object']!='') and spo not in self_corpus:
                    self_corpus.append(spo)
        e['plan_spo_list']=e.get('plan_spo_list',[])+self_corpus
    with open("./dataset/test2_data/new_test2_data.json","w") as w:
        for e in test2_text_data:
            w.write(json.dumps(e, ensure_ascii=False)+'\n')
else:
    print("loading...")
    new_test2_text_data=[]
    with open("./dataset/test2_data/new_test2_data.json","r") as r:
        raw_data=r.readlines()
        for d in raw_data:
            new_test2_text_data.append(json.loads(d))


loading...


###### 知识库长词替换及空缺填充

In [82]:
def cross_contain(a,b):
    if a.find(b)!=-1 or b.find(a)!=-1:
        return True
    return False
for idx in tqdm(range(len(pred_data))):
    for spo in new_test2_text_data[idx]['plan_spo_list']:
            if spo['predicate'].split("_")[0] not in special_rels:
                        ob_type='@value'
            else:
                        ob_type=spo['predicate'].split("_")[1]
            rel_dict={'object':{ob_type:spo['object']},'subject':spo['subject']}
            rel_dict.update(id2schema[rels2id[spo['predicate']]])
            if rel_dict not in pred_data[idx]['spo_list']:
                rel=rel_dict['predicate']

                flag=True
                for t in pred_data[idx]['spo_list']:
                    if ob_type not in t['object']:
                        continue
                    if t['predicate']==rel and rel in ['丈夫','妻子'] and t['subject']==rel_dict['object']['@value']\
                                and t['object']['@value']==rel_dict['subject']:
                                t['subject']=rel_dict['object']['@value']
                                t['object']['@value']=rel_dict['subject']
                    if t['predicate']==rel and cross_contain(rel_dict['subject'],t['subject']) \
                    and cross_contain(rel_dict['object'][ob_type],t['object'][ob_type]):
                            flag=False
                            if len(rel_dict['subject'])+len(rel_dict['object'][ob_type])>len(t['subject'])+len(t['object'][ob_type]):
                                    t['subject']=rel_dict['subject']
                                    t['object'][ob_type]=rel_dict['object'][ob_type]
                            break
                if flag:
                    # 
                    #'董事长',,'歌手',
                    if rel in ['父亲','母亲','校长','作者','董事长','主持人','歌手','主演']\
                    and  rel_dict not in pred_data[idx]['spo_list'] \
                    and len(rel_dict['subject'])!=1 and len(rel_dict['object']['@value'])!=1:
#                         print(rel_dict)
#                         print(pred_data[idx])
#                         print("======================")
                            pred_data[idx]['spo_list'].append(copy.deepcopy(rel_dict))
#     if len(pred_data[idx]['spo_list'])==0:
#         for spo in new_test2_text_data[idx]['plan_spo_list']:
#                 if spo['predicate'].split("_")[0] not in special_rels:
#                             ob_type='@value'
#                             rel_dict={'object':{ob_type:spo['object']},'subject':spo['subject']}
#                             rel_dict.update(id2schema[rels2id[spo['predicate']]])
#                             if ob_type=='@value':
#                                 pred_data[idx]['spo_list'].append(rel_dict)
#                 else:
#                     ob_type=spo['predicate'].split("_")[1]
#                     if ob_type=='@value':
#                         rel_dict={'object':{ob_type:spo['object']},'subject':spo['subject']}
#                         rel_dict.update(id2schema[rels2id[spo['predicate']]])
#                         print(rel_dict)
#                         flag=True
#                         for e in new_test2_text_data[idx]['plan_spo_list']:
#                             if e['predicate'].split("_")[0] in special_rels:
#                                 e_ob_type=e['predicate'].split("_")[1]
#                                 if e['predicate'].split("_")[0]==spo['predicate'].split("_")[0] and e_ob_type!='@value':
#                                     print("affiliate",e)
#                                     flag=False
#                                     rel_dict['object'].update({e_ob_type:e['object']})
#                                     rel_dict['object_type'].update(copy.deepcopy(id2schema[rels2id[e['predicate']]]['object_type']))
#                                     print(rel_dict)
#                                     pred_data[idx]['spo_list'].append(copy.deepcopy(rel_dict))
#                         if flag:
#                             print(rel_dict)
#                             pred_data[idx]['spo_list'].append(copy.deepcopy(rel_dict))
#         print("==================")
                        


/home/huangweilin/anaconda3/envs/fjw/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [1]:
for idx in range(len(pred_data)):
    remove_spo=[]
    for j,t in enumerate(pred_data[idx]['spo_list']):
        for k,spo in enumerate(pred_data[idx]['spo_list']):
            if k==j:
                continue
            if t['predicate'] not in special_rels and  t['predicate']==spo['predicate'] and cross_contain(spo['subject'],t['subject']) \
            and cross_contain(spo['object']['@value'],t['object']['@value']):
                    if len(spo['subject'])+len(spo['object']['@value'])>len(t['subject'])+len(t['object']['@value']):
                                        remove_spo.append(t)
                    else:
                        remove_spo.append(spo)
    if len(remove_spo)>0:
        print(pred_data[idx]['spo_list'])
        for e in remove_spo:
            print(e)
            if e in pred_data[idx]['spo_list']:
                pred_data[idx]['spo_list'].remove(e)
        print("after")
        print(pred_data[idx]['spo_list'])

        print("===========================")

###### 一致性检验

In [2]:
for idx in tqdm(range(len(pred_data))):
    removing_spo=[]
    for spo in pred_data[idx]['spo_list']:
        try:
            pred_list=obs_predicate.get(spo['object']['@value'],dict())
        except:
            print(spo)
            pred_data[idx]['spo_list'].remove(spo)
            continue
        if len(pred_list)>0 and spo['predicate']  in pred_list:
            if pred_list[spo['predicate']]==0:
                num=0
                for e in pair_pre_list[pair_pre2id[spo['predicate']]]:
                    if e!=spo['predicate']:
                        num+=pred_list[e]
                if (spo['predicate'] in ['编剧','导演','主演','作词','作曲','制片人'] and num>13) or (spo['predicate'] in ['父亲','母亲','妻子','丈夫'] and num>0):
                      print(idx)
                      print(spo)
                      print(pred_list)
                      print(pred_data[idx])
                      removing_spo.append(spo)
                      print(pred_data[idx])
    for spo in removing_spo:
        pred_data[idx]['spo_list'].remove(spo)



In [3]:
for idx,e in enumerate(pred_data):
    for spo in e['spo_list']:
        while spo['subject'][0]=='《' and ( spo['subject'][-1]=='》' or  spo['subject'].find("》")==-1):
            print("sub",spo['subject'])
            spo['subject']=spo['subject'].replace("《","").replace("》","")
            print("sub",spo['subject'])         
            break
        poped_keys=[]
        for key,value in spo['object'].items():
            if len(value)==0:
                print(spo)
                poped_keys.append(key)
            if len(poped_keys)>0:
                for t in poped_keys:
                    print(t)
                    spo['object'].pop(t)
                    spo['object_type'].pop(t)
                print(idx,pred_data[idx])
                break
            while value[0]=='《' and (value[-1]=='》' or value.find("》")==-1):
                print("ob",key,value)
                spo['object'][key]=value.replace("《","").replace("》","")
                print("ob",spo['object'][key])        
                break

In [86]:
with open("./frzhu/test2_res kg.json","w") as w:
    for e in pred_data:
        w.write(json.dumps(e, ensure_ascii=False)+'\n')

### 对比

In [104]:
pred_data_1=[]
pred_num_1=0
with open("./test2 predict_res kg.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        pred_data_1.append(json.loads(d))
        pred_num_1+=len(pred_data_1[-1]['spo_list'])
pred_num_1

309022

In [101]:
pred_data_2=[]
pred_num_2=0
with open("./frzhu/test2_res kg.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        pred_data_2.append(json.loads(d))
        pred_num_2+=len(pred_data_2[-1]['spo_list'])
pred_num_2

226369

In [94]:
def cross_contain(a,b):
    if a.find(b)!=-1 or b.find(a)!=-1:
        return True
    return False
for idx in range(len(pred_data_1)):
    for spo in pred_data_2[idx]['spo_list']:
        flag=True
        for temp in pred_data_1[idx]['spo_list']:
            if cross_contain(temp['subject'],spo['subject']) or cross_contain(temp['object']['@value'],spo['object']['@value'])\
              or len(spo['subject'])>10 or len(spo['object']['@value'])>10:
                flag=False
                break
        if flag:
            pred_data_1[idx]['spo_list'].append(spo)
    

In [95]:
with open("./test2 predict_res kg combine.json","w") as w:
    for e in pred_data_1:
        w.write(json.dumps(e, ensure_ascii=False)+'\n')

In [102]:
pred_data_2=[]
pred_num_2=0
with open("./test2 predict_res kg combine.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        pred_data_2.append(json.loads(d))
        pred_num_2+=len(pred_data_2[-1]['spo_list'])
pred_num_2

310078

In [4]:
for idx in range(len(pred_data_1)):
    for spo in pred_data_2[idx]['spo_list']:
        if spo['predicate']  in special_rels:
            print(spo)
#     if not len(pred_data_1[idx]['spo_list'])==len(pred_data_2[idx]['spo_list']):
#         print(idx)
#         print(pred_data_1[idx]['text'])
#         print(pred_data_1[idx]['spo_list'])
#         print("===========================")
#         print(pred_data_2[idx]['spo_list'])
#         print("-------------------------------")